In [0]:
from datetime import datetime
now = datetime.now()
now.strftime("%H:%M:%S")

'09:11:46'

In [0]:
import os
import docx
from pyspark.sql.types import *

max_excel_row = 1000
max_rows_permitted = 5000000

In [0]:
%run ./__Utilities__.py 

In [0]:

dbutils.widgets.removeAll()
dbutils.widgets.dropdown("Env","DEV",["DEV","QA","PROD","PROD_DR"])
dbutils.widgets.dropdown("Comparison Options","Netezza Vs Synapse-(7 Years Transactional),Netizza Vs Gold-(Full)",["Netezza Vs Synapse-(7 Years Transactional),Netizza Vs Gold-(Full)","Netezza Vs Synapse-Non Transactional Full,Netizza to Gold-(Full)"])
dbutils.widgets.text("Log_Id","1")
dbutils.widgets.dropdown("Load Type","Historical",["Historical","Non-Historical"])
dbutils.widgets.dropdown("Report Type","Summary Report",["Summary Report","Detailed Report"])
dbutils.widgets.dropdown("Level", "Job",["Job","Table"])
dbutils.widgets.text("Job", "JB_HRM_BUILD_SG")
dbutils.widgets.text("Tables", "WAREHOUSE.CONTACTS_HIST,WAREHOUSE.ADDRESS_HIST,WAREHOUSE.SKU_PRC_HIST,STAGING.TICKET_ITEM_LINK,STAGING.INVENTORY_LINK")


In [0]:
Env = dbutils.widgets.get("Env")
TR_NonTR= dbutils.widgets.get("Comparison Options")
Log_Id = dbutils.widgets.get("Log_Id")
Load_Type = dbutils.widgets.get("Load Type")
RptType=dbutils.widgets.get("Report Type")
Level = dbutils.widgets.get("Level")

if Level == 'Job':
    Job = str(dbutils.widgets.get("Job")).upper()
    if Job == '':
        print("No job name entered for validation. Enter a job name")
        dbutils.notebook.exit('No job name entered for validation')
    else:
        ## Check if job name is valid
        job_list_query = '''(select distinct UPPER(Job) as job from dbo.DataflowSourceTargets)Config'''
        job_list_df = readfromdbSQL(Env, job_list_query)
        job_list = [row[0] for row in job_list_df.collect()]
        if Job not in job_list:
            print('{Job} is not a valid job entry. Please enter a valid job name')
            dbutils.notebook.exit('Invalid job entry')
        else:
            print(f"Performing validation for job: {Job}")

elif Level == 'Table':
    Tables = str(dbutils.widgets.get("Tables"))
    if Tables == '':
        print("No tables listed for validation. Enter a list of tables to be configured")
        dbutils.notebook.exit('No tables listed for validation')
    else:
        print(f"Performing validation on the specified tables")

if TR_NonTR == 'Netezza Vs Synapse-Non Transactional Full,Netizza to Gold-(Full)':
    PL_TL='Full_GS'
elif TR_NonTR == 'Netezza Vs Synapse-(7 Years Transactional),Netizza Vs Gold-(Full)':
    PL_TL='7GS'
   
print(PL_TL,RptType)
print(Load_Type)


Performing validation for job: JOB_DELTA_REFERENCE
Full_GS Summary Report
Non-Historical


In [0]:
from datetime import datetime
mountSilver = "/mnt/silver/"
today = date.today()
#print(today)
dt = date.today()
ct=datetime.now()
t=str(ct)[11:19]
times = t.replace(":", "-")

In [0]:
if Level == 'Job':
    path = f"/dbfs/mnt/silver/DPT_TEMP/{dt}/{times}->{Job}/"
elif Level == 'Table':
    path = f"/dbfs/mnt/silver/DPT_TEMP/{dt}/{times}->Tables/"
os.makedirs(path, exist_ok=True)
print(path)

/dbfs/mnt/silver/DPT_TEMP/2023-07-26/09-11-48->JOB_DELTA_REFERENCE/


In [0]:
def save_log(doc):
    doc.save('log.docx')
    shutil.copy2('log.docx',path+'log.docx')
    os.remove('log.docx')
    return

In [0]:
## Create log header
doc = docx.Document()
if Level == 'Job':
    doc.add_heading(f'Logs for {RptType} for {Load_Type} load of {Job} run in {Env} environment on {dt}', 0)
if Level == 'Table':
    doc.add_heading(f'Logs for {RptType} for {Load_Type} load of specified tables run in {Env} environment on {dt}', 0)
doc.add_heading(f'Comparison Option: {TR_NonTR}', 1)
doc.add_paragraph('')

In [0]:
if Level == 'Job':
    query = """
    (select UPPER(Job) as Job, DataSource as SchemaName,EntityName as TableName,
    concat(TRIM(DataSource),'.',TRIM(EntityName)) as FullTableName
    from dbo.DataflowSourceTargets
    where UPPER(Job) = '{}' and 
        SourceType = 'Netezza'
        and EntityUsage = 'Target'
        group by Job,EntityName,DataSource
        )Config""".format(Job)
    print(query)


    (select UPPER(Job) as Job, DataSource as SchemaName,EntityName as TableName,
    concat(TRIM(DataSource),'.',TRIM(EntityName)) as FullTableName
    from dbo.DataflowSourceTargets
    where UPPER(Job) = 'JOB_DELTA_REFERENCE' and 
        SourceType = 'Netezza'
        and EntityUsage = 'Target'
        group by Job,EntityName,DataSource
        )Config


In [0]:
if Level == 'Job':
    job_tables = readfromdbSQL(Env, query)
    display(job_tables)

    if job_tables.count() == 0:
        print(f"No table configured for job {Job}")
        dbutils.notebook.exit(f"No table configured for job {Job}")

Job,SchemaName,TableName,FullTableName
JOB_DELTA_REFERENCE,Warehouse,ADDRESS,Warehouse.ADDRESS
JOB_DELTA_REFERENCE,WAREHOUSE,ADDRESS_HIST_NTZSTAGE,WAREHOUSE.ADDRESS_HIST_NTZSTAGE
JOB_DELTA_REFERENCE,Staging,ADDRESS_LINK,Staging.ADDRESS_LINK
JOB_DELTA_REFERENCE,Staging,BUDGET_DEPARTMENT_XREF,Staging.BUDGET_DEPARTMENT_XREF
JOB_DELTA_REFERENCE,Staging,BUYER,Staging.BUYER
JOB_DELTA_REFERENCE,Warehouse,BUYER,Warehouse.BUYER
JOB_DELTA_REFERENCE,Warehouse,BUYER_HIST,Warehouse.BUYER_HIST
JOB_DELTA_REFERENCE,Staging,CBSA,Staging.CBSA
JOB_DELTA_REFERENCE,Warehouse,CBSA,Warehouse.CBSA
JOB_DELTA_REFERENCE,Warehouse,CLASS_HIST,Warehouse.CLASS_HIST


In [0]:
if Level == 'Job':
    tables_list = job_tables.select("FullTableName").rdd.flatMap(lambda x: x).collect()
    tables_list = "("+",".join(["'"+str(x)+"'" for x in tables_list])+")"
    print(tables_list)

('Warehouse.ADDRESS','WAREHOUSE.ADDRESS_HIST_NTZSTAGE','Staging.ADDRESS_LINK','Staging.BUDGET_DEPARTMENT_XREF','Staging.BUYER','Warehouse.BUYER','Warehouse.BUYER_HIST','Staging.CBSA','Warehouse.CBSA','Warehouse.CLASS_HIST','STAGING.CLASS_NTZSTAGE','WAREHOUSE.CLASS_NTZSTAGE','Staging.CLUB_CALL_LIST_CUSTOMERS_CURRENT','Staging.CLUB_CONTACT','Warehouse.COLOR_HIST','STAGING.COLOR_NTZSTAGE','WAREHOUSE.COLOR_NTZSTAGE','Staging.COMPANY','WAREHOUSE.COMPANY_NTZSTAGE','Warehouse.CONTACTS_HIST','WAREHOUSE.CONTACTS_NTZSTAGE','Staging.COUNTRY','Warehouse.COUNTRY','WAREHOUSE.CUSTOMER_ROLE_HIST_NTZSTAGE','WAREHOUSE.CUSTOMER_ROLE_NTZSTAGE','Staging.DEPARTMENT','Warehouse.DEPARTMENT','Warehouse.DEPARTMENT_HIST','Staging.DF_LAB_COST_TK_TOTAL_1','WAREHOUSE.DISCOUNT_SKU_HIST_NTZSTAGE','WAREHOUSE.DISCOUNT_SKU_NTZSTAGE','Staging.DISCOUNT_TYPE','WAREHOUSE.DISCOUNT_TYPE_NTZSTAGE','Staging.DISTRIBUTION_METHOD','Warehouse.DISTRIBUTION_METHOD','Warehouse.DISTRIBUTION_METHOD_HIST','Staging.DISTRICT','Warehouse.DI

In [0]:
if Level == 'Table':
    tables_list = "("+",".join(["'"+str(x)+"'" for x in Tables.split(",")])+")"
    print(tables_list)

In [0]:
configquery = """(SELECT * FROM DBO.FWK_GOLD_TABLES 
WHERE [DATABASE]+'.'+TABLE_NAME IN {} )Config""".format(tables_list)
gold_df1 = readfromdbSQL(Env,configquery)
display(gold_df1)

FWK_TABLE_ID,DATABASE,SCHEMA,TABLE_NAME,KEY_COLUMN,ACTIVE_FLAG,PARTITION_COLUMN,KEY_TYPE,KEY_TYPE_UNIQUE,PARTITION_ON,PARTITION_QUERY_NETEZZA,PARTITION_QUERY_SYNAPSE,FIELD_VALIDATION_QUERY_NETEZZA,FIELD_VALIDATION_QUERY_SYNAPSE,MEASURE_QUERY_NETEZZA,MEASURE_QUERY_SYNAPSE,PARTITON_QUERY_NETEZZA_SYNAPSE_SEVEN_YEARS,MEASURE_COLUMN,MEASURE_QUERY_NETEZZA_SEVEN_YEARS,MEASURE_QUERY_SYNAPSE_SEVEN_YEARS,ADLS_QUERY_FOR_FIELD,ADLS_QUERY_FOR_MEASURE
19945,STAGING,ADMIN,ADDRESS_LINK,ADDR_ID,Y,SOURCE_CREATE_DATE,PK,N,DAY,"SELECT to_char(SOURCE_CREATE_DATE, 'YYYY Mon DD') AS SOURCE_CREATE_DATE, COUNT(*) as CNT FROM db_tmp.ADMIN.ADDRESS_LINK GROUP BY to_char(SOURCE_CREATE_DATE, 'YYYY Mon DD')",null,"SELECT * FROM db_tmp.ADMIN.ADDRESS_LINK WHERE to_char(SOURCE_CREATE_DATE, 'YYYY Mon DD')='2023 Mar 15'",null,null,null,null,null,null,null,"SELECT * from Data_Lake_df WHERE DATE_FORMAT(SOURCE_CREATE_DATE, 'y MMM dd') ='2023 Mar 15'",null
19946,STAGING,ADMIN,ADDRESS_LINK,ADDR_TYPE,N,SOURCE_CREATE_DATE,PK,Y,DAY,"SELECT to_char(SOURCE_CREATE_DATE, 'YYYY Mon DD') AS SOURCE_CREATE_DATE, COUNT(*) as CNT FROM db_tmp.ADMIN.ADDRESS_LINK GROUP BY to_char(SOURCE_CREATE_DATE, 'YYYY Mon DD')",null,"SELECT * FROM db_tmp.ADMIN.ADDRESS_LINK WHERE to_char(SOURCE_CREATE_DATE, 'YYYY Mon DD')='2023 Mar 15'",null,null,null,null,null,null,null,"SELECT * from Data_Lake_df WHERE DATE_FORMAT(SOURCE_CREATE_DATE, 'y MMM dd') ='2023 Mar 15'",null
20025,STAGING,ADMIN,BUDGET_DEPARTMENT_XREF,PBU_BUDGET_GROUP,N,null,DK,N,null,null,null,null,null,null,null,null,null,null,null,null,null
20033,STAGING,ADMIN,BUYER,BUYER_CODE,N,null,PK,Y,null,null,null,null,null,null,null,null,null,null,null,null,null
20034,STAGING,ADMIN,CBSA,CBSA_ID,N,null,PK,Y,null,null,null,null,null,null,null,null,null,null,null,null,null
20048,STAGING,ADMIN,CLUB_CALL_LIST_CUSTOMERS_CURRENT,CLUB_ID,N,null,DK,Y,null,null,null,null,null,null,null,null,null,null,null,null,null
20049,STAGING,ADMIN,CLUB_CONTACT,CLUB_CONTACT_ID,N,null,PK,Y,null,null,null,null,null,null,null,null,null,null,null,null,null
20078,STAGING,ADMIN,COMPANY,CMPNY_ID,N,null,PK,Y,null,null,null,null,null,null,null,null,null,null,null,null,null
20083,STAGING,ADMIN,COUNTRY,DW_PRCSS_ID,N,null,DK,N,null,null,null,null,null,null,null,null,null,null,null,null,null
20150,STAGING,ADMIN,DEPARTMENT,DEPT_ID,N,null,PK,Y,null,null,null,null,null,null,null,null,null,null,null,null,null


In [0]:
silver_list=['STAGING','WAREHOUSE','HRSTAGING','SDS_STAGING','SDS_CU']
gold_list = ['ACCOUNTING','FINANCE','HH_ALLINK','HRMART','HYBRIS','MARKETING','MART','OPERATIONS','REALESTATE']

In [0]:
## The table details required for filtering the transactional table for the 7 years filter
trQuery = "(select distinct SUBSTRING(TableName, PATINDEX('%[.]%', TableName)+1,LEN(TableName)) TableName ,nullif(RetentionColumn,'UNKNOWN') Filter7 from  [dbo].[DtIngestionMetadata] where RetentionColumn<>'UNKNOWN')trQuery"
TR_df = readfromdbSQL(Env,trQuery)
display(TR_df)

TableName,Filter7
CT_DRILLTHRU_STAGE,ACCOUNTING_DATE
CT_PROJTABLE,CREATED
DIM_APPOINTMENT,DATE
DIM_INVOICE,INVC_TS
DIM_TICKET,TICKET_PRCHS_DATE
DIM_TICKET_ITEM,TICKET_FISCAL_YEAR
FACT_APPOINTMENT,APPNTMNT_LAST_MDFD
FACT_CAPACITY_HIST,LOAD_DTM
FACT_CLINICAL_ATTRITION,LOAD_DATE
FACT_COMMISSIONS,UPDATE_DTM


In [0]:
## Join tables based on the comparison option choosen
## This dataframe determines which tables will be included in the validation
if (PL_TL == '7GS'):
    ## Narrows the table select to only transactional tables involved in the job
    gold_df = gold_df1.join(TR_df,[TR_df.TableName == gold_df1.TABLE_NAME]
    ,'inner'
    ).select(gold_df1['*'],TR_df.Filter7).distinct() 
else:
    ## Includes all tables involved in the job
    gold_df = gold_df1.join(TR_df,[TR_df.TableName == gold_df1.TABLE_NAME]
    ,'left'
    ).select(gold_df1['*'],TR_df.Filter7)#.where(TR_df.TableName.isNull())
#display(gold_df.select(gold_df.TABLE_NAME,TR_df.Filter7))
gold_df.cache().count()
gold_df.createOrReplaceTempView("gold_df")

In [0]:
display(gold_df)

FWK_TABLE_ID,DATABASE,SCHEMA,TABLE_NAME,KEY_COLUMN,ACTIVE_FLAG,PARTITION_COLUMN,KEY_TYPE,KEY_TYPE_UNIQUE,PARTITION_ON,PARTITION_QUERY_NETEZZA,PARTITION_QUERY_SYNAPSE,FIELD_VALIDATION_QUERY_NETEZZA,FIELD_VALIDATION_QUERY_SYNAPSE,MEASURE_QUERY_NETEZZA,MEASURE_QUERY_SYNAPSE,PARTITON_QUERY_NETEZZA_SYNAPSE_SEVEN_YEARS,MEASURE_COLUMN,MEASURE_QUERY_NETEZZA_SEVEN_YEARS,MEASURE_QUERY_SYNAPSE_SEVEN_YEARS,ADLS_QUERY_FOR_FIELD,ADLS_QUERY_FOR_MEASURE,Filter7
21441,WAREHOUSE,ADMIN,INVENTORY_DAILY,INVTRY_ID,N,DATE,PK,N,DAY,"SELECT to_char(DATE,'YYYY Mon dd') AS DATE, COUNT(*) AS CNT FROM db_tmp.ADMIN.INVENTORY_DAILY GROUP BY to_char(DATE,'YYYY Mon dd')",null,"SELECT * FROM db_tmp.ADMIN.INVENTORY_DAILY where to_char(DATE, 'YYYY Mon dd')='2023 May 25'",null,null,null,null,null,null,null,"SELECT * from Data_Lake_df WHERE DATE_FORMAT(DATE, 'y MMM d') ='2023 May 25'",null,null
20609,STAGING,ADMIN,SALES_OUTLET_BUDGET,FCLTY_ID,Y,null,PK,Y,null,null,null,null,null,null,null,null,null,null,null,null,null,null
20610,STAGING,ADMIN,SALES_OUTLET_BUDGET,PBU_BUDGET_GROUP,Y,SALES_OUTLET_BUDGET_DATE,PK,Y,MONTH,"SELECT to_char(SALES_OUTLET_BUDGET_DATE, 'YYYY Mon') AS SALES_OUTLET_BUDGET_DATE,COUNT(*) as CNT FROM db_tmp.ADMIN.SALES_OUTLET_BUDGET GROUP BY to_char(SALES_OUTLET_BUDGET_DATE, 'YYYY Mon')","SELECT SUBSTRING(SALES_OUTLET_BUDGET_DATE,1,7) AS SALES_OUTLET_BUDGET_DATE, COUNT(*) FROM delta.`/mnt/silver/STAGING/ADMIN/SALES_OUTLET_BUDGET/`GROUP BY SUBSTRING(SALES_OUTLET_BUDGET_DATE,1,7)","SELECT * FROM db_tmp.ADMIN.SALES_OUTLET_BUDGET where to_char(SALES_OUTLET_BUDGET_DATE, 'YYYY Mon')='2023 May'","SELECT * FROM delta.`/mnt/silver/STAGING/ADMIN/SALES_OUTLET_BUDGET/` WHERE SUBSTRING(SALES_OUTLET_BUDGET_DATE,1,7)='2021-07'","SELECT to_char(SALES_OUTLET_BUDGET_DATE,'YYYY Mon') AS SALES_OUTLET_BUDGET_DATE, SUM(FNNCL_BUDGET_AMT) AS FNNCL_BUDGET_AMT FROM STAGING.ADMIN.SALES_OUTLET_BUDGET GROUP BY to_char(SALES_OUTLET_BUDGET_DATE,'YYYY Mon')","SELECT SUM(FNNCL_BUDGET_AMT) AS FNNCL_BUDGET_AMT, SUM(OPRTNL_BUDGET_AMT) AS OPRTNL_BUDGET_AMT, SUBSTRING(SALES_OUTLET_BUDGET_DATE,1,7) AS SALES_OUTLET_BUDGET_DATE FROM delta.`/mnt/silver/STAGING/ADMIN/SALES_OUTLET_BUDGET/`GROUP BY SUBSTRING(SALES_OUTLET_BUDGET_DATE,1,7)",null,FNNCL_BUDGET_AMT,null,null,"SELECT * from Data_Lake_df WHERE DATE_FORMAT(SALES_OUTLET_BUDGET_DATE, 'y MMM') ='2023 May'","SELECT DATE_FORMAT(SALES_OUTLET_BUDGET_DATE, 'y MMM') AS SALES_OUTLET_BUDGET_DATE, SUM(FNNCL_BUDGET_AMT) AS FNNCL_BUDGET_AMT FROM Data_Lake_df GROUP BY DATE_FORMAT(SALES_OUTLET_BUDGET_DATE, 'y MMM')",null
20611,STAGING,ADMIN,SALES_OUTLET_BUDGET,SALES_OUTLET_BUDGET_DATE,Y,null,PK,Y,null,null,null,null,null,null,null,null,null,null,null,null,null,null
21563,WAREHOUSE,ADMIN,SALES_OUTLET_BUDGET,FCLTY_ID,N,null,PK,Y,null,null,null,null,null,null,null,null,null,null,null,null,null,null
21564,WAREHOUSE,ADMIN,SALES_OUTLET_BUDGET,PBU_BUDGET_GROUP,N,null,PK,Y,null,null,null,null,null,null,null,null,null,null,null,null,null,null
21565,WAREHOUSE,ADMIN,SALES_OUTLET_BUDGET,SALES_OUTLET_BUDGET_DATE,N,SALES_OUTLET_BUDGET_DATE,PK,Y,MONTH,"SELECT to_char(SALES_OUTLET_BUDGET_DATE, 'YYYY Mon') AS SALES_OUTLET_BUDGET_DATE,COUNT(*) as CNT FROM db_tmp.ADMIN.SALES_OUTLET_BUDGET GROUP BY to_char(SALES_OUTLET_BUDGET_DATE, 'YYYY Mon')",null,"SELECT * FROM db_tmp.ADMIN.SALES_OUTLET_BUDGET where to_char(SALES_OUTLET_BUDGET_DATE, 'YYYY Mon')='2023 May'",null,"SELECT to_char(SALES_OUTLET_BUDGET_DATE,'YYYY Mon') AS SALES_OUTLET_BUDGET_DATE,SUM(OPRTNL_BUDGET_AMT) AS OPRTNL_BUDGET_AMT FROM db_tmp.ADMIN.SALES_OUTLET_BUDGET GROUP BY to_char(SALES_OUTLET_BUDGET_DATE,'YYYY Mon')",null,null,OPRTNL_BUDGET_AMT,null,null,"SELECT * from Data_Lake_df WHERE DATE_FORMAT(SALES_OUTLET_BUDGET_DATE, 'y MMM') ='2023 May'","SELECT DATE_FORMAT(SALES_OUTLET_BUDGET_DATE, 'y MMM') AS SALES_OUTLET_BUDGET_DATE, SUM(OPRTNL_BUDGET_AMT) AS OPRTNL_BUDGET_AMT FROM Data_Lake_df GROUP BY DATE_FORMAT(SALES_OUTLET_BUDGET_DATE, 'y MMM')",null
20389,STAGING,ADMIN,HUMAN_RESOURCE_FACILITY_ASSIGNMENT,EF

In [0]:
cnttable=gold_df.count()
if cnttable==0:
    dbutils.notebook.exit(cnttable) 

In [0]:
## List of unique databases with tables to be validated
validation_databases = [row.DATABASE for row in gold_df.select('DATABASE').distinct().collect()]

## Instantiate list of dictionaries
validation_tables_info = []

## Append dictionary of database with list of distinct tables
for db in validation_databases:
    #print(db)
    dict_schema={}
    list_tables = [row.TABLE_NAME for row in gold_df.select(['TABLE_NAME']).where(gold_df.DATABASE == db).distinct().collect()]
    dict_schema[db]=[*set(list_tables)]
    cp_dict_schema=dict_schema.copy()
    #print(dict_schema)
    validation_tables_info.append(cp_dict_schema) 
print(validation_tables_info)

[{'WAREHOUSE': ['DOCTOR', 'HUMAN_RESOURCE_HIST', 'STORE_HIST', 'CLASS_HIST', 'FACILITY_HIERARCHY', 'JOB_TYPE', 'GIFT_CERTIFICATE_SKU_HIST', 'COUNTRY', 'LAB', 'PHONE_NUMBER_HIST', 'DISTRIBUTION_METHOD_HIST', 'DISTRICT_REGION_ALIGNMENT', 'MSA_HIST', 'REGION_SUBDIVISION_ALIGNMENT', 'SKU_ENTITY_INLINE', 'ENTITY_PACKAGE_HIST', 'FACILITY_HIERARCHY_HISTORY', 'EMPLOYEE_COMPANY_HIST', 'DOCTOR_SUBTYPE_HIST', 'STORE_REMODEL_TYPE', 'STORE_USAGE_TYPE', 'MATERIAL', 'SUBCLASS_HIST', 'FACILITY', 'DISTRICT', 'STORE_CLOSED_TYPE', 'STORE_CLOSE_REASON_TYPE', 'DEPARTMENT', 'DOCTOR_TYPE_HIST', 'SKU_PACKAGE_COMPONENT', 'TICKET_ITEM_STATUS', 'JOB_CLASSIFICATION', 'EMPLOYEE_COMPANY', 'DISTRICT_REGION_ALIGNMENT_HISTORY', 'SUBDIVISION_DIVISION_ALIGNMENT', 'PLANOGRAM', 'LAB_COST', 'DISTRIBUTION_METHOD', 'DEPARTMENT_HIST', 'DISTRICT_SUBREGION_ALIGNMENT', 'REGION', 'STATE', 'SKU_HIST', 'SUBDEPARTMENT_HIST', 'SALES_OUTLET_BUDGET_HIST', 'POSTAL_ADDRESS_HIST', 'STORE_STATUS_TYPE', 'FACILITY_ADDRESS', 'DISTRICT_SUBREGI

In [0]:
from pyspark.sql.types import *
import pyspark.sql.functions as sf
from pyspark.sql.functions import col,when
from pyspark.sql import SQLContext
from datetime import datetime
from datetime import date
import datetime
import pandas as pd
import shutil

if PL_TL == '7GS':
    netezza_sub = '7 yrs'
else:
    netezza_sub = 'Full'

## Header for log document
doc.add_heading('Log From Metadata Info Generation and Summary Report', 2)

try:
    tab = 0

    ## Define the schema for the netezza metadata report
    netezza_metadata_schema = StructType([StructField('NETEZZA_DATABASE', StringType(), True),
                                StructField('NETEZZA_TABLE_NAME', StringType(), True),
                                StructField('NETEZZA_COLUMN_NAME', StringType(), True),
                                StructField('NETEZZA_DATATYPE', StringType(), True),
                                StructField('NETEZZA_ORDINAL_POSITION', StringType(), True)])

    ## Define the schema for the synapse metadata report
    synapse_metadata_schema = StructType([StructField('SYNAPSE_DATABASE', StringType(), True),
                                StructField('SYNAPSE_TABLE_NAME', StringType(), True),
                                StructField('SYNAPSE_COLUMN_NAME', StringType(), True),
                                StructField('SYNAPSE_DATATYPE', StringType(), True),
                                StructField('SYNAPSE_ORDINAL_POSITION', StringType(), True)])

    ## Define the schema for the gold metadata report
    gold_metadata_schema = StructType([StructField('GOLD_DATABASE', StringType(), True),
                                StructField('GOLD_TABLE_NAME', StringType(), True),
                                StructField('GOLD_COLUMN_NAME', StringType(), True),
                                StructField('GOLD_DATATYPE', StringType(), True)
                                ])

    ## Define the schema for the silver metadata report
    silver_metadata_schema = StructType([StructField('SILVER_DATABASE', StringType(), True),
                                StructField('SILVER_TABLE_NAME', StringType(), True),
                                StructField('SILVER_COLUMN_NAME', StringType(), True),
                                StructField('SILVER_DATATYPE', StringType(), True)
                                ])

    ## Define the schema for the summary report
    

    if (PL_TL == '7GS') | (Load_Type != 'Historical'):
        report_dataframe_schema_1= StructType([StructField('NETEZZA_SCHEMA', StringType(), True), #SAP HANA SCHEMA NAME
                                    StructField('NETEZZA_TABLE', StringType(), True), #SAP HANA TABLE NAME
                                    StructField('SYNAPSE_SCHEMA', StringType(), True),
                                    StructField('SYNAPSE_TABLE', StringType(), True), #ADLS DELTA TABLE NAME
                                    StructField('MISMATCHED_COLUMNS (NETEZZA - SYNAPSE)', ArrayType(StringType()), True), #list of columns in Netezza which is not present in Synapse                                                                                               
                                    StructField('MISMATCHED_COLUMNS (SYNAPSE - NETEZZA)', ArrayType(StringType()), True),#list of columns in Synapse which is not present in Netezza 
                                    StructField(f'NETEZZA_TABLE_ROW_COUNT ({netezza_sub})', StringType(), True), # row count in Netezza TABLE
                                    StructField('SYNAPSE_TABLE_ROW_COUNT', StringType(), True), # row count in Netezza TABLE
                                    #StructField('GOLD_TABLE_ROW_COUNT', StringType(), True), # row count in Synapse TABLE
                                    StructField('ROW_COUNT_DIFFERENCE(NETEZZA - SYNAPSE)', StringType(), True)]) # difference in row counts
                                    #StructField('ROW_COUNT_DIFFERENCE(NETEZZA - GOLD)', StringType(), True)]) # difference in row counts

    else:
        report_dataframe_schema_1= StructType([StructField('NETEZZA_SCHEMA', StringType(), True), #SAP HANA SCHEMA NAME
                                    StructField('NETEZZA_TABLE', StringType(), True), #SAP HANA TABLE NAME
                                    StructField('SYNAPSE_SCHEMA', StringType(), True),
                                    StructField('SYNAPSE_TABLE', StringType(), True), #ADLS DELTA TABLE NAME
                                    StructField('MISMATCHED_COLUMNS (NETEZZA - SYNAPSE)', ArrayType(StringType()), True), #list of columns in Netezza which is not present in Synapse                                                                                               
                                    StructField('MISMATCHED_COLUMNS (SYNAPSE - NETEZZA)', ArrayType(StringType()), True),#list of columns in Synapse which is not present in Netezza 
                                    StructField(f'NETEZZA_TABLE_ROW_COUNT ({netezza_sub})', StringType(), True), # row count in Netezza TABLE
                                    StructField('SYNAPSE_TABLE_ROW_COUNT', StringType(), True), # row count in Netezza TABLE
                                    StructField('GOLD_TABLE_ROW_COUNT', StringType(), True), # row count in Synapse TABLE
                                    StructField('ROW_COUNT_DIFFERENCE(NETEZZA - SYNAPSE)', StringType(), True), # difference in row counts
                                    StructField('ROW_COUNT_DIFFERENCE(NETEZZA - GOLD)', StringType(), True)]) # difference in row counts


                                    
    if Load_Type == 'Historical':
        report_dataframe_schema_2= StructType([StructField('NETEZZA_SCHEMA', StringType(), True), #SAP HANA SCHEMA NAME
                                    StructField('NETEZZA_TABLE', StringType(), True), #SAP HANA TABLE NAME
                                    StructField('SILVER_SCHEMA', StringType(), True),
                                    StructField('SILVER_TABLE', StringType(), True),
                                    StructField('MISMATCHED_COLUMNS (NETEZZA - SILVER)', ArrayType(StringType()), True),
                                    StructField('MISMATCHED_COLUMNS (SILVER - NETEZZA)', ArrayType(StringType()), True),
                                    StructField(f'NETEZZA_TABLE_ROW_COUNT ({netezza_sub})', StringType(), True), # row count in Netezza TABLE
                                    StructField('SILVER_TABLE_ROW_COUNT', StringType(), True),
                                    StructField('ROW_COUNT_DIFFERENCE(NETEZZA - SILVER)', StringType(), True)])
            

    report_dataframe_info=[]

    ## Create list of gold database with tables to be validated
    relevant_gold_db = set(validation_databases).intersection(set(gold_list))
    ## Create list of silver database with tables to be validated
    relevant_silver_db = set(validation_databases).intersection(set(silver_list))




    ## Summary report for netezza v. synapse v. gold tables
    if len(relevant_gold_db) > 0:

        netezza_metadata_1 = []
        synapse_metadata = []
        gold_metadata = []
        report_dataframe_info_1=[]


        for database in relevant_gold_db:

            if Env == 'QA':
                db_tmp = f"GDC_{database}"
            elif Env == 'PROD_DR':
                db_tmp = f"{database}_DR"
            else:
                db_tmp = database

            #print(database)
            tables=[v for d in validation_tables_info for k,v in d.items() if k == database][0]
            for table in tables:
                print(f'{database}.{table}')

                ## Header for log document
                doc.add_heading(f'{database}.{table}', 3)

                try : 
                    ## NETEZZA METADATA
                    query_for_columns_info = f"""select attname,format_type,attnum from  {db_tmp}._V_RELATION_COLUMN WHERE NAME = '{table}'""" 
                    netezza_df=readfromNetizza(Env,query_for_columns_info,database)
                    ## List of columns in the table
                    netezza_side_columns = [row[0] for row in netezza_df.select('attname').collect()]  #netezza_df.rdd.map(lambda x: x[0]).collect()
                    ## List of datatype for each column in table
                    netezza_side_datatype= [row[0] for row in netezza_df.select('format_type').collect()]
                    ## List of ordinal position
                    netezza_ordinal_position = [row[0] for row in netezza_df.select('attnum').collect()]

                    if (PL_TL == '7GS'):
                        ## Retrieve the filter column for the table. This is a date column used to identify the minimum date in which data from in table was loaded from Netizza
                        filterColumn=gold_df.select('Filter7').where((gold_df.TABLE_NAME==table)&(gold_df.DATABASE==database)).distinct()  #.rdd.map(lambda x: x.Filter7).collect()
                        filterColumn = [row[0] for row in filterColumn.collect()]
                        #print(filterColumn)
                        filterColumn=str(filterColumn)
                        filterColumn=filterColumn.replace("[","").replace("]","").replace("'","")
                        #print('filterCol', filterColumn)
                        if filterColumn!='None':
                            #print('filterCol2:', filterColumn) 
                            ## Fetch the minimum date from synapse to avoid mismatch on the day where data is not loaded in netizza
                            query_forValuefilter= f"""select CAST(min({filterColumn}) as varchar(10)) {filterColumn} from {database}.{table}""" 
                            #print(query_forValuefilter)
                            filterValue1=readfromSynapse(Env,query_forValuefilter)
                            filterValue=[row[0] for row in filterValue1.collect()]  #filterValue1.rdd.map(lambda x: x[0]).collect()
                            filterValue=str(filterValue)
                            filterValue=filterValue.replace("[","").replace("]","")
                            if filterValue!='None':
                                netezza_count_info_query = f"""select count(*) from {db_tmp}.{table} where {filterColumn} >= {filterValue}""" 
                                print('filtering Netezza for 7 years only')
                                doc.add_paragraph('filtering Netezza for 7 years only')
                            else:
                                netezza_count_info_query = f"""select count(*) from {db_tmp}.{table}"""
                                print('No 7 year filter value configured. Using full Netezza table')
                                doc.add_paragraph('No 7 year filter value configured. Using full Netezza table', style='List Bullet')
                        else:     
                            netezza_count_info_query = f"""select count(*) from {db_tmp}.{table}"""
                            print('No 7 year filter column configured. Using full Netezza table')
                            doc.add_paragraph('No 7 year filter column configured. Using full Netezza table', style='List Bullet')  
                    else:
                        netezza_count_info_query = f"""select count(*) from {db_tmp}.{table}""" 
                        print('Using full Netezza table')
                        doc.add_paragraph('Using full Netezza table', style='List Bullet')

                    ## Get number of rows in the table   
                    netezza_df_count=readfromNetizza(Env,netezza_count_info_query,database)
                    netezza_side_no_of_rows = netezza_df_count.collect()[0][0]

                    for i in range(0, len(netezza_side_columns)):
                        netezza_metadata_row = [database, table, netezza_side_columns[i], netezza_side_datatype[i], netezza_ordinal_position[i]]
                        netezza_metadata_1.append(netezza_metadata_row)

                    print(f"Successfully generate Netezza metadata info for {db_tmp}.{table}")
                    doc.add_paragraph(f"Successfully generate Netezza metadata info for {db_tmp}.{table}", style='List Bullet')

                except Exception as e:
                    print(f"Failed to generate Netezza metadata info for {db_tmp}.{table}")
                    doc.add_paragraph(f"Failed to generate Netezza metadata info for {db_tmp}.{table}", style='List Bullet')
                    print(e)
                    doc.add_paragraph(f'{e}', style='Intense Quote')

                try:
                    ## SYNAPSE METADATA 
                    synapse_query_for_columns_info = f"""select column_name, data_type, ordinal_position from INFORMATION_SCHEMA.COLUMNS where table_schema= '{database}' and table_name='{table}'""" 
                                                    # query can be customized
                    synapse_columns_info=readfromSynapse(Env,synapse_query_for_columns_info)
                    
                    ## List of columns in the table
                    synapse_side_columns = [result['column_name'] for result in synapse_columns_info.select('*').collect()]
                    ## List of datatype for each column in table
                    synapse_side_datatype = [result['data_type'] for result in synapse_columns_info.select('*').collect()]
                    ## Create list of ordinal position
                    synapse_ordinal_position = [result['ordinal_position'] for result in synapse_columns_info.select('*').collect()]

                    synapse_count_info_query = f"""select count(*) as record_count from {database}.{table}"""
                    #print(synapse_count_info_query)
                    synapse_df_count =readfromSynapse(Env,synapse_count_info_query)
                    synapse_side_no_of_rows = synapse_df_count.select(['record_count']).collect()[0][0]

                    for i in range(0, len(synapse_side_columns)):
                        synapse_metadata_row = [database, table, synapse_side_columns[i], synapse_side_datatype[i], synapse_ordinal_position[i]]
                        synapse_metadata.append(synapse_metadata_row)

                    print(f"Successfully generate Synapse metadata info for {database}.{table}")
                    doc.add_paragraph(f"Successfully generate Synapse metadata info for {database}.{table}", style='List Bullet')

                except Exception as e:
                    print(f"Failed to generate Synapse metadata info for {database}.{table}")
                    doc.add_paragraph(f"Failed to generate Synapse metadata info for {database}.{table}", style='List Bullet')
                    print(e)
                    doc.add_paragraph(f'{e}', style='Intense Quote')

                try:
                    ## GOLD METADATA
                    gold_columns_info=spark.read.format("delta").option("header","true").load("/mnt/gold/"+database+"/ADMIN/"+table) 
                    gold_columns_info.createOrReplaceTempView("gold_columns_info")
                    ## List of columns in the table
                    gold_side_columns = [result[0] for result in gold_columns_info.dtypes]
                    #print(col)
                    ## List of datatype for each column in table
                    gold_side_datatype = [result[1] for result in gold_columns_info.dtypes]
                    #print(datatype)

                    gold_df_count =spark.sql(f"select count(*) as record_count from gold_columns_info")
                    gold_side_no_of_rows = gold_df_count.select(['record_count']).collect()[0][0]

                    for i in range(0, len(gold_side_columns)):
                        gold_metadata_row = [database, table, gold_side_columns[i], gold_side_datatype[i]]
                        gold_metadata.append(gold_metadata_row)

                    print(f"Successfully generate Gold metadata info for {database}.{table}")
                    doc.add_paragraph(f"Successfully generate Gold metadata info for {database}.{table}", style='List Bullet')

                except Exception as e:
                    print(f"Failed to generate Gold metadata info for {database}.{table}")
                    doc.add_paragraph(f"Failed to generate Gold metadata info for {database}.{table}", style='List Bullet')
                    print(e)
                    doc.add_paragraph(f'{e}', style='Intense Quote')


                ## SUMMARY REPORT
                try:
                    table_report=[]                
                    table_report.append(database); table_report.append(table); 
                    table_report.append(database);table_report.append(table);
                    table_report.append(list(set(netezza_side_columns)-set(synapse_side_columns))); # comparing columns column in netezza not in synapse
                    table_report.append(list(set(synapse_side_columns)-set(netezza_side_columns))); # column in synapse but not in netezza
                    table_report.append(str(netezza_side_no_of_rows)); 
                    table_report.append(str(synapse_side_no_of_rows));
                    
                    if (PL_TL == '7GS') | (Load_Type != 'Historical'):
                        table_report.append(str(netezza_side_no_of_rows - synapse_side_no_of_rows));
                    else:
                        table_report.append(str(gold_side_no_of_rows));
                        table_report.append(str(netezza_side_no_of_rows - synapse_side_no_of_rows))
                        table_report.append(str(netezza_side_no_of_rows - gold_side_no_of_rows));
                    

                    report_dataframe_info_1.append(table_report)
                    print(f'Successfully generated summary report for {database}.{table}')
                    doc.add_paragraph(f'Successfully generated summary report for {database}.{table}', style='List Bullet')        

                except KeyError:
                    table_report=[]
                    table_report.append(database); table_report.append(table);
                    table_report.append('Missing');table_report.append('Missing in SYNAPSE');
                    table_report.append(None); #difference in row count of SAP and ADLS-DELTA Table
                    table_report.append(None);
                    table_report.append(str(netezza_side_dictionary[database][table]['no_of_rows'])); 
                    table_report.append(''); 
                    
                    if (PL_TL == '7GS') | (Load_Type != 'Historical'):
                        table_report.append('FAILED');
                    else:
                        table_report.append(''); 
                        table_report.append('FAILED');
                        table_report.append('FAILED');
                    

                    report_dataframe_info_1.append(table_report)
                    print(f'Failed to generate summary report for {database}.{table}')
                    doc.add_paragraph(f'Failed to generate summary report for {database}.{table}', style='List Bullet')
                    print(KeyError)
                    doc.add_paragraph(f'{KeyError}')

                tab += 1
                print(tab)

        


        netezza_metadata_dataframe_1 = spark.createDataFrame(netezza_metadata_1,netezza_metadata_schema)
        netezza_metadata_dataframe_pd_1 = netezza_metadata_dataframe_1.toPandas()

        synapse_metadata_dataframe = spark.createDataFrame(synapse_metadata,synapse_metadata_schema)
        synapse_metadata_dataframe_pd = synapse_metadata_dataframe.toPandas()
    
        gold_metadata_dataframe = spark.createDataFrame(gold_metadata,gold_metadata_schema)
        gold_metadata_dataframe_pd = gold_metadata_dataframe.toPandas()

        report_dataframe_1=spark.createDataFrame(report_dataframe_info_1,report_dataframe_schema_1).withColumn("Rpt_Exec_Ts",sf.current_timestamp())
        report_dataframe_pd_1 = report_dataframe_1.toPandas()


    ## Summary report for netezza v. silver tables
    if len(relevant_silver_db) > 0:

        netezza_metadata_2 = []
        silver_metadata = []
        report_dataframe_info_2=[]


        for database in list(relevant_silver_db):

            if Env == 'QA':
                db_tmp = f"GDC_{database}"
            elif Env == 'PROD_DR':
                db_tmp = f"{database}_DR"
            else:
                db_tmp = database

            #print(database)
            tables=[v for d in validation_tables_info for k,v in d.items() if k == database][0]
            for table in tables:
                print(f'{database}.{table}')

                ## Header for log document
                doc.add_heading(f'{database}.{table}', 3)

                try : 
                    ## NETEZZA METADATA
                    query_for_columns_info = f"""select attname,format_type,attnum from  {db_tmp}._V_RELATION_COLUMN WHERE NAME = '{table}'""" 
                    netezza_df=readfromNetizza(Env,query_for_columns_info,database)
                    ## List of columns in the table
                    netezza_side_columns = [row[0] for row in netezza_df.select('attname').collect()]  #netezza_df.rdd.map(lambda x: x[0]).collect()
                    ## List of datatype for each column in table
                    netezza_side_datatype= [row[0] for row in netezza_df.select('format_type').collect()]
                    ## List of ordinal position
                    netezza_ordinal_position = [row[0] for row in netezza_df.select('attnum').collect()]

                    if (PL_TL == '7GS'):
                        ## Retrieve the filter column for the table. This is a date column used to identify the minimum date in which data from in table was loaded from Netizza
                        filterColumn=gold_df.select('Filter7').where((gold_df.TABLE_NAME==table)&(gold_df.DATABASE==database)).distinct()  #.rdd.map(lambda x: x.Filter7).collect()
                        filterColumn = [row[0] for row in filterColumn.collect()]
                        #print(filterColumn)
                        filterColumn=str(filterColumn)
                        filterColumn=filterColumn.replace("[","").replace("]","").replace("'","")
                        #print('filterCol', filterColumn)
                        if filterColumn!='None':
                            #print('filterCol2:', filterColumn) 
                            ## Fetch the minimum date from synapse to avoid mismatch on the day where data is not loaded in netizza
                            query_forValuefilter= f"""select CAST(min({filterColumn}) as varchar(10)) {filterColumn} from {database}.{table}""" 
                            #print(query_forValuefilter)
                            filterValue1=readfromSynapse(Env,query_forValuefilter)
                            filterValue=[row[0] for row in filterValue1.collect()]  #filterValue1.rdd.map(lambda x: x[0]).collect()
                            filterValue=str(filterValue)
                            filterValue=filterValue.replace("[","").replace("]","")
                            if filterValue!='None':
                                netezza_count_info_query = f"""select count(*) from {db_tmp}.{table} where {filterColumn} >= {filterValue}"""
                                print('filtering Netezza for 7 years only')
                                doc.add_paragraph('filtering Netezza for 7 years only')
                            else:
                                netezza_count_info_query = f"""select count(*) from {db_tmp}.{table}"""
                                print('No 7 year filter value configured. Using full Netezza table')
                                doc.add_paragraph('No 7 year filter value configured. Using full Netezza table', style='List Bullet')
                        else:     
                            netezza_count_info_query = f"""select count(*) from {db_tmp}.{table}"""  
                            print('No 7 year filter column configured. Using full Netezza table')
                            doc.add_paragraph('No 7 year filter column configured. Using full Netezza table', style='List Bullet')  
                    else:
                        netezza_count_info_query = f"""select count(*) from {db_tmp}.{table}""" 
                        print('Using full Netezza table')
                        doc.add_paragraph('Using full Netezza table', style='List Bullet')

                    ## Get number of rows in the table   
                    netezza_df_count=readfromNetizza(Env,netezza_count_info_query,database)
                    netezza_side_no_of_rows = netezza_df_count.collect()[0][0]

                    for i in range(0, len(netezza_side_columns)):
                        netezza_metadata_row = [database, table, netezza_side_columns[i], netezza_side_datatype[i], netezza_ordinal_position[i]]
                        netezza_metadata_2.append(netezza_metadata_row)

                    print(f"Successfully generate Netezza metadata info for {db_tmp}.{table}")
                    doc.add_paragraph(f"Successfully generate Netezza metadata info for {db_tmp}.{table}", style='List Bullet')
                except Exception as e:
                    print(f"Failed to generate Netezza metadata info for {db_tmp}.{table}")
                    doc.add_paragraph(f"Failed to generate Netezza metadata info for {db_tmp}.{table}", style='List Bullet')
                    print(e)
                    doc.add_paragraph(f'{e}', style='Intense Quote')

                try:
                    ## SILVER METADATA
                    silver_columns_info=spark.read.format("delta").option("header","true").load("/mnt/silver/"+database+"/ADMIN/"+table) 
                    silver_columns_info.createOrReplaceTempView("silver_columns_info")
                    ## List of columns in the table
                    silver_side_columns = [result[0] for result in silver_columns_info.dtypes]
                    #print(col)
                    ## List of datatype for each column in table
                    silver_side_datatype = [result[1] for result in silver_columns_info.dtypes]
                    #print(datatype)

                    silver_df_count =spark.sql(f"select count(*) as record_count from silver_columns_info")
                    silver_side_no_of_rows = silver_df_count.select(['record_count']).collect()[0][0]

                    for i in range(0, len(silver_side_columns)):
                        silver_metadata_row = [database, table, silver_side_columns[i], silver_side_datatype[i]]
                        silver_metadata.append(silver_metadata_row)

                    print(f"Successfully generate Silver metadata info for {database}.{table}")
                    doc.add_paragraph(f"Successfully generate Silver metadata info for {database}.{table}", style='List Bullet')
                except Exception as e:
                    print(f"Failed to generate Silver metadata info for {database}.{table}")
                    doc.add_paragraph(f"Failed to generate Silver metadata info for {database}.{table}", style='List Bullet')
                    print(e)
                    doc.add_paragraph(f'{e}', style='Intense Quote')


                try:
                    table_report=[]
                    table_report.append(database); table_report.append(table); 
                    table_report.append(database);table_report.append(table);
                    table_report.append(list(set(netezza_side_columns)-set(silver_side_columns)));
                    table_report.append(list(set(silver_side_columns)-set(netezza_side_columns)));
                    table_report.append(str(netezza_side_no_of_rows)); 
                    table_report.append(str(silver_side_no_of_rows));
                    table_report.append(str(netezza_side_no_of_rows - silver_side_no_of_rows))
                    
                    report_dataframe_info_2.append(table_report)
                    print(f'Successfully generated summary report for {database}.{table}')
                    doc.add_paragraph(f'Successfully generated summary report for {database}.{table}', style='List Bullet') 

                except KeyError:
                    table_report=[]
                    table_report.append(database); table_report.append(table);
                    table_report.append('Missing');table_report.append('Missing in SILVER');
                    table_report.append(None); #difference in row count of SAP and ADLS-DELTA Table
                    table_report.append(None);
                    table_report.append(str(netezza_side_dictionary[database][table]['no_of_rows'])); 
                    table_report.append(''); 
                    table_report.append('FAILED')
                    
                    report_dataframe_info_2.append(table_report)
                    print(f'Failed to generate summary report for {database}.{table}')
                    doc.add_paragraph(f'Failed to generate summary report for {database}.{table}', style='List Bullet')
                    print(KeyError)
                    doc.add_paragraph(f'{KeyError}') 

                tab += 1
                print(tab)

        netezza_metadata_dataframe_2 = spark.createDataFrame(netezza_metadata_2,netezza_metadata_schema)
        netezza_metadata_dataframe_pd_2 = netezza_metadata_dataframe_2.toPandas()

        silver_metadata_dataframe = spark.createDataFrame(silver_metadata,silver_metadata_schema)
        silver_metadata_dataframe_pd = silver_metadata_dataframe.toPandas()

        if Load_Type == 'Historical':
            report_dataframe_2=spark.createDataFrame(report_dataframe_info_2,report_dataframe_schema_2).withColumn("Rpt_Exec_Ts",sf.current_timestamp())
            report_dataframe_pd_2 = report_dataframe_2.toPandas()






        
    ## Define excel writer
    try:
        with pd.ExcelWriter("METADATA_INFO.xlsx", engine="openpyxl", mode = 'w') as writer: 
            if len(relevant_gold_db) > 0:
                ## Write metadata info for Netezza, Synapse and Gold to sheet in excel workbook
                sheet_name =  "NETEZZA v. SYNAPSE v. GOLD"
                netezza_metadata_dataframe_pd_1.to_excel(writer, sheet_name = sheet_name, startrow = 0, startcol=0, index = False)
                synapse_metadata_dataframe_pd.to_excel(writer, sheet_name = sheet_name, startrow = 0, startcol=6, index = False)
                gold_metadata_dataframe_pd.to_excel(writer, sheet_name = sheet_name, startrow = 0, startcol=12, index = False)
            if len(relevant_silver_db) > 0:
                ## Write metadata info for Netezza and Silver to sheet in excel workbook
                sheet_name =  "NETEZZA v. SILVER"
                netezza_metadata_dataframe_pd_2.to_excel(writer, sheet_name = sheet_name, startrow = 0, startcol=0, index = False)
                silver_metadata_dataframe_pd.to_excel(writer, sheet_name = sheet_name, startrow = 0, startcol=6, index = False)

        workbook_path = path + 'METADATA_INFO.xlsx'            
        shutil.copy2('METADATA_INFO.xlsx',workbook_path)
        print(f"Successfully uploaded metadata info to path {workbook_path.replace('/dbfs/mnt', '')}")
        doc.add_paragraph(f"Successfully uploaded metadata info to path {workbook_path.replace('/dbfs/mnt', '')}", style='List Bullet')
        os.remove('METADATA_INFO.xlsx')

    except Exception as e:
        print(f"Failed to upload metadata info")
        doc.add_paragraph(f"Failed to upload metadata info", style='List Bullet')
        print(e)
        doc.add_paragraph(f'{e}', style='Intense Quote')


    try:
        with pd.ExcelWriter("SUMMARY_REPORT.xlsx", engine="openpyxl", mode = 'w') as writer1:
            if len(relevant_gold_db) > 0:
                ## Write summary report for Netezza, Synapse and Gold to sheet in excel workbook
                if (PL_TL == '7GS') | (Load_Type != 'Historical'):
                    sheet_name =  "NETEZZA v. SYNAPSE"
                else:
                    sheet_name =  "NETEZZA v. SYNAPSE v. GOLD"
                report_dataframe_pd_1.to_excel(writer1, sheet_name = sheet_name, startrow = 0, startcol=0, index = False)
            if (Load_Type == 'Historical') & (len(relevant_silver_db) > 0):
                ## Write summary report for Netezza and Silver to sheet in excel workbook
                sheet_name =  "NETEZZA v. SILVER"
                report_dataframe_pd_2.to_excel(writer1, sheet_name = sheet_name, startrow = 0, startcol=0, index = False)
            
        workbook_path = path + 'SUMMARY_REPORT.xlsx'            
        shutil.copy2('SUMMARY_REPORT.xlsx',workbook_path)
        print(f"Successfully uploaded summary report to path {workbook_path.replace('/dbfs/mnt', '')}")
        doc.add_paragraph(f"Successfully uploaded summary report to path {workbook_path.replace('/dbfs/mnt', '')}", style='List Bullet')
        os.remove('SUMMARY_REPORT.xlsx')

    except Exception as e:
        print(f"Failed to upload summary report")
        doc.add_paragraph(f"Failed to upload summary report", style='List Bullet')
        print(e)
        doc.add_paragraph(f'{e}', style='Intense Quote')

                        


    doc.add_paragraph('')

    ## Upload log file
    save_log(doc)

except Exception as e:
    doc.add_paragraph("Data Validation Run Failed", style="List Bullet")
    print(e)
    doc.add_paragraph(f'{e}', style="Intense Quote")
    doc.add_paragraph('')

    ## Upload log file
    save_log(doc)

    ## Exit the notebook run
    dbutils.notebook.exit("Data Validation Run Failed") 





MART.FACT_TICKET_ITEM
Using full Netezza table
Successfully generate Netezza metadata info for GDC_MART.FACT_TICKET_ITEM
Successfully generate Synapse metadata info for MART.FACT_TICKET_ITEM
Successfully generate Gold metadata info for MART.FACT_TICKET_ITEM
Successfully generated summary report for MART.FACT_TICKET_ITEM
1
STAGING.DOCTOR
Using full Netezza table
Successfully generate Netezza metadata info for GDC_STAGING.DOCTOR
Successfully generate Silver metadata info for STAGING.DOCTOR
Successfully generated summary report for STAGING.DOCTOR
2
STAGING.ADDRESS_LINK
Using full Netezza table
Successfully generate Netezza metadata info for GDC_STAGING.ADDRESS_LINK
Successfully generate Silver metadata info for STAGING.ADDRESS_LINK
Successfully generated summary report for STAGING.ADDRESS_LINK
3
STAGING.HUMAN_RESOURCE_FACILITY_ASSIGNMENT
Using full Netezza table
Successfully generate Netezza metadata info for GDC_STAGING.HUMAN_RESOURCE_FACILITY_ASSIGNMENT
Successfully generate Silver meta

In [0]:
from datetime import datetime
now = datetime.now()
now.strftime("%H:%M:%S")

'09:08:15'

In [0]:
if RptType=='Summary Report':

    ## Upload the log file
    doc.save('log.docx')
    shutil.copy2('log.docx',path+'log.docx')
    os.remove('log.docx') 

    ## Exit the notebook run
    dbutils.notebook.exit(RptType) 

In [0]:
## Instantiate list of dictionaries
validation_tables_info_2 = []

for db in validation_databases:  
    #print(db)
    dict_schema={}
    list_tables = [row.TABLE_NAME for row in gold_df.select(['TABLE_NAME']).where(gold_df.DATABASE == db).distinct().collect()]
    indi_schema_list=[]
    for table in list_tables:
        dict_table={}
        list_primary_key=[row.KEY_COLUMN for row in gold_df.select(['KEY_COLUMN'])\
                                .where((gold_df.TABLE_NAME==table) & (gold_df.DATABASE == db)).distinct().collect()]
        dict_table[table]=list_primary_key
        indi_schema_list.append(dict_table)
    dict_schema[db]=indi_schema_list
    cp_dict_schema=dict_schema.copy()

    validation_tables_info_2.append(cp_dict_schema)
print(validation_tables_info_2)

[{'STAGING': [{'ADDRESS_LINK': ['ADDR_ID', 'ADDR_TYPE']}]}]


In [0]:
gold_df.createOrReplaceTempView("gold_df")
display(gold_df)

FWK_TABLE_ID,DATABASE,SCHEMA,TABLE_NAME,KEY_COLUMN,ACTIVE_FLAG,PARTITION_COLUMN,KEY_TYPE,KEY_TYPE_UNIQUE,PARTITION_ON,PARTITION_QUERY_NETEZZA,PARTITION_QUERY_SYNAPSE,FIELD_VALIDATION_QUERY_NETEZZA,FIELD_VALIDATION_QUERY_SYNAPSE,MEASURE_QUERY_NETEZZA,MEASURE_QUERY_SYNAPSE,PARTITON_QUERY_NETEZZA_SYNAPSE_SEVEN_YEARS,MEASURE_COLUMN,MEASURE_QUERY_NETEZZA_SEVEN_YEARS,MEASURE_QUERY_SYNAPSE_SEVEN_YEARS,ADLS_QUERY_FOR_FIELD,ADLS_QUERY_FOR_MEASURE,Filter7
19945,STAGING,ADMIN,ADDRESS_LINK,ADDR_ID,Y,SOURCE_CREATE_DATE,PK,N,DAY,"SELECT to_char(SOURCE_CREATE_DATE, 'YYYY Mon DD') AS SOURCE_CREATE_DATE, COUNT(*) as CNT FROM db_tmp.ADMIN.ADDRESS_LINK GROUP BY to_char(SOURCE_CREATE_DATE, 'YYYY Mon DD')",null,"SELECT * FROM db_tmp.ADMIN.ADDRESS_LINK WHERE to_char(SOURCE_CREATE_DATE, 'YYYY Mon DD')='2023 Mar 15'",null,null,null,null,null,null,null,"SELECT * from Data_Lake_df WHERE DATE_FORMAT(SOURCE_CREATE_DATE, 'y MMM dd') ='2023 Mar 15'",null,null
19946,STAGING,ADMIN,ADDRESS_LINK,ADDR_TYPE,N,SOURCE_CREATE_DATE,PK,Y,DAY,"SELECT to_char(SOURCE_CREATE_DATE, 'YYYY Mon DD') AS SOURCE_CREATE_DATE, COUNT(*) as CNT FROM db_tmp.ADMIN.ADDRESS_LINK GROUP BY to_char(SOURCE_CREATE_DATE, 'YYYY Mon DD')",null,"SELECT * FROM db_tmp.ADMIN.ADDRESS_LINK WHERE to_char(SOURCE_CREATE_DATE, 'YYYY Mon DD')='2023 Mar 15'",null,null,null,null,null,null,null,"SELECT * from Data_Lake_df WHERE DATE_FORMAT(SOURCE_CREATE_DATE, 'y MMM dd') ='2023 Mar 15'",null,null


In [0]:
import pandas as pd 
from pyspark.sql.functions import col
import pyspark.pandas as ps

def get_pandas(spark_df):
    #find all decimal columns in your SparkDF
    decimals_cols = [c for c in spark_df.columns if 'Decimal' in str(spark_df.schema[c].dataType)]
    timestamp_cols = [c for c in spark_df.columns if 'Timestamp' in str(spark_df.schema[c].dataType)]

    #convert all decimals columns to floats
    for col in decimals_cols:
        spark_df = spark_df.withColumn(col, spark_df[col].cast(FloatType()))

    #Convert all timestamp columns to string
    for col in timestamp_cols:
        spark_df = spark_df.withColumn(col, spark_df[col].cast(StringType()))
    
    #pd_df = spark_df.toPandas()
    pd_df = ps.DataFrame(spark_df)
    return pd_df


In [0]:
def trim_space(spark_df):
    string_cols = [c for c in spark_df.columns if 'String' in str(spark_df.schema[c].dataType)]

    for col in string_cols:
        spark_df = spark_df.withColumn(col, trim(col))

    return spark_df 

In [0]:
import pyspark.sql.functions as F
from pyspark.sql import Row
import datetime
from pyspark.sql.types import StructType, StructField, StringType
import pyspark.pandas as ps 


def find_Missing(data1, data2, id_columns): 
    data1 = data1.select( *[ F.when(F.col(column).isNull(),'').otherwise(F.col(column)).alias(column) for column in data1.columns])
    data2 = data2.select( *[ F.when(F.col(column).isNull(),'').otherwise(F.col(column)).alias(column) for column in data2.columns])

    cols = data1.columns

    print('WORKING ON MISSING RECORDS')
    append_str='tgt_'
    pre_res = [append_str + sub for sub in id_columns]
    pre_res1 = [append_str + sub for sub in cols]
    data_difference_1 = data1.subtract(data2)
    data_difference_1.cache()
    #join_on_col = list(set(id_columns).intersection(set(data_difference_1.columns)))
    data_difference_1 = data_difference_1.join(data2.select(*[F.col(i).alias(f'tgt_{i}') if i not in id_columns else F.col(i).alias(i) for i in cols]), on = id_columns, how = 'left')
    data_difference_1.cache()

    data_difference_1 = Missing_condition(pre_res1,pre_res,data_difference_1)
    data_difference_1.cache()
    #counts = data_difference_1.select([F.count(i).alias(i) for i in data_difference_1.columns]).toPandas()
    counts = ps.DataFrame(data_difference_1.select([F.count(i).alias(i) for i in data_difference_1.columns]))
    
    
    output = data_difference_1.select(*counts.columns[counts.ne(0).iloc[0]]).orderBy(id_columns)
    output.cache()
    cnt=output.count() 
    output_pd = get_pandas(output)   #.sort_values(by=id_columns)
    output_pd.spark.cache()
    '''
    ## TEMP - ONLY for TESTING LOGIC
    cnt = 0
    ## END
    '''
    if cnt == 0:
        columns = StructType([StructField('Missing Records',
                                  StringType(), True)])
        output = spark.createDataFrame(data = [],
                           schema = columns)
        output_pd = get_pandas(output)
        output_pd.spark.cache()
        print("No Missing Records Found")
        doc.add_paragraph("No Missing Records Found", style='List Bullet')
    elif cnt > max_excel_row:
        output.write.format('delta').mode('overwrite').save(path.replace("/dbfs", '')+f"{container}_{db}_{table_name}_missing_records")


    '''
    ## Convert to pandas dataframe
    #output = convertDecimalToFloat(output).toPandas()
    output = get_pandas(output.limit(max_excel_row))
    output.spark.cache()
    '''

    
    
    return cnt, output_pd
     


def find_Mismatched(data1, data2, id_columns):
    data1 = data1.select( *[ F.when(F.col(column).isNull(),'').otherwise(F.col(column)).alias(column) for column in data1.columns])
    data2 = data2.select( *[ F.when(F.col(column).isNull(),'').otherwise(F.col(column)).alias(column) for column in data2.columns])

    print('WORKING ON MISMATCHED RECORDS')
    cols = data1.columns
    data_difference = data2.subtract(data1) # First you find the difference in 
    data_difference.cache()
    print(data_difference.columns)
    #join_on_col = list(set(id_columns).intersection(set(data_difference.columns)))
    print('id_columns', id_columns)
    data_difference = data_difference.join(data1.select(*[F.col(i).alias(f'SRC_{i}') if i not in id_columns else F.col(i).alias(i) for i in cols]), on = id_columns, how = 'left')
    data_difference.cache()
    mismatched_record_df = data_difference.alias("mismatched_record_df").orderBy(id_columns)
    #display(mismatched_record_df.limit(5))
    mismatched_record_df.cache()
    cnt_diff = mismatched_record_df.count()
    mismatched_record_df_pd = get_pandas(mismatched_record_df)   #.sort_values(by=id_columns)
    mismatched_record_df_pd.spark.cache()
    '''
    ## TEMP - ONLY for TESTING LOGIC
    cnt_diff = 0
    ## END
    '''
    if cnt_diff == 0:
        columns = StructType([StructField('Mismatched Records', StringType(), True)])
        mismatched_record_df = spark.createDataFrame(data = [], schema = columns)
        mismatched_record_df_pd = get_pandas(mismatched_record_df)
        mismatched_record_df_pd.spark.cache()
        print("No Mismatched Records Found")
        doc.add_paragraph("No Mismatched Records Found", style='List Bullet')
    elif cnt_diff > max_excel_row:
        mismatched_record_df.write.format('delta').mode("overwrite").save(path.replace("/dbfs", '')+f"{container}_{db}_{table_name}_mismatched_records")
        
    
    print('WORKING ON MISMATCHED RECORDS DETAILS')
    # you join the differences with the second dataframe
    #final_df=data_difference.select( *[ F.when(F.col(column).isNull(),'').otherwise(F.col(column)).alias(column) for column in data_difference.columns]).orderBy(id_columns)
    #display(final_df.limit(5))
    #final_df.cache()
    final_df=data_difference.select(*[is_different(x, f"SRC_{x}",data_difference.columns, data_difference) for x in cols]).orderBy(id_columns)
    #display(final_df.limit(5))
    final_df.cache()
    cnt_final=final_df.count()
    final_df_pd = get_pandas(final_df)   #.sort_values(by=id_columns)
    final_df_pd.spark.cache()
    '''
    ## TEMP - ONLY for TESTING LOGIC
    cnt_final = 0
    ## END
    '''
    if cnt_final == 0:
        columns = StructType([StructField('Mismatched Records Details', StringType(), True)])
        final_df = spark.createDataFrame(data = [], schema = columns)
        final_df_pd = get_pandas(final_df)
        final_df_pd.spark.cache()
        print("No Mismatched Records Found")
        doc.add_paragraph("No Mismatched Records Found", style='List Bullet')
    elif cnt_final > max_excel_row:
        final_df.write.format('delta').mode('overwrite').save(path.replace("/dbfs", '')+f"{container}_{db}_{table_name}_mismatched_details")

    '''
    ## Convert to pandas dataframe
    #mismatched_record_df = convertDecimalToFloat(mismatched_record_df).toPandas()
    mismatched_record_df = get_pandas(mismatched_record_df.limit(max_excel_row))
    mismatched_record_df.spark.cache()
    #final_df = convertDecimalToFloat(final_df).toPandas()
    final_df = get_pandas(final_df.limit(max_excel_row))
    final_df.spark.cache
    '''

    return cnt_diff, mismatched_record_df_pd, cnt_final, final_df_pd
    

   

In [0]:
from pyspark.sql.types import FloatType
from pyspark.sql.functions import col



def find_difference(data1, data2, id_columns):
    data1 = data1.select( *[ F.when(F.col(column).isNull(),'').otherwise(F.col(column)).alias(column) for column in data1.columns])
    data2 = data2.select( *[ F.when(F.col(column).isNull(),'').otherwise(F.col(column)).alias(column) for column in data2.columns])

    print('WORKING ON MISMATCHED RECORDS')
    cols = data1.columns
    data_difference = data2.subtract(data1)
    data_difference.cache() # First you find the difference in rows
    #join_on_col = list(set(id_columns).intersection(set(data_difference.columns)))
    data_difference = data_difference.join(data1.select(*[F.col(i).alias(f'SRC_{i}') if i not in id_columns else F.col(i).alias(i) for i in cols]), on = id_columns, how = 'left')
    data_difference.cache()
    mismatched_record_df = data_difference.alias("mismatched_record_df").orderBy(id_columns)
    #display(mismatched_record_df.limit(5))
    mismatched_record_df.cache()
    cnt_diff = mismatched_record_df.count()
    #print(cnt_diff)
    #mismatched_record_df = get_pandas(mismatched_record_df.limit(max_excel_row))
    mismatched_record_df_pd = get_pandas(mismatched_record_df)
    mismatched_record_df_pd.spark.cache()
    '''
    ## TEMP - ONLY for TESTING LOGIC
    cnt_diff = 0
    ## END
    '''
    if cnt_diff == 0:
        columns = StructType([StructField('Mismatched Records', StringType(), True)])
        mismatched_record_df = spark.createDataFrame(data = [], schema = columns)
        mismatched_record_df_pd = get_pandas(mismatched_record_df)
        mismatched_record_df_pd.spark.cache()
        print("No Mismatched Records Found")
        doc.add_paragraph("No Mismatched Records Found", style='List Bullet')
    elif cnt_diff > max_excel_row:
        mismatched_record_df.write.format('delta').mode("overwrite").save(path.replace("/dbfs", '')+f"{container}_{db}_{table_name}_mismatched_records")
        
    

    print('WORKING ON MISMATCHED RECORDS DETAILS')
    # you join the differences with the second dataframe
    #final_df=data_difference.select( *[ F.when(F.col(column).isNull(),'').otherwise(F.col(column)).alias(column) for column in data_difference.columns]).orderBy(id_columns)
    #display(final_df.limit(5))
    #final_df.cache()
    final_df=data_difference.select(*[is_different(x, f"SRC_{x}",data_difference.columns, data_difference) for x in cols]).orderBy(id_columns)
    #display(final_df.limit(5))
    final_df.cache()
    cnt_final=final_df.count()
    #final_df = get_pandas(final_df.limit(max_excel_row))  #
    final_df_pd = get_pandas(final_df)
    final_df_pd.spark.cache()
    '''
    ## TEMP - ONLY for TESTING LOGIC
    cnt_final = 0
    ## END
    '''
    if cnt_final == 0:
        columns = StructType([StructField('Mismatched Records Details', StringType(), True)])
        final_df = spark.createDataFrame(data = [], schema = columns)
        final_df_pd = get_pandas(final_df)
        final_df_pd.spark.cache()
        print("No Mismatched Records Found")
        doc.add_paragraph("No Mismatched Records Found", style='List Bullet')
    elif cnt_final > max_excel_row:
        final_df.write.format('delta').mode('overwrite').save(path.replace("/dbfs", '')+f"{container}_{db}_{table_name}_mismatched_details")
        
        


    ## Find missing records
    print('WORKING ON MISSING RECORDS')
    append_str='tgt_'
    pre_res = [append_str + sub for sub in id_columns]

    pre_res1 = [append_str + sub for sub in cols]
    #print(type(pre_res1))
    data_difference_1 = data1.subtract(data2)
    data_difference_1.cache()
    #join_on_col = list(set(id_columns).intersection(set(data_difference_1.columns)))
    data_difference_1 = data_difference_1.join(data2.select(*[F.col(i).alias(f'tgt_{i}') if i not in id_columns else F.col(i).alias(i) for i in cols]), on = id_columns, how = 'left')
    data_difference_1.cache()


    data_difference_1 = Missing_condition(pre_res1,pre_res,data_difference_1)
    data_difference_1.cache()
    
    #counts = data_difference_1.select([F.count(i).alias(i) for i in data_difference_1.columns]).toPandas()
    counts = ps.DataFrame(data_difference_1.select([F.count(i).alias(i) for i in data_difference_1.columns]))
    
    
    output = data_difference_1.select(*counts.columns[counts.ne(0).iloc[0]]).orderBy(id_columns)
    output.cache()
    cnt=output.count()
    #output = get_pandas(output.limit(max_excel_row))  #
    output_pd = get_pandas(output)
    output_pd.spark.cache()
    
    '''
    ## TEMP - ONLY for TESTING LOGIC
    cnt = 0
    ## END
    '''
    if cnt == 0:
        columns = StructType([StructField('Missing Records',
                                  StringType(), True)])
        output = spark.createDataFrame(data = [],
                           schema = columns)
        output_pd = get_pandas(output)
        output_pd.spark.cache()
        print("No Missing Records Found")
        doc.add_paragraph("No Missing Records Found", style='List Bullet')
    elif cnt > max_excel_row:
        output.write.format('delta').mode('overwrite').save(path.replace("/dbfs", '')+f"{container}_{db}_{table_name}_missing_records")
       

    '''
    ## Convert to pandas dataframe
    #mismatched_record_df = convertDecimalToFloat(mismatched_record_df).toPandas()
    mismatched_record_df = get_pandas(mismatched_record_df.limit(max_excel_row))
    mismatched_record_df.spark.cache()
    #final_df = convertDecimalToFloat(final_df).toPandas()
    final_df = get_pandas(final_df.limit(max_excel_row))
    final_df.spark.cache()
    #output = convertDecimalToFloat(output).toPandas()
    output = get_pandas(output.limit(max_excel_row))
    output.spark.cache()
    '''

    return cnt_diff, mismatched_record_df_pd, cnt_final, final_df_pd, cnt, output_pd

In [0]:
netezza_synapse_database_name = []
netezza_synapse_table_name = []
netezza_synapse_partition_count_mismatched_cnt = []
netezza_synapse_measure_level_mismatched_cnt_diff = []
netezza_synapse_partition_level_mismatched_cnt_diff = []
netezza_synapse_partition_level_missing_cnt = []
netezza_synapse_table_level_mismatched_cnt_diff = []
netezza_synapse_table_level_missing_cnt = []
netezza_synapse_measure_level_diff_cnt = []
netezza_synapse_partition_level_diff_cnt = []
netezza_synapse_table_level_diff_cnt = []
netezza_synapse_detailed_report_path = []

In [0]:
tab_sy_cnt = 0

In [0]:
#Netezza Tables with  Synapse Comparsion.
from pyspark.sql.functions import *
from pyspark.sql.types import DecimalType, IntegerType, DateType, StructType, TimestampType
from datetime import date, timedelta, datetime
from delta.tables import *

# Header for log document
doc.add_heading('Log From Detailed Report Generation Comparing Netezza v. Synapse', 2)
doc.add_heading('KEY TYPE UNIQUE = Y', 3)
container = 'synapse'

try:

    for db in relevant_gold_db:

        if Env == 'QA':
            db_tmp = f"GDC_{db}"
        elif Env == 'PROD_DR':
            db_tmp = f"{db}_DR"
        else:
            db_tmp = db
        print('DATABASE: ', db)


        for table_dict in [v for d in validation_tables_info_2 for k,v in d.items() if k == db][0]:
            #print(table_dict)
            table_name=list(table_dict.keys())[0]
            print('TABLE:',table_name)      
 

            try:
                partition_key=list(table_dict.values())[0][0]
                #print(type(partition_key))
                primary_key=list(table_dict.values())[0]
                print('PRIMARY KEY:', primary_key)
                Partition_column=spark.sql(f"select distinct PARTITION_COLUMN from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and PARTITION_COLUMN is not null")
                KEY_TYPE=spark.sql(f"select distinct KEY_TYPE_UNIQUE from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and KEY_TYPE_UNIQUE is not null")
                KEY_TYPES=KEY_TYPE.collect()[0][0]

            
        
                if KEY_TYPES == "Y" :   
                    tab_sy_cnt += 1
                    print(tab_sy_cnt)

                    doc.add_heading(f"{db}.{table_name}", 4)             
                
                    ## Check if a partition column is configured for this table
                    print(f'Checking if a partition column is configured for {db}.{table_name}')
                    doc.add_paragraph(f'Checking if a partition column is configured for {db}.{table_name}', style='List Bullet')
                    if Partition_column.count() == 0:
                        Partition_col = None
                        print(f'No partition column configured for {db}.{table_name}')
                        doc.add_paragraph(f'No partition column configured for {db}.{table_name}', style='List Bullet')
                    else:
                        Partition_col =Partition_column.collect()[0][0]
                        print(f'Partition column configured for {db}.{table_name} is {Partition_col}')
                        doc.add_paragraph(f'Partition column configured for {db}.{table_name} is {Partition_col}', style='List Bullet')
                

                    if ((Partition_col != None) and (Partition_col != 'NA') ):
                        ## Retrieve partition, measure and field validation queries from metadata table
                        print(f'Retrieving Partition, Measure and Field Validation Queries from Fwk_Gold_Tables for {db}.{table_name}')
                        doc.add_paragraph(f'Retrieving Partition, Measure and Field Validation Queries from Fwk_Gold_Tables for {db}.{table_name}', style='List Bullet')
                        if (PL_TL == '7GS'):  
                            Query_Agg_Netezza=spark.sql(f"select distinct PARTITON_QUERY_NETEZZA_SYNAPSE_SEVEN_YEARS from gold_df \
                                                    where TABLE_NAME='{table_name}' and DATABASE ='{db}' and PARTITON_QUERY_NETEZZA_SYNAPSE_SEVEN_YEARS is not null") 
                            Query_Agg_Synapse=spark.sql(f"select distinct PARTITION_QUERY_SYNAPSE from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}'\
                                                    and PARTITION_QUERY_SYNAPSE is not null")    
                            Query_Measure_Netezza=spark.sql(f"select distinct MEASURE_QUERY_NETEZZA_SEVEN_YEARS from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}'\
                                                            and MEASURE_QUERY_NETEZZA_SEVEN_YEARS is not null")
                            Query_Measure_Synapse=spark.sql(f"select distinct MEASURE_QUERY_SYNAPSE from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}'\
                                                    and MEASURE_QUERY_SYNAPSE is not null")                      
                        else :
                            #below is for NON transactional and transactional where we have full data in synapse as well
                            Query_Agg_Netezza=spark.sql(f"select distinct PARTITION_QUERY_NETEZZA from gold_df where TABLE_NAME = '{table_name}' and DATABASE ='{db}' and PARTITION_QUERY_NETEZZA is not null")
                            Query_Agg_Synapse=spark.sql(f"select distinct PARTITION_QUERY_SYNAPSE from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and PARTITION_QUERY_SYNAPSE is not null")  
                            Query_Measure_Netezza=spark.sql(f" select distinct MEASURE_QUERY_NETEZZA from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and MEASURE_QUERY_NETEZZA is not null")
                            Query_Measure_Synapse=spark.sql(f" select distinct MEASURE_QUERY_SYNAPSE from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and MEASURE_QUERY_SYNAPSE is not null")

                        Query_Field_Level_Netezza=spark.sql(f"select distinct FIELD_VALIDATION_QUERY_NETEZZA from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and FIELD_VALIDATION_QUERY_NETEZZA is not null")
                        Query_Field_Level_Synapse=spark.sql(f"select distinct FIELD_VALIDATION_QUERY_SYNAPSE from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and FIELD_VALIDATION_QUERY_SYNAPSE is not null")   

                        ## Skip table if no queries are configured
                        if (Query_Field_Level_Netezza.count() == 0) & (Query_Measure_Netezza.count() == 0) & (Query_Agg_Netezza.count() == 0):
                            print(f"No field validation query or measure query or partition query configured. No detailed report generated for {db}.{table_name}")
                            doc.add_paragraph(f"No field validation query or measure query or partition query configured. No detailed report generated for {db}.{table_name}", style = 'List Bullet')
                            continue

                        '''
                        ## Retrieve date part used for table partition (YEAR, MONTH, DAY)
                        PARTITION_ON=spark.sql(f"select PARTITION_ON from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and PARTITION_ON is not null")
                        Partition_Type=PARTITION_ON.collect()[0][0].upper()
                        print('PARTITION TYPE:', Partition_Type)
                        '''
                        

                        ## PERFORM PARTITION COUNT VALIDATION
                        
                        if (Query_Agg_Netezza.count() == 0) | (Query_Agg_Synapse.count() == 0):
                            print(f'No partition queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or Synapse. No partition count validation performed.')
                            doc.add_paragraph(f'No partition queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or Synapse. No partition count validation performed.', style='List Bullet')
                            partition_count_validation_status = 'False'
                        else:
                            query_PARTITION_CNT_NZ = Query_Agg_Netezza.collect()[0][0]
                            query_PARTITION_CNT_SY = Query_Agg_Synapse.collect()[0][0]
                            
                            if (query_PARTITION_CNT_NZ == '') | (query_PARTITION_CNT_SY == ''):
                                partition_count_validation_status = 'False'
                                print('No Partition Query is configured. No partition count validation performed')
                                doc.add_paragraph(('No Partition Query is configured. No partition count validation performed'), style= 'List Bullet')
                            else:
                                print(f'Attempting to perform partition count validation for {db}.{table_name}')
                                doc.add_paragraph(f'Attempting to perform partition count validation for {db}.{table_name}', style='List Bullet')

                                try:
                                    query_PARTITION_CNT_NZ = query_PARTITION_CNT_NZ.replace('db_tmp', db_tmp)
                                    netezza_partition_cnt_df = readfromNetizza(Env, query_PARTITION_CNT_NZ, db)
                                    synapse_partition_cnt_df = readfromSynapse(Env,query_PARTITION_CNT_SY)

                                    try:
                                        partition_count_mismatched_cnt, partition_count_mismatched_df, partition_count_mismatched_detail_cnt, partition_count_mismatched_detail_df = find_Mismatched(netezza_partition_cnt_df,synapse_partition_cnt_df,Partition_col)
                                        print(f"Successfully performed partition count validation for {db}.{table_name}")
                                        doc.add_paragraph(f"Successfully performed partition count validation for {db}.{table_name}", style = 'List Bullet')
                                        partition_count_validation_status = 'True'
                                    except Exception as e:
                                        partition_count_validation_status = 'False'
                                        print(f'Failed to perform partition count validation for {db}.{table_name}')
                                        doc.add_paragraph(f'Failed to perform partition count validation for {db}.{table_name}', style='List Bullet')
                                        print(e)
                                        doc.add_paragraph(f'{e}', style='Intense Quote')
                                except Exception as e:
                                    partition_count_validation_status = 'False'
                                    print(f'Bad partition query configured for {db}.{table_name}. Please correct the query. No partition count validation performed')
                                    doc.add_paragraph(f'Bad partition query configured for {db}.{table_name}. Please correct the query. No partition count validation performed', style = 'List Bullet')
                                    print(e)
                                    doc.add_paragraph(f'{e}', style='Intense Quote')



                        
                        '''
                        if (PL_TL == '7GS'):
                            if Query_Agg_Netezza.count()!=0:
                                query_PARTITION_CNT_NZ = Query_Agg_Netezza.collect()[0][0]
                            else:
                                query_PARTITION_CNT_NZ = ''
                                print('No Partition_Query_Netezza_Synapse_Seven_Years is configured')
                                doc.add_paragraph('No Partition_Query_Netezza_Synapse_Seven_Years is configured', style='List Bullet')
                            if Partition_Type == 'YEAR' :
                                query_PARTITION_CNT_SY= f"""select FORMAT({Partition_col}, 'yyyy') as {Partition_col}, count(*) as CNT from {db}.{table_name} group by FORMAT({Partition_col}, 'yyyy')"""
                            elif Partition_Type == 'MONTH' :
                                query_PARTITION_CNT_SY=f"""select FORMAT({Partition_col}, 'yyyy MMM') as {Partition_col}, count(*) as CNT from {db}.{table_name} group by FORMAT({Partition_col}, 'yyyy MMM')"""
                            elif Partition_Type == 'DAY' :
                                query_PARTITION_CNT_SY=f"""select FORMAT({Partition_col}, 'yyyy MMM dd') as {Partition_col}, count(*) as CNT from {db}.{table_name} group by FORMAT({Partition_col}, 'yyyy MMM dd')"""
                            
                        else:
                            if Partition_Type == 'YEAR' :
                                query_PARTITION_CNT_NZ = f"""select to_char({Partition_col}, 'YYYY') as {Partition_col}, count(*) as CNT from {db_tmp}.{table_name} group by to_char({Partition_col}, 'YYYY')"""
                                query_PARTITION_CNT_SY= f"""select FORMAT({Partition_col}, 'yyyy') as {Partition_col}, count(*) as CNT from {db}.{table_name} group by FORMAT({Partition_col}, 'yyyy')"""
                            elif Partition_Type == 'MONTH' :
                                query_PARTITION_CNT_NZ = f"""select to_char({Partition_col}, 'YYYY Mon') as {Partition_col}, count(*) as CNT from {db_tmp}.{table_name} group by to_char({Partition_col}, 'YYYY Mon')"""
                                query_PARTITION_CNT_SY=f"""select FORMAT({Partition_col}, 'yyyy MMM') as {Partition_col}, count(*) as CNT from {db}.{table_name} group by FORMAT({Partition_col}, 'yyyy MMM')"""
                            elif Partition_Type == 'DAY' :
                                query_PARTITION_CNT_NZ = f"""select to_char({Partition_col}, 'YYYY Mon dd') as {Partition_col}, count(*) as CNT from {db_tmp}.{table_name} group by to_char({Partition_col}, 'YYYY Mon dd')"""
                                query_PARTITION_CNT_SY=f"""select FORMAT({Partition_col}, 'yyyy MMM dd') as {Partition_col}, count(*) as CNT from {db}.{table_name} group by FORMAT({Partition_col}, 'yyyy MMM dd')"""
                            
                        if (query_PARTITION_CNT_NZ == '') | (query_PARTITION_CNT_SY == ''):
                            partition_count_validation_status = 'False'
                            print('No Partition Query is configured. No partition count validation performed')
                            doc.add_paragraph(('No Partition Query is configured. No partition count validation performed'), style= 'List Bullet')
                        else:
                            netezza_partition_cnt_df = readfromNetizza(Env, query_PARTITION_CNT_NZ, db)
                            synapse_partition_cnt_df = readfromSynapse(Env,query_PARTITION_CNT_SY)

                            try:
                                partition_count_mismatched_cnt, partition_count_mismatched_df, partition_count_mismatched_detail_cnt, partition_count_mismatched_detail_df = find_Mismatched(netezza_partition_cnt_df,synapse_partition_cnt_df,Partition_col)
                                print(f"Successfully performed partition count validation for {db}.{table_name}")
                                doc.add_paragraph(f"Successfully performed partition count validation for {db}.{table_name}", style = 'List Bullet')
                                partition_count_validation_status = 'True'
                            except Exception as e:
                                partition_count_validation_status = 'False'
                                print(f'Failed to perform partition count validation for {db}.{table_name}')
                                doc.add_paragraph(f'Failed to perform partition count validation for {db}.{table_name}', style='List Bullet')
                                print(e)
                                doc.add_paragraph(f'{e}', style='Intense Quote')
                        '''
                        
                           
                        
                    
                        ## PERFORM MEASURE LEVEL VALIDATION

                        if (Query_Measure_Netezza.count() == 0) | (Query_Measure_Synapse.count() == 0):
                            print(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or Synapse. No measure level validation performed.')
                            doc.add_paragraph(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or Synapse. No measure level validation performed.', style='List Bullet')
                            measure_validation_status = 'False'
                        else:
                            query_measure_netezza =Query_Measure_Netezza.collect()[0][0]
                            query_measure_synapse = Query_Measure_Synapse.collect()[0][0]
                            
                            if (query_measure_netezza == '') | (query_measure_synapse == ''):
                                print(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or Synapse. No measure level validation performed.')
                                doc.add_paragraph(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or Synapse. No measure level validation performed.', style='List Bullet')
                                measure_validation_status = 'False' 
                            else:
                                print(f'Attempting to perform measure level validation for {db}.{table_name}')
                                doc.add_paragraph(f'Attempting to perform measure level validation for {db}.{table_name}', style='List Bullet')

                                try:
                                    query_measure_netezza = query_measure_netezza.replace('db_tmp', db_tmp)
                                    netezza_df_measure=readfromNetizza(Env,query_measure_netezza,db)
                                    synapse_df_measure=readfromSynapse(Env,query_measure_synapse)

                                    try:
                                        measure_level_mismatched_cnt_diff, measure_level_mismatched_record_df, measure_level_mismatched_detail_cnt, measure_level_mismatched_detail_df = find_Mismatched(netezza_df_measure,synapse_df_measure,Partition_col)
                                        print(f"Successfully performed measure level validation for {db}.{table_name}")
                                        doc.add_paragraph(f"Successfully performed measure level validation for {db}.{table_name}", style='List Bullet')
                                        measure_validation_status = 'True'
                                    except Exception as e:
                                        measure_validation_status = 'False'
                                        print(f'Failed to perform measure level validation for {db}.{table_name}')
                                        doc.add_paragraph(f'Failed to perform measure level validation for {db}.{table_name}', style='List Bullet')
                                        print(e)
                                        doc.add_paragraph(f'{e}', style='Intense Quote')
                                except Exception as e:
                                    measure_validation_status = 'False'
                                    print(f'Bad measure query configured for {db}.{table_name}. Please correct the query. No measure level validation performed')
                                    doc.add_paragraph(f'Bad measure query configured for {db}.{table_name}. Please correct the query. No measure level validation performed', style = 'List Bullet')
                                    print(e)
                                    doc.add_paragraph(f'{e}', style='Intense Quote')
    


                        ## PERFORM PARTITION LEVEL VALIDATION
                        
                        if (Query_Field_Level_Netezza.count()==0) | (Query_Field_Level_Synapse.count()==0):
                            print(f'No field validation queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or Synapse. No partition level validation performed.')
                            doc.add_paragraph(f'No field validation queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or Synapse. No partition level validation performed.', style='List Bullet')
                            partition_validation_status = 'False'
                        else:
                            query_PARTITION_NZ = Query_Field_Level_Netezza.collect()[0][0]
                            query_PARTITION_SY = Query_Field_Level_Synapse.collect()[0][0]
                            if (query_PARTITION_NZ == '') | (query_PARTITION_SY == ''):
                                print(f'No field validation queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or Synapse. No partition level validation performed.')
                                doc.add_paragraph(f'No field validation queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or Synapse. No partition level validation performed.', style='List Bullet')
                                partition_validation_status = 'False'  
                            else:
                                try:
                                    query_PARTITION_NZ = query_PARTITION_NZ.replace('db_tmp', db_tmp)
                                    netezza_df_partition=readfromNetizza(Env,query_PARTITION_NZ,db)
                                    synapse_df_partition=readfromSynapse(Env,query_PARTITION_SY)  

                                    ## Validate that the partition size is within the max row permitted limit
                                    print('Attempting to perform partition level validation. Need to confirm that the partition size is within acceptable limit')
                                    doc.add_paragraph('Attempting to perform partition level validation. Need to confirm that the partition size is within acceptable limit', style = 'List Bullet')
                                    
                                    cnt_netezza_df_partition = netezza_df_partition.limit(max_rows_permitted + 1).count()
                                    cnt_synapse_df_partition = synapse_df_partition.limit(max_rows_permitted + 1).count()

                                    if (cnt_netezza_df_partition == 0) | (cnt_synapse_df_partition == 0):
                                        print(f'Partition for table {db}.{table_name} is empty either in Netezza or in Synapse. No partition validation performed')
                                        doc.add_paragraph(f'Partition for table {db}.{table_name} is empty either in Netezza or in Synapse. No partition validation performed', style = 'List Bullet')
                                        partition_validation_status = 'False'
                                    elif (cnt_netezza_df_partition > max_rows_permitted) | (cnt_synapse_df_partition > max_rows_permitted):
                                        print(f'Partition for {db}.{table_name} is greater than {max_rows_permitted}. No partition validation performed. Please reconfigure the partition for {db}.{table_name}')
                                        doc.add_paragraph(f'Partition for {db}.{table_name} is greater than {max_rows_permitted}. No partition validation performed. Please reconfigure the partition for {db}.{table_name}', style = 'List Bullet')
                                        partition_validation_status = 'False'
                                    else:
                                        ## Perform validation on the partition
                                        print(f'Partition for {db}.{table_name} is less than or equal to {max_rows_permitted}.')
                                        doc.add_paragraph(f'Partition for {db}.{table_name} is less than or equal to {max_rows_permitted}.', style = 'List Bullet')
                                        try:
                                            #partition_level_mismatched_cnt_diff, partition_level_mismatched_record_df, partition_level_mismatched_record_detail_cnt,partition_level_mismatched_record_detail_df, partition_level_missing_cnt, partition_level_missing_output_df = find_difference(netezza_df_partition, synapse_df_partition, primary_key)
                                            partition_level_mismatched_cnt_diff, partition_level_mismatched_record_df, partition_level_mismatched_record_detail_cnt,partition_level_mismatched_record_detail_df, partition_level_missing_cnt, partition_level_missing_output_df = find_difference(trim_space(netezza_df_partition), trim_space(synapse_df_partition), primary_key)
                                            print(f"Successfully performed partition level validation for {db}.{table_name}")
                                            doc.add_paragraph(f"Successfully performed partition level validation for {db}.{table_name}", style='List Bullet')
                                            partition_validation_status = 'True'
                                        except Exception as e:
                                            print(f'Failed to perform partiton level validation for {db}.{table_name}_(Netezza v. Synapse)')
                                            doc.add_paragraph(f'Failed to perform partiton level validation for {db}.{table_name}_(Netezza v. Synapse)', style='List Bullet')
                                            print(e)
                                            doc.add_paragraph(f'{e}', style='Intense Quote')
                                            partition_validation_status = 'False'
                                except Exception as e:
                                    partition_validation_status = 'False'
                                    print(f'Bad field validation query configured for {db}.{table_name}. Please correct the query. No partition level validation performed')
                                    doc.add_paragraph(f'Bad field validation query configured for {db}.{table_name}. Please correct the query. No partition level validation performed', style = 'List Bullet')
                                    print(e)
                                    doc.add_paragraph(f'{e}', style='Intense Quote')

                        
                        ## Define excel writer
                        try:
                            with pd.ExcelWriter(f"{db}-{table_name}_(Netezza v. Synapse).xlsx", engine="openpyxl", mode = 'w') as writer:

                                ## Write missing and mismatched information to sheets in excel workbook
                                if partition_validation_status == 'True':
                                    try:
                                        partition_level_missing_output_df.sort_values(by=primary_key).head(max_excel_row).to_excel(writer, sheet_name = f"missing_record_partition", startrow = 0, startcol=0, index = False)
                                    except:
                                        partition_level_missing_output_df.head(max_excel_row).to_excel(writer, sheet_name = f"missing_record_partition", startrow = 0, startcol=0, index = False)
                                    try:
                                        partition_level_mismatched_record_df.sort_values(by=primary_key).head(max_excel_row).to_excel(writer, sheet_name = f"mismatched_record_partition", startrow = 0, startcol=0, index = False)
                                        partition_level_mismatched_record_detail_df.sort_values(by=primary_key).head(max_excel_row).to_excel(writer, sheet_name = f"mismatched_detail_partition", startrow = 0, startcol=0, index = False)
                                    except:
                                        partition_level_mismatched_record_df.head(max_excel_row).to_excel(writer, sheet_name = f"mismatched_record_partition", startrow = 0, startcol=0, index = False)
                                        partition_level_mismatched_record_detail_df.head(max_excel_row).to_excel(writer, sheet_name = f"mismatched_detail_partition", startrow = 0, startcol=0, index = False)
                                if measure_validation_status == 'True':
                                    try:
                                        measure_level_mismatched_record_df.sort_values(by=Partition_col).head(max_excel_row).to_excel(writer, sheet_name = "measure_mismatched_record", startrow = 0, startcol=0, index = False)
                                        measure_level_mismatched_detail_df.sort_values(by=Partition_col).head(max_excel_row).to_excel(writer, sheet_name = "measure_mismatched_detail", startrow = 0, startcol=0, index = False)
                                    except:
                                        measure_level_mismatched_record_df.head(max_excel_row).to_excel(writer, sheet_name = "measure_mismatched_record", startrow = 0, startcol=0, index = False)
                                        measure_level_mismatched_detail_df.head(max_excel_row).to_excel(writer, sheet_name = "measure_mismatched_detail", startrow = 0, startcol=0, index = False)
                                if partition_count_validation_status == 'True':
                                    try:
                                        partition_count_mismatched_df.sort_values(by=Partition_col).head(max_excel_row).to_excel(writer, sheet_name = "partition_count_mismatched_record", startrow = 0, startcol=0, index = False)
                                        partition_count_mismatched_detail_df.sort_values(by=Partition_col).head(max_excel_row).to_excel(writer, sheet_name = "partition_count_mismatched_detail", startrow = 0, startcol=0, index = False)
                                    except:
                                        partition_count_mismatched_df.head(max_excel_row).to_excel(writer, sheet_name = "partition_count_mismatched_record", startrow = 0, startcol=0, index = False)
                                        partition_count_mismatched_detail_df.head(max_excel_row).to_excel(writer, sheet_name = "partition_count_mismatched_detail", startrow = 0, startcol=0, index = False)
                            print('Detailed report generated')
                            doc.add_paragraph('Detailed report generated', style = 'List Bullet')
                            detailed_report_status = 'True'
                        except Exception as e:
                            detailed_report_status = 'False'
                            print('No detailed report generated')
                            doc.add_paragraph('No detailed report generated', style = 'List Bullet')
                            print(e)
                            doc.add_paragraph(f'{e}', style= 'Intense Quote')
                                

                        ## Upload workbook to storage
                        if detailed_report_status == 'True':
                            try:        
                                workbook_path = path + f"{db}-{table_name}_(Netezza v. Synapse).xlsx" 
                                shutil.copy2(f"{db}-{table_name}_(Netezza v. Synapse).xlsx",workbook_path)
                                os.remove(f"{db}-{table_name}_(Netezza v. Synapse).xlsx")

                                ## Append detail summary to lists
                                netezza_synapse_database_name.append(db)
                                netezza_synapse_table_name.append(table_name)
                                if partition_count_validation_status == 'True':
                                    netezza_synapse_partition_count_mismatched_cnt.append(str(partition_count_mismatched_cnt))
                                else:
                                    netezza_synapse_partition_count_mismatched_cnt.append('')
                                if measure_validation_status == 'True':
                                    netezza_synapse_measure_level_mismatched_cnt_diff.append(str(measure_level_mismatched_cnt_diff))
                                else:
                                    netezza_synapse_measure_level_mismatched_cnt_diff.append('')
                                if partition_validation_status == 'True':
                                    netezza_synapse_partition_level_mismatched_cnt_diff.append(str(partition_level_mismatched_cnt_diff))
                                    netezza_synapse_partition_level_missing_cnt.append(str(partition_level_missing_cnt))
                                else:
                                    netezza_synapse_partition_level_mismatched_cnt_diff.append('')
                                    netezza_synapse_partition_level_missing_cnt.append('')
                                netezza_synapse_table_level_mismatched_cnt_diff.append('')
                                netezza_synapse_table_level_missing_cnt.append('')
                                netezza_synapse_partition_level_diff_cnt.append('')
                                netezza_synapse_table_level_diff_cnt.append('')
                                netezza_synapse_detailed_report_path.append(workbook_path)

                                print(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Synapse)")
                                doc.add_paragraph(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Synapse)", style='List Bullet')
                            

                            except Exception as e:
                                print(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Synapse)")
                                doc.add_paragraph(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Synapse)", style='List Bullet')
                                print(e)
                                doc.add_paragraph(f'{e}', style='Intense Quote')
                                
            
                    else :
                        ## This condition is to perform detailed validation for small and medium size tables
                        print(f'Attempting to perform detailed validation on {db}.{table_name}_(Netezza v. Synapse). Need to confirm it is a small to medium size table')
                        if (PL_TL == '7GS'):

                            ## Retrieve measure query                        
                            Query_Measure_Netezza=spark.sql(f"select distinct MEASURE_QUERY_NETEZZA_SEVEN_YEARS from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}'\
                                                            and MEASURE_QUERY_NETEZZA_SEVEN_YEARS is not null")
                            Query_Measure_Synapse=spark.sql(f"select distinct MEASURE_QUERY_SYNAPSE from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}'\
                                                    and MEASURE_QUERY_SYNAPSE is not null")

                            ## Retrieve the retention column
                            filterColumn=gold_df.select('Filter7').where((gold_df.TABLE_NAME==table_name)&(gold_df.DATABASE==db)).distinct().rdd.map(lambda x: x.Filter7).collect()
                            filterColumn=str(filterColumn)
                            filterColumn=filterColumn.replace("[","").replace("]","").replace("'","")
                            ## BELOW is fetch the minimum date from synapse to avoid mismatch on the day where data is not loaded in netizza
                            print(filterColumn)

                            ## Retrieve the retention value
                            query_forValuefilter= f"""select CAST(min({filterColumn}) as varchar(10)) {filterColumn} from {db}.{table_name}""" 
                        
                            if filterColumn!='None':
                                filterValue1=readfromSynapse(Env,query_forValuefilter)
                                filterValue=filterValue1.rdd.map(lambda x: x[0]).collect()
                                filterValue=str(filterValue)
                                filterValue=filterValue.replace("[","").replace("]","")
                            else:
                                filterValue='None'

                            ## Query netezza table based on retention values. Get netezza table for validation
                            if filterValue!='None':
                                Query_Netezza = f"""select * from {db_tmp}.{table_name} where {filterColumn} >= {filterValue}""" 
                            else:
                                Query_Netezza = f"""select * from {db_tmp}.{table_name}"""
                            #below is filter for 7 years for netizza 
                            print(Query_Netezza)
                            netezza_df=readfromNetizza(Env,Query_Netezza,db)

                            ## Get synapse table for validation
                            Query_Synapse = f"""select * from {db}.{table_name}"""
                            df_synapse=readfromSynapse(Env,Query_Synapse)

                        else:

                            ## Retrieve measure query                        
                            Query_Measure_Netezza=spark.sql(f"select distinct MEASURE_QUERY_NETEZZA from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}'\
                                                            and MEASURE_QUERY_NETEZZA is not null")
                            Query_Measure_Synapse=spark.sql(f"select distinct MEASURE_QUERY_SYNAPSE from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}'\
                                                    and MEASURE_QUERY_SYNAPSE is not null")

                            ## Query full netezza table. Get netezza table for validation
                            Query_Netezza = f"""select * from {db_tmp}.{table_name}"""
                            netezza_df=readfromNetizza(Env,Query_Netezza,db)

                            ## Get synapse table for validation
                            Query_Synapse = f"""select * from {db}.{table_name}"""
                            #print(Query_Synapse)
                            df_synapse=readfromSynapse(Env,Query_Synapse)


                            

                        ## Confirm table is small or medium size (i.e. Not greater than 5 million rows)
                        
                        try:
                            ## Retrieve row count
                            cnt_netezza_df = int(report_dataframe_1.select(f'NETEZZA_TABLE_ROW_COUNT ({netezza_sub})').filter((col('NETEZZA_SCHEMA') == f'{db}') & (col('NETEZZA_TABLE') == f'{table_name}')).collect()[0][0]) #netezza_df.limit(max_rows_permitted + 1).count()
                            cnt_df_synapse = int(report_dataframe_1.select('SYNAPSE_TABLE_ROW_COUNT').filter((col('SYNAPSE_SCHEMA') == f'{db}') & (col('SYNAPSE_TABLE') == f'{table_name}')).collect()[0][0])#df_synapse.limit(max_rows_permitted + 1).count()
                        except:
                            print(f'{db}.{table_name} not available in summary report')
                            cnt_netezza_df = netezza_df.limit(max_rows_permitted + 1).count()
                            cnt_df_synapse = df_synapse.limit(max_rows_permitted + 1).count()

                        if (cnt_netezza_df == 0) | (cnt_df_synapse == 0):
                            print(f'Table {db}.{table_name} is empty either in Netezza or in Synapse. No detailed validation performed')
                            doc.add_paragraph(f'Table {db}.{table_name} is empty either in Netezza or in Synapse. No detailed validation performed', style = 'List Bullet')

                        elif (cnt_netezza_df > max_rows_permitted) | (cnt_df_synapse > max_rows_permitted):
                            print(f'Table {db}.{table_name}_(Netezza v. Synapse) is a big table, with no partition column configured. No table level validation is performed')
                            doc.add_paragraph(f'Table {db}.{table_name}_(Netezza v. Synapse) is a big table, with no partition column configured. No table level validation is performed', style='List Bullet')

                            print('Will attempt to perform measure level validation')
                            doc.add_paragraph('Attempting measure level validation for a big table with no partition configured', style= 'List Bullet')

                            ## PERFORM MEASURE LEVEL VALIDATION
                            if (Query_Measure_Netezza.count() == 0) | (Query_Measure_Synapse.count() == 0):
                                print(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or Synapse. No measure level validation performed.')
                                doc.add_paragraph(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or Synapse. No measure level validation performed.', style='List Bullet')
                                measure_validation_status = 'False'
                            else:
                                query_measure_netezza =Query_Measure_Netezza.collect()[0][0]
                                query_measure_synapse = Query_Measure_Synapse.collect()[0][0]
                                if (query_measure_netezza == '') | (query_measure_synapse == ''):
                                    print(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or Synapse. No measure level validation performed.')
                                    doc.add_paragraph(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or Synapse. No measure level validation performed.', style='List Bullet')
                                    measure_validation_status = 'False' 
                                else:
                                    print(f'Measure queries configured. Attempting to perform measure level validation for {db}.{table_name}')
                                    doc.add_paragraph(f'Measure queries configured. Attempting to perform measure level validation for {db}.{table_name}', style='List Bullet')

                                    try:
                                        query_measure_netezza = query_measure_netezza.replace('db_tmp', db_tmp)
                                        netezza_df_measure=readfromNetizza(Env,query_measure_netezza,db)
                                        synapse_df_measure=readfromSynapse(Env,query_measure_synapse)

                                        try:
                                            measure_level_mismatched_record_df = get_pandas(synapse_df_measure.subtract(netezza_df_measure))
                                            measure_level_mismatched_cnt_diff = len(measure_level_mismatched_record_df)
                                            print(f"Successfully performed measure mismatched validation on {db}.{table_name}. This is a big table does not have a partition column configured")
                                            doc.add_paragraph(f"Successfully performed measure mismatched validation on {db}.{table_name}. This is a big table does not have a partition column configured", style='List Bullet')
                                            measure_validation_status = 'True'

                                            ## Define excel writer
                                            with pd.ExcelWriter(f"{db}-{table_name}_(Netezza v. Synapse).xlsx", engine="openpyxl", mode = 'w') as writer:

                                                ## Write missing and mismatched information to sheets in excel workbook
                                                measure_level_mismatched_record_df.head(max_excel_row).to_excel(writer, sheet_name = f"measure_mismatched_record", startrow = 0, startcol=0, index = False)

                                            ## Upload workbook to storage
                                            try:        
                                                workbook_path = path + f"{db}-{table_name}_(Netezza v. Synapse).xlsx" 
                                                shutil.copy2(f"{db}-{table_name}_(Netezza v. Synapse).xlsx",workbook_path)
                                                os.remove(f"{db}-{table_name}_(Netezza v. Synapse).xlsx")

                                                ## Append detail summary to lists
                                                netezza_synapse_database_name.append(db)
                                                netezza_synapse_table_name.append(table_name)
                                                netezza_synapse_partition_count_mismatched_cnt.append('')
                                                netezza_synapse_measure_level_mismatched_cnt_diff.append(str(measure_level_mismatched_cnt_diff))
                                                netezza_synapse_partition_level_mismatched_cnt_diff.append('')
                                                netezza_synapse_partition_level_missing_cnt.append('')
                                                netezza_synapse_table_level_mismatched_cnt_diff.append('')
                                                netezza_synapse_table_level_missing_cnt.append('')
                                                netezza_synapse_partition_level_diff_cnt.append('')
                                                netezza_synapse_table_level_diff_cnt.append('')
                                                netezza_synapse_detailed_report_path.append(workbook_path)

                                                print(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Synapse)")
                                                doc.add_paragraph(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Synapse)", style='List Bullet')

                                            except Exception as e:
                                                    print(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Synapse)")
                                                    doc.add_paragraph(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Synapse)", style='List Bullet')
                                                    print(e)
                                                    doc.add_paragraph(f'{e}', style='Intense Quote')
                                        except Exception as e:
                                                print(f"Failed to perform detailed validation on {db}.{table_name}")
                                                doc.add_paragraph(f"Failed to perform detailed validation on {db}.{table_name}", style='List Bullet')
                                                print(e)
                                                doc.add_paragraph(f'{e}', style='Intense Quote')
                                                continue
                                    except Exception as e:
                                        print(f'Bad measure query configured for {db}.{table_name}. Please correct the query. No measure level validation performed')
                                        doc.add_paragraph(f'Bad measure query configured for {db}.{table_name}. Please correct the query. No measure level validation performed', style = 'List Bullet')
                                        print(e)
                                        doc.add_paragraph(f'{e}', style='Intense Quote')
                            '''
                            try:

                                if Query_Measure_Netezza.count() != 0:
                                    query_measure_netezza =Query_Measure_Netezza.collect()[0][0]
                                    netezza_df_measure=readfromNetizza(Env,query_measure_netezza,db)
                                    #print("QUERY MEASURE NETEZZA:", query_measure_netezza)                            

                                    if Query_Measure_Synapse.count() != 0:
                                        query_measure_synapse = Query_Measure_Synapse.collect()[0][0]
                                        synapse_df_measure=readfromSynapse(Env,query_measure_synapse)

                                        try:
                                            measure_level_mismatched_record_df = get_pandas(synapse_df_measure.subtract(netezza_df_measure))
                                            measure_level_mismatched_cnt_diff = len(measure_level_mismatched_record_df)
                                            print(f"Successfully performed measure mismatched validation on {db}.{table_name}. This is a big table does not have a partition column configured")
                                            doc.add_paragraph(f"Successfully performed measure mismatched validation on {db}.{table_name}. This is a big table does not have a partition column configured", style='List Bullet')
                                            measure_validation_status = 'True'

                                            ## Define excel writer
                                            with pd.ExcelWriter(f"{db}-{table_name}_(Netezza v. Synapse).xlsx", engine="openpyxl", mode = 'w') as writer:

                                                ## Write missing and mismatched information to sheets in excel workbook
                                                measure_level_mismatched_record_df.head(max_excel_row).to_excel(writer, sheet_name = f"measure_mismatched_record", startrow = 0, startcol=0, index = False)

                                            ## Upload workbook to storage
                                            try:        
                                                workbook_path = path + f"{db}-{table_name}_(Netezza v. Synapse).xlsx" 
                                                shutil.copy2(f"{db}-{table_name}_(Netezza v. Synapse).xlsx",workbook_path)
                                                os.remove(f"{db}-{table_name}_(Netezza v. Synapse).xlsx")

                                                ## Append detail summary to lists
                                                netezza_synapse_database_name.append(db)
                                                netezza_synapse_table_name.append(table_name)
                                                netezza_synapse_partition_count_mismatched_cnt.append('')
                                                netezza_synapse_measure_level_mismatched_cnt_diff.append(str(measure_level_mismatched_cnt_diff))
                                                netezza_synapse_partition_level_mismatched_cnt_diff.append('')
                                                netezza_synapse_partition_level_missing_cnt.append('')
                                                netezza_synapse_table_level_mismatched_cnt_diff.append('')
                                                netezza_synapse_table_level_missing_cnt.append('')
                                                netezza_synapse_partition_level_diff_cnt.append('')
                                                netezza_synapse_table_level_diff_cnt.append('')
                                                netezza_synapse_detailed_report_path.append(workbook_path)

                                                print(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Synapse)")
                                                doc.add_paragraph(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Synapse)", style='List Bullet')

                                            except Exception as e:
                                                    print(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Synapse)")
                                                    doc.add_paragraph(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Synapse)", style='List Bullet')
                                                    print(e)
                                                    doc.add_paragraph(f'{e}', style='Intense Quote')
                                                
                                        except Exception as e:
                                            print(f"Failed to perform detailed validation on {db}.{table_name}")
                                            doc.add_paragraph(f"Failed to perform detailed validation on {db}.{table_name}", style='List Bullet')
                                            print(e)
                                            doc.add_paragraph(f'{e}', style='Intense Quote')
                                            continue
                                    else:
                                        print(f"No measure query configured for {db}.{table_name} in Synapse")
                                        doc.add_paragraph(f"No measure query configured for {db}.{table_name} in Synapse", style='List Bullet')
                                else:
                                    print(f"No measure query configured for {db}.{table_name} in Netezza")
                                    doc.add_paragraph(f"No measure query configured for {db}.{table_name} in Netezza", style='List Bullet')

                            except Exception as e:
                                measure_validation_status = 'False' 
                                print(f'Failed to perform measure level validation for {db}.{table_name}. Re-examine the configured measure queries')
                                doc.add_paragraph(f'Failed to perform measure level validation for {db}.{table_name}. Re-examine the configured measure queries', style='List Bullet')
                                print(e)
                                doc.add_paragraph(f'{e}', style='Intense Quote')
                            ''' 

                        else:
                            print(f'Table size is less than or equal to {max_rows_permitted}. Performing detailed validation on full table {db}.{table_name}')
                            doc.add_paragraph(f'Table size is less than or equal to {max_rows_permitted}. Performing detailed validation on full table {db}.{table_name}', style = 'List Bullet')
                            try:
                                #table_level_mismatched_cnt_diff, table_level_mismatched_record_df, table_level_mismatched_record_detail_cnt, table_level_mismatched_record_detail_df, table_level_missing_cnt, table_level_missing_output_df = find_difference(netezza_df, df_synapse, primary_key)
                                table_level_mismatched_cnt_diff, table_level_mismatched_record_df, table_level_mismatched_record_detail_cnt, table_level_mismatched_record_detail_df, table_level_missing_cnt, table_level_missing_output_df = find_difference(trim_space(netezza_df), trim_space(df_synapse), primary_key)
                                print(f'Successfully performed full table validation on {db}.{table_name}, comparing Netezza to Synapse table')
                                doc.add_paragraph(f'Successfully performed full table validation on {db}.{table_name}, comparing Netezza to Synapse table', style='List Bullet')

                                ## Define excel writer
                                with pd.ExcelWriter(f"{db}-{table_name}_(Netezza v. Synapse).xlsx", engine="openpyxl", mode = 'w') as writer:

                                    ## Write missing and mismatched information to sheets in excel workbook
                                    try:
                                        table_level_missing_output_df.sort_values(by=primary_key).head(max_excel_row).to_excel(writer, sheet_name = "missing_record", startrow = 0, startcol=0, index = False)
                                    except:
                                        table_level_missing_output_df.head(max_excel_row).to_excel(writer, sheet_name = "missing_record", startrow = 0, startcol=0, index = False)
                                    try:
                                        table_level_mismatched_record_df.sort_values(by=primary_key).head(max_excel_row).to_excel(writer, sheet_name = "mismatched_record", startrow = 0, startcol=0, index = False)
                                        table_level_mismatched_record_detail_df.sort_values(by=primary_key).head(max_excel_row).to_excel(writer, sheet_name = "mismatched_detail", startrow = 0, startcol=0, index = False)
                                    except:
                                        table_level_mismatched_record_df.head(max_excel_row).to_excel(writer, sheet_name = "mismatched_record", startrow = 0, startcol=0, index = False)
                                        table_level_mismatched_record_detail_df.head(max_excel_row).to_excel(writer, sheet_name = "mismatched_detail", startrow = 0, startcol=0, index = False)

                                    
                                ## Upload workbook to storage
                                try:        
                                    workbook_path = path + f"{db}-{table_name}_(Netezza v. Synapse).xlsx" 
                                    shutil.copy2(f"{db}-{table_name}_(Netezza v. Synapse).xlsx",workbook_path)
                                    os.remove(f"{db}-{table_name}_(Netezza v. Synapse).xlsx")

                                    ## Append detail summary to lists
                                    netezza_synapse_database_name.append(db)
                                    netezza_synapse_table_name.append(table_name)
                                    netezza_synapse_partition_count_mismatched_cnt.append('')
                                    netezza_synapse_measure_level_mismatched_cnt_diff.append('')
                                    netezza_synapse_partition_level_mismatched_cnt_diff.append('')
                                    netezza_synapse_partition_level_missing_cnt.append('')
                                    netezza_synapse_table_level_mismatched_cnt_diff.append(str(table_level_mismatched_cnt_diff))
                                    netezza_synapse_table_level_missing_cnt.append(str(table_level_missing_cnt))
                                    netezza_synapse_partition_level_diff_cnt.append('')
                                    netezza_synapse_table_level_diff_cnt.append('')
                                    netezza_synapse_detailed_report_path.append(workbook_path)

                                    print(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Synapse)")
                                    doc.add_paragraph(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Synapse)", style='List Bullet')
                                    
                                except Exception as e:
                                    print(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Synapse)")
                                    doc.add_paragraph(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Synapse)", style='List Bullet')
                                    print(e)
                                    doc.add_paragraph(f'{e}', style='Intense Quote')
                                        

                            except Exception as e:
                                print(f'Failed to generate detailed report for {db}.{table_name}, when comparing Netezza to Synapse table')
                                doc.add_paragraph(f'Failed to generate detailed report for {db}.{table_name}, when comparing Netezza to Synapse table', style='List Bullet')
                                print(e)
                                doc.add_paragraph(f'{e}', style='Intense Quote')

            except Exception as e:
                print(f'ERROR OCCURRED: Failed to generate detailed report for {db}.{table_name}, when comparing Netezza to Synapse table')
                doc.add_paragraph(f'ERROR OCCURRED: Failed to generate detailed report for {db}.{table_name}, when comparing Netezza to Synapse table', style='List Bullet')
                print(e)
                doc.add_paragraph(f'{e}', style='Intense Quote')
                
                          
    ## Upload log file
    save_log(doc)

except Exception as e:
    doc.add_paragraph("Data Validation Run Failed", style="List Bullet")
    doc.add_paragraph('')
    print(e)
    doc.add_paragraph(f'{e}', style='Intense Quote')

    ## Upload log file
    save_log(doc)

    ## Exit the notebook run
    dbutils.notebook.exit("Data Validation Run Failed") 

            
        


In [0]:
#Netezza Tables with  Synapse Comparsion.
from pyspark.sql.functions import *
from pyspark.sql.types import DecimalType, IntegerType, DateType, StructType, TimestampType
from datetime import date, timedelta, datetime
from delta.tables import *

# Header for log document
doc.add_heading('KEY TYPE UNIQUE = N', 3)
container = 'synapse'

try:

    for db in relevant_gold_db:

        if Env == 'QA':
            db_tmp = f"GDC_{db}"
        elif Env == 'PROD_DR':
            db_tmp = f"{db}_DR"
        else:
            db_tmp = db
        print('DATABASE: ', db)


        for table_dict in [v for d in validation_tables_info_2 for k,v in d.items() if k == db][0]:
            #print(table_dict)
            table_name=list(table_dict.keys())[0]
            print('TABLE:',table_name)
 
            
            try:
            
                Partition_column=spark.sql(f"select distinct PARTITION_COLUMN from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and PARTITION_COLUMN is not null")
                KEY_TYPE=spark.sql(f"select distinct KEY_TYPE_UNIQUE from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and KEY_TYPE_UNIQUE is not null")
                KEY_TYPES=KEY_TYPE.collect()[0][0]
                
                

                if KEY_TYPES == "N" :
                    tab_sy_cnt += 1
                    print(tab_sy_cnt)

                    doc.add_heading(f"{db}.{table_name}", 4)

                    ## Check if a partition column is configured for this table
                    print(f'Checking if a partition column is configured for {db}.{table_name}')
                    doc.add_paragraph(f'Checking if a partition column is configured for {db}.{table_name}', style='List Bullet')
                    if Partition_column.count() == 0:
                        Partition_col = None
                        print(f'No partition column configured for {db}.{table_name}')
                        doc.add_paragraph(f'No partition column configured for {db}.{table_name}', style='List Bullet')
                    else:
                        Partition_col =Partition_column.collect()[0][0]
                        print(f'Partition column configured for {db}.{table_name} is {Partition_col}')
                        doc.add_paragraph(f'Partition column configured for {db}.{table_name} is {Partition_col}', style='List Bullet')


                    if ((Partition_col != None) and (Partition_col != 'NA') ):
                        ## Retrieve partition, measure and field validation queries from metadata table
                        print(f'Retrieving Partition, Measure and Field Validation Queries from Fwk_Gold_Tables for {db}.{table_name}')
                        if (PL_TL == '7GS'):  
                            Query_Agg_Netezza=spark.sql(f"select distinct PARTITON_QUERY_NETEZZA_SYNAPSE_SEVEN_YEARS from gold_df \
                                                    where TABLE_NAME='{table_name}' and DATABASE ='{db}' and PARTITON_QUERY_NETEZZA_SYNAPSE_SEVEN_YEARS is not null")
                            Query_Agg_Synapse=spark.sql(f"select distinct PARTITION_QUERY_SYNAPSE from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}'\
                                                    and PARTITION_QUERY_SYNAPSE is not null") 
                            Query_Measure_Netezza=spark.sql(f"select distinct MEASURE_QUERY_NETEZZA_SEVEN_YEARS from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}'\
                                                            and MEASURE_QUERY_NETEZZA_SEVEN_YEARS is not null")
                            Query_Measure_Synapse=spark.sql(f"select distinct MEASURE_QUERY_SYNAPSE from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}'\
                                                    and MEASURE_QUERY_SYNAPSE is not null")
                        else :
                            #below is for NON transactional and transactional where we have full data in synapse as well
                            Query_Agg_Netezza=spark.sql(f"select distinct PARTITION_QUERY_NETEZZA from gold_df where TABLE_NAME = '{table_name}' and DATABASE ='{db}' and PARTITION_QUERY_NETEZZA is not null")
                            Query_Agg_Synapse=spark.sql(f"select distinct PARTITION_QUERY_SYNAPSE from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and PARTITION_QUERY_SYNAPSE is not null") 
                            Query_Measure_Netezza=spark.sql(f" select distinct MEASURE_QUERY_NETEZZA from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and MEASURE_QUERY_NETEZZA is not null")
                            Query_Measure_Synapse=spark.sql(f" select distinct MEASURE_QUERY_SYNAPSE from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and MEASURE_QUERY_SYNAPSE is not null")


                        Query_Field_Level_Netezza=spark.sql(f"select distinct FIELD_VALIDATION_QUERY_NETEZZA from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and FIELD_VALIDATION_QUERY_NETEZZA is not null")
                        Query_Field_Level_Synapse=spark.sql(f"select distinct FIELD_VALIDATION_QUERY_SYNAPSE from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and FIELD_VALIDATION_QUERY_SYNAPSE is not null")   

                        '''
                        ## Retrieve dataframe of the field used for validation (YEAR, MONTH, DAY)
                        PARTITION_ON=spark.sql(f"select PARTITION_ON from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and PARTITION_ON is not null")
                        Partition_Type=PARTITION_ON.collect()[0][0].upper()
                        print('PARTITON TYPE:', Partition_Type)
                        '''

                        ## Skip table if no queries are configured
                        if (Query_Field_Level_Netezza.count() == 0) & (Query_Measure_Netezza.count() == 0) & (Query_Agg_Netezza.count() == 0):
                            print(f"No field validation query or measure query configured. No detailed report generated for {db}.{table_name}")
                            doc.add_paragraph(f"No field validation query or measure query configured. No detailed report generated for {db}.{table_name}", style = 'List Bullet')
                            continue
                    
                        ## PERFORM PARTITION COUNT VALIDATION
                        
                        if (Query_Agg_Netezza.count() == 0) | (Query_Agg_Synapse.count() == 0):
                            print(f'No partition queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or Synapse. No partition count validation performed.')
                            doc.add_paragraph(f'No partition queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or Synapse. No partition count validation performed.', style='List Bullet')
                            partition_count_validation_status = 'False'
                        else:
                            query_PARTITION_CNT_NZ = Query_Agg_Netezza.collect()[0][0]
                            query_PARTITION_CNT_SY = Query_Agg_Synapse.collect()[0][0]
                            if (query_PARTITION_CNT_NZ == '') | (query_PARTITION_CNT_SY == ''):
                                partition_count_validation_status = 'False'
                                print('No Partition Query is configured. No partition count validation performed')
                                doc.add_paragraph(('No Partition Query is configured. No partition count validation performed'), style= 'List Bullet')
                            else:
                                print(f'Attempting to perform partition count validation for {db}.{table_name}')
                                doc.add_paragraph(f'Attempting to perform partition count validation for {db}.{table_name}', style='List Bullet')

                                try:
                                    query_PARTITION_CNT_NZ = query_PARTITION_CNT_NZ.replace('db_tmp', db_tmp)
                                    netezza_partition_cnt_df = readfromNetizza(Env, query_PARTITION_CNT_NZ, db)
                                    synapse_partition_cnt_df = readfromSynapse(Env,query_PARTITION_CNT_SY)

                                    try:
                                        partition_count_mismatched_cnt, partition_count_mismatched_df, partition_count_mismatched_detail_cnt, partition_count_mismatched_detail_df = find_Mismatched(netezza_partition_cnt_df,synapse_partition_cnt_df,Partition_col)
                                        print(f"Successfully performed partition count validation for {db}.{table_name}")
                                        doc.add_paragraph(f"Successfully performed partition count validation for {db}.{table_name}", style = 'List Bullet')
                                        partition_count_validation_status = 'True'
                                    except Exception as e:
                                        partition_count_validation_status = 'False'
                                        print(f'Failed to perform partition count validation for {db}.{table_name}')
                                        doc.add_paragraph(f'Failed to perform partition count validation for {db}.{table_name}', style='List Bullet')
                                        print(e)
                                        doc.add_paragraph(f'{e}', style='Intense Quote')
                                except Exception as e:
                                    partition_count_validation_status = 'False'
                                    print(f'Bad partition query configured for {db}.{table_name}. Please correct the query. No partition count validation performed')
                                    doc.add_paragraph(f'Bad partition query configured for {db}.{table_name}. Please correct the query. No partition count validation performed', style = 'List Bullet')
                                    print(e)
                                    doc.add_paragraph(f'{e}', style='Intense Quote')

                        
                        

                        '''
                        if (PL_TL == '7GS'): 
                            query_PARTITION_CNT_NZ = Query_Agg_Netezza.collect()[0][0]
                            if Partition_Type == 'YEAR' :
                                query_PARTITION_CNT_SY= f"""select FORMAT({Partition_col}, 'yyyy') as {Partition_col}, count(*) as CNT from {db}.{table_name} group by FORMAT({Partition_col}, 'yyyy')"""
                            elif Partition_Type == 'MONTH' :
                                query_PARTITION_CNT_SY=f"""select FORMAT({Partition_col}, 'yyyy MMM') as {Partition_col}, count(*) as CNT from {db}.{table_name} group by FORMAT({Partition_col}, 'yyyy MMM')"""
                            elif Partition_Type == 'DAY' :
                                query_PARTITION_CNT_SY=f"""select FORMAT({Partition_col}, 'yyyy MMM dd') as {Partition_col}, count(*) as CNT from {db}.{table_name} group by FORMAT({Partition_col}, 'yyyy MMM dd')"""
                        else:
                            if Partition_Type == 'YEAR' :
                                query_PARTITION_CNT_NZ = f"""select to_char({Partition_col}, 'YYYY') as {Partition_col}, count(*) as CNT from {db_tmp}.{table_name} group by to_char({Partition_col}, 'YYYY')"""
                                query_PARTITION_CNT_SY= f"""select FORMAT({Partition_col}, 'yyyy') as {Partition_col}, count(*) as CNT from {db}.{table_name} group by FORMAT({Partition_col}, 'yyyy')"""
                            elif Partition_Type == 'MONTH' :
                                query_PARTITION_CNT_NZ = f"""select to_char({Partition_col}, 'YYYY Mon') as {Partition_col}, count(*) as CNT from {db_tmp}.{table_name} group by to_char({Partition_col}, 'YYYY Mon')"""
                                query_PARTITION_CNT_SY=f"""select FORMAT({Partition_col}, 'yyyy MMM') as {Partition_col}, count(*) as CNT from {db}.{table_name} group by FORMAT({Partition_col}, 'yyyy MMM')"""
                            elif Partition_Type == 'DAY' :
                                query_PARTITION_CNT_NZ = f"""select to_char({Partition_col}, 'YYYY Mon dd') as {Partition_col}, count(*) as CNT from {db_tmp}.{table_name} group by to_char({Partition_col}, 'YYYY Mon dd')"""
                                query_PARTITION_CNT_SY=f"""select FORMAT({Partition_col}, 'yyyy MMM dd') as {Partition_col}, count(*) as CNT from {db}.{table_name} group by FORMAT({Partition_col}, 'yyyy MMM dd')"""

                        netezza_partition_cnt_df = readfromNetizza(Env, query_PARTITION_CNT_NZ, db)
                        synapse_partition_cnt_df = readfromSynapse(Env,query_PARTITION_CNT_SY)

                        try:
                            partition_count_mismatched_cnt, partition_count_mismatched_df, partition_count_mismatched_detail_cnt, partition_count_mismatched_detail_df = find_Mismatched(netezza_partition_cnt_df,synapse_partition_cnt_df,Partition_col)
                            print(f"Successfully performed partition count validation for {db}.{table_name}")
                            doc.add_paragraph(f"Successfully performed partition count validation for {db}.{table_name}", style = 'List Bullet')
                            partition_count_validation_status = 'True'
                        except Exception as e:
                            partition_count_validation_status = 'False'
                            print(f'Failed to perform partition count validation for {db}.{table_name}')
                            doc.add_paragraph(f'Failed to perform partition count validation for {db}.{table_name}', style='List Bullet')
                            print(e)
                            doc.add_paragraph(f'{e}', style='Intense Quote')
                        '''

                        
                        ## PERFORM MEASURE LEVEL VALIDATION
                        if (Query_Measure_Netezza.count() == 0) | (Query_Measure_Synapse.count() == 0):
                            print(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or Synapse. No measure level validation performed.')
                            doc.add_paragraph(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or Synapse. No measure level validation performed.', style='List Bullet')
                            measure_validation_status = 'False'
                        else:
                            query_measure_netezza =Query_Measure_Netezza.collect()[0][0]
                            query_measure_synapse = Query_Measure_Synapse.collect()[0][0]
                            if (query_measure_netezza == '') | (query_measure_synapse == ''):
                                print(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or Synapse. No measure level validation performed.')
                                doc.add_paragraph(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or Synapse. No measure level validation performed.', style='List Bullet')
                                measure_validation_status = 'False' 
                            else:
                                print(f'Attempting to perform measure level validation for {db}.{table_name}')
                                doc.add_paragraph(f'Attempting to perform measure level validation for {db}.{table_name}', style='List Bullet')

                                try:
                                    query_measure_netezza = query_measure_netezza.replace('db_tmp', db_tmp)
                                    netezza_df_measure=readfromNetizza(Env,query_measure_netezza,db)
                                    synapse_df_measure=readfromSynapse(Env,query_measure_synapse)

                                    try:
                                        measure_level_mismatched_cnt_diff, measure_level_mismatched_record_df, measure_level_mismatched_detail_cnt, measure_level_mismatched_detail_df = find_Mismatched(netezza_df_measure,synapse_df_measure,Partition_col)
                                        print(f"Successfully performed measure level validation for {db}.{table_name}")
                                        doc.add_paragraph(f"Successfully performed measure level validation for {db}.{table_name}", style='List Bullet')
                                        measure_validation_status = 'True'
                                    except Exception as e:
                                        measure_validation_status = 'False'
                                        print(f'Failed to perform measure level validation for {db}.{table_name}')
                                        doc.add_paragraph(f'Failed to perform measure level validation for {db}.{table_name}', style='List Bullet')
                                        print(e)
                                        doc.add_paragraph(f'{e}', style='Intense Quote')
                                except Exception as e:
                                    measure_validation_status = 'False'
                                    print(f'Bad measure query configured for {db}.{table_name}. Please correct the query. No measure level validation performed')
                                    doc.add_paragraph(f'Bad measure query configured for {db}.{table_name}. Please correct the query. No measure level validation performed', style = 'List Bullet')
                                    print(e)
                                    doc.add_paragraph(f'{e}', style='Intense Quote')
                        

                        

                        ## PERFORM PARTITION LEVEL VALIDATION

                        if (Query_Field_Level_Netezza.count()==0) | (Query_Field_Level_Synapse.count()==0):
                            print(f'No field validation queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or Synapse. No partition level validation performed.')
                            doc.add_paragraph(f'No field validation queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or Synapse. No partition level validation performed.', style='List Bullet')
                            partition_validation_status = 'False'
                        else:
                            query_PARTITION_NZ = Query_Field_Level_Netezza.collect()[0][0]
                            query_PARTITION_SY = Query_Field_Level_Synapse.collect()[0][0]
                            if (query_PARTITION_NZ == '') | (query_PARTITION_SY == ''):
                                print(f'No field validation queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or Synapse. No partition level validation performed.')
                                doc.add_paragraph(f'No field validation queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or Synapse. No partition level validation performed.', style='List Bullet')
                                partition_validation_status = 'False'  
                            else:
                                try:
                                    query_PARTITION_NZ = query_PARTITION_NZ.replace('db_tmp', db_tmp)
                                    netezza_df_partition=readfromNetizza(Env,query_PARTITION_NZ,db)
                                    synapse_df_partition=readfromSynapse(Env,query_PARTITION_SY)  

                                    ## Validate that the partition size is within the max row permitted limit
                                    print('Attempting to perform partition level validation. Need to confirm that the partition size is within acceptable limit')
                                    doc.add_paragraph('Attempting to perform partition level validation. Need to confirm that the partition size is within acceptable limit', style = 'List Bullet')
                                    
                                    cnt_netezza_df_partition = netezza_df_partition.limit(max_rows_permitted + 1).count()
                                    cnt_synapse_df_partition = synapse_df_partition.limit(max_rows_permitted + 1).count()

                                    if (cnt_netezza_df_partition == 0) | (cnt_synapse_df_partition == 0):
                                        print(f'Partition for table {db}.{table_name} is empty either in Netezza or in Synapse. No partition validation performed')
                                        doc.add_paragraph(f'Partition for table {db}.{table_name} is empty either in Netezza or in Synapse. No partition validation performed', style = 'List Bullet')
                                        partition_validation_status = 'False'
                                    elif (cnt_netezza_df_partition > max_rows_permitted) | (cnt_synapse_df_partition > max_rows_permitted):
                                        print(f'Partition for {db}.{table_name} is greater than {max_rows_permitted}. No partition validation performed. Please reconfigure the partition for {db}.{table_name}')
                                        doc.add_paragraph(f'Partition for {db}.{table_name} is greater than {max_rows_permitted}. No partition validation performed. Please reconfigure the partition for {db}.{table_name}', style = 'List Bullet')
                                        partition_validation_status = 'False'
                                    else:
                                        ## Perform validation on the partition
                                        print(f'Partition for {db}.{table_name} is less than or equal to {max_rows_permitted}.')
                                        doc.add_paragraph(f'Partition for {db}.{table_name} is less than or equal to {max_rows_permitted}.', style = 'List Bullet')
                                        try:
                                            #partition_level_diff = get_pandas(netezza_df_partition.subtract(synapse_df_partition))
                                            partition_level_diff = get_pandas(trim_space(netezza_df_partition).subtract(trim_space(synapse_df_partition)))
                                            partition_level_diff_cnt = len(partition_level_diff)
                                            print(f"Successfully performed partition level validation for {db}.{table_name}")
                                            doc.add_paragraph(f"Successfully performed partition level validation for {db}.{table_name}", style='List Bullet')
                                            partition_validation_status = 'True'
                                        except Exception as e:
                                            partition_validation_status = 'False'
                                            print(f'Failed to perform partiton level validation for {db}.{table_name}_(Netezza v. Synapse)')
                                            doc.add_paragraph(f'Failed to perform partiton level validation for {db}.{table_name}_(Netezza v. Synapse)', style='List Bullet')
                                            print(e)
                                            doc.add_paragraph(f'{e}', style='Intense Quote')
                                except Exception as e:
                                    partition_validation_status = 'False'
                                    print(f'Bad field validation query configured for {db}.{table_name}. Please correct the query. No partition level validation performed')
                                    doc.add_paragraph(f'Bad field validation query configured for {db}.{table_name}. Please correct the query. No partition level validation performed', style = 'List Bullet')
                                    print(e)
                                    doc.add_paragraph(f'{e}', style='Intense Quote')

                        
                        ## Define excel writer
                        try:
                            with pd.ExcelWriter(f"{db}-{table_name}_(Netezza v. Synapse).xlsx", engine="openpyxl", mode = 'w') as writer:
                                
                                ## Write missing and mismatched information to sheets in excel workbook
                                if partition_validation_status == 'True':
                                    partition_level_diff.head(max_excel_row).to_excel(writer, sheet_name = f"difference_record_partition", startrow = 0, startcol=0, index = False)
                                if measure_validation_status == 'True':
                                    try:
                                        measure_level_mismatched_record_df.sort_values(by=Partition_col).head(max_excel_row).to_excel(writer, sheet_name = "measure_mismatched_record", startrow = 0, startcol=0, index = False)
                                        measure_level_mismatched_detail_df.sort_values(by=Partition_col).head(max_excel_row).to_excel(writer, sheet_name = "measure_mismatched_detail", startrow = 0, startcol=0, index = False)
                                    except:
                                        measure_level_mismatched_record_df.head(max_excel_row).to_excel(writer, sheet_name = "measure_mismatched_record", startrow = 0, startcol=0, index = False)
                                        measure_level_mismatched_detail_df.head(max_excel_row).to_excel(writer, sheet_name = "measure_mismatched_detail", startrow = 0, startcol=0, index = False)
                                if partition_count_validation_status == 'True':
                                    try:
                                        partition_count_mismatched_df.sort_values(by=Partition_col).head(max_excel_row).to_excel(writer, sheet_name = "partition_count_mismatched_record", startrow = 0, startcol=0, index = False)
                                        partition_count_mismatched_detail_df.sort_values(by=Partition_col).head(max_excel_row).to_excel(writer, sheet_name = "partition_count_mismatched_detail", startrow = 0, startcol=0, index = False)
                                    except:
                                        partition_count_mismatched_df.head(max_excel_row).to_excel(writer, sheet_name = "partition_count_mismatched_record", startrow = 0, startcol=0, index = False)
                                        partition_count_mismatched_detail_df.head(max_excel_row).to_excel(writer, sheet_name = "partition_count_mismatched_detail", startrow = 0, startcol=0, index = False)
                            print('Detailed report generated')
                            doc.add_paragraph('Detailed report generated', style = 'List Bullet')
                            detailed_report_status = 'True'
                        except Exception as e:
                            detailed_report_status = 'False'
                            print('No detailed report generated')
                            doc.add_paragraph('No detailed report generated', style = 'List Bullet')
                            print(e)
                            doc.add_paragraph(f'{e}', style= 'Intense Quote')
                            
                                

                        ## Upload workbook to storage
                        if detailed_report_status == 'True':
                            try:        
                                workbook_path = path + f"{db}-{table_name}_(Netezza v. Synapse).xlsx" 
                                shutil.copy2(f"{db}-{table_name}_(Netezza v. Synapse).xlsx",workbook_path)
                                os.remove(f"{db}-{table_name}_(Netezza v. Synapse).xlsx")

                                ## Append detail summary to lists
                                netezza_synapse_database_name.append(db)
                                netezza_synapse_table_name.append(table_name)
                                if partition_count_validation_status == 'True':
                                    netezza_synapse_partition_count_mismatched_cnt.append(str(partition_count_mismatched_cnt))
                                else:
                                    netezza_synapse_partition_count_mismatched_cnt.append('')
                                if measure_validation_status == 'True':
                                    netezza_synapse_measure_level_mismatched_cnt_diff.append(str(measure_level_mismatched_cnt_diff))
                                else:
                                    netezza_synapse_measure_level_mismatched_cnt_diff.append('')
                                netezza_synapse_partition_level_mismatched_cnt_diff.append('')
                                netezza_synapse_partition_level_missing_cnt.append('')
                                netezza_synapse_table_level_mismatched_cnt_diff.append('')
                                netezza_synapse_table_level_missing_cnt.append('')
                                if partition_validation_status == 'True':
                                    netezza_synapse_partition_level_diff_cnt.append(str(partition_level_diff_cnt))
                                else:
                                    netezza_synapse_partition_level_diff_cnt.append('')
                                netezza_synapse_table_level_diff_cnt.append('')
                                netezza_synapse_detailed_report_path.append(workbook_path)

                                print(f"Successfully uploaded detailed report for {db}-{table_name}_(Netezza v. Synapse)")
                                doc.add_paragraph(f"Successfully uploaded detailed report for {db}-{table_name}_(Netezza v. Synapse)", style='List Bullet')
                            

                            except Exception as e:
                                print(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Synapse)")
                                doc.add_paragraph(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Synapse)", style='List Bullet')
                                print(e)
                                doc.add_paragraph(f'{e}', style='Intense Quote')

                            
                            

            
                    else :
                        ## This condition is to perform detailed validation for small and medium size tables
                        print(f'Attempting to perform detailed validation on {db}.{table_name}_(Netezza v. Synapse). Need to confirm it is a small to medium size table')
                        if (PL_TL == '7GS'):
                            
                            ## Retrieve measure level query
                            Query_Measure_Netezza=spark.sql(f"select distinct MEASURE_QUERY_NETEZZA_SEVEN_YEARS from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}'\
                                                            and MEASURE_QUERY_NETEZZA_SEVEN_YEARS is not null")
                            Query_Measure_Synapse=spark.sql(f"select distinct MEASURE_QUERY_SYNAPSE from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}'\
                                                    and MEASURE_QUERY_SYNAPSE is not null")

                            ## Retrieve the retention column
                            filterColumn=gold_df.select('Filter7').where((gold_df.TABLE_NAME==table_name)&(gold_df.DATABASE==db)).distinct().rdd.map(lambda x: x.Filter7).collect()
                            filterColumn=str(filterColumn)
                            filterColumn=filterColumn.replace("[","").replace("]","").replace("'","")
                            ## BELOW is fetch the minimum date from synapse to avoid mismatch on the day where data is not loaded in netizza
                            print(filterColumn)

                            ## Retrieve the retention value
                            query_forValuefilter= f"""select CAST(min({filterColumn}) as varchar(10)) {filterColumn} from {db}.{table_name}""" 
                    
                            if filterColumn!='None':
                                filterValue1=readfromSynapse(Env,query_forValuefilter)
                                filterValue=filterValue1.rdd.map(lambda x: x[0]).collect()
                                filterValue=str(filterValue)
                                filterValue=filterValue.replace("[","").replace("]","")
                            else:
                                filterValue='None'

                            ## Query netezza table based on retention values. Get netezza table for validation
                            if filterValue!='None':
                                Query_Netezza = f"""select * from {db_tmp}.{table_name} where {filterColumn} >= {filterValue}""" 
                            else:
                                Query_Netezza = f"""select * from {db_tmp}.{table_name}"""
                            #below is filter for 7 years for netizza 
                            print(Query_Netezza)
                            netezza_df=readfromNetizza(Env,Query_Netezza,db)

                            ## Get synapse table for validation
                            Query_Synapse = f"""select * from {db}.{table_name}"""
                            df_synapse=readfromSynapse(Env,Query_Synapse)

                        else:
                            ## Retrieve measure level query
                            Query_Measure_Netezza=spark.sql(f" select distinct MEASURE_QUERY_NETEZZA from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and MEASURE_QUERY_NETEZZA is not null")
                            Query_Measure_Synapse=spark.sql(f" select distinct MEASURE_QUERY_SYNAPSE from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and MEASURE_QUERY_SYNAPSE is not null")

                            print("else Full load Non Transactional ")
                            ## Get netezza table for validation
                            Query_Netezza = f"""select * from {db_tmp}.{table_name}"""
                            netezza_df=readfromNetizza(Env,Query_Netezza,db)

                            ## Get synapse table for validation
                            Query_Synapse = f"""select * from {db}.{table_name}"""
                            #print(Query_Synapse)
                            df_synapse=readfromSynapse(Env,Query_Synapse)


                        ## Confirm table is small or medium size (i.e. Not greater than 5 million rows)
                        #max_rows_permitted = 5000000

                        try:
                            ## Retrieve row count
                            cnt_netezza_df = int(report_dataframe_1.select(f'NETEZZA_TABLE_ROW_COUNT ({netezza_sub})').filter((col('NETEZZA_SCHEMA') == f'{db}') & (col('NETEZZA_TABLE') == f'{table_name}')).collect()[0][0])   #netezza_df.limit(max_rows_permitted + 1).count()
                            cnt_df_synapse = int(report_dataframe_1.select('SYNAPSE_TABLE_ROW_COUNT').filter((col('SYNAPSE_SCHEMA') == f'{db}') & (col('SYNAPSE_TABLE') == f'{table_name}')).collect()[0][0])  #df_synapse.limit(max_rows_permitted + 1).count()
                        except:
                            print(f'{db}.{table_name} not available in summary report')
                            cnt_netezza_df = netezza_df.limit(max_rows_permitted + 1).count()
                            cnt_df_synapse = df_synapse.limit(max_rows_permitted + 1).count()


                        if (cnt_netezza_df == 0) | (cnt_df_synapse == 0):
                            print(f'Table {db}.{table_name} is empty either in Netezza or in Synapse. No detailed validation performed')
                            doc.add_paragraph(f'Table {db}.{table_name} is empty either in Netezza or in Synapse. No detailed validation performed', style = 'List Bullet')

                        elif (cnt_netezza_df > max_rows_permitted) | (cnt_df_synapse > max_rows_permitted):
                            print(f'Table {db}.{table_name}_(Netezza v. Synapse) is a big table, with no partition column configures. No table level validation performed')
                            doc.add_paragraph(f'Table {db}.{table_name}_(Netezza v. Synapse) is a big table, with no partition column configures. No table level validation performed', style='List Bullet')

                            print('Will attempt to perform measure level validation')
                            doc.add_paragraph('Attempting measure level validation for a big table with no partition configured', style= 'List Bullet')

                            ## PERFORM MEASURE LEVEL VALIDATION
                            if (Query_Measure_Netezza.count() == 0) | (Query_Measure_Synapse.count() == 0):
                                print(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or Synapse. No measure level validation performed.')
                                doc.add_paragraph(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or Synapse. No measure level validation performed.', style='List Bullet')
                                measure_validation_status = 'False'
                            else:
                                query_measure_netezza =Query_Measure_Netezza.collect()[0][0]
                                query_measure_synapse = Query_Measure_Synapse.collect()[0][0]
                                if (query_measure_netezza == '') | (query_measure_synapse == ''):
                                    print(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or Synapse. No measure level validation performed.')
                                    doc.add_paragraph(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or Synapse. No measure level validation performed.', style='List Bullet')
                                    measure_validation_status = 'False' 
                                else:
                                    print(f'Measure queries configured. Attempting to perform measure level validation for {db}.{table_name}')
                                    doc.add_paragraph(f'Measure queries configured. Attempting to perform measure level validation for {db}.{table_name}', style='List Bullet')

                                    try:
                                        query_measure_netezza = query_measure_netezza.replace('db_tmp', db_tmp)
                                        netezza_df_measure=readfromNetizza(Env,query_measure_netezza,db)
                                        synapse_df_measure=readfromSynapse(Env,query_measure_synapse)

                                        try:
                                            measure_level_mismatched_record_df = get_pandas(synapse_df_measure.subtract(netezza_df_measure))
                                            measure_level_mismatched_cnt_diff = len(measure_level_mismatched_record_df)
                                            print(f"Successfully performed measure mismatched validation on {db}.{table_name}. This is a big table does not have a partition column configured")
                                            doc.add_paragraph(f"Successfully performed measure mismatched validation on {db}.{table_name}. This is a big table does not have a partition column configured", style='List Bullet')
                                            measure_validation_status = 'True'

                                            ## Define excel writer
                                            with pd.ExcelWriter(f"{db}-{table_name}_(Netezza v. Synapse).xlsx", engine="openpyxl", mode = 'w') as writer:

                                                ## Write missing and mismatched information to sheets in excel workbook
                                                measure_level_mismatched_record_df.head(max_excel_row).to_excel(writer, sheet_name = f"measure_mismatched_record", startrow = 0, startcol=0, index = False)

                                            ## Upload workbook to storage
                                            try:        
                                                workbook_path = path + f"{db}-{table_name}_(Netezza v. Synapse).xlsx" 
                                                shutil.copy2(f"{db}-{table_name}_(Netezza v. Synapse).xlsx",workbook_path)
                                                os.remove(f"{db}-{table_name}_(Netezza v. Synapse).xlsx")

                                                ## Append detail summary to lists
                                                netezza_synapse_database_name.append(db)
                                                netezza_synapse_table_name.append(table_name)
                                                netezza_synapse_partition_count_mismatched_cnt.append('')
                                                netezza_synapse_measure_level_mismatched_cnt_diff.append(str(measure_level_mismatched_cnt_diff))
                                                netezza_synapse_partition_level_mismatched_cnt_diff.append('')
                                                netezza_synapse_partition_level_missing_cnt.append('')
                                                netezza_synapse_table_level_mismatched_cnt_diff.append('')
                                                netezza_synapse_table_level_missing_cnt.append('')
                                                netezza_synapse_partition_level_diff_cnt.append('')
                                                netezza_synapse_table_level_diff_cnt.append('')
                                                netezza_synapse_detailed_report_path.append(workbook_path)

                                                print(f"Successfully uploaded detailed report for {db}-{table_name}_(Netezza v. Synapse)")
                                                doc.add_paragraph(f"Successfully uploaded detailed report for {db}-{table_name}_(Netezza v. Synapse)", style='List Bullet')

                                            except Exception as e:
                                                print(f"Failed to upload detailed report for {db}-{table_name}_(Netezza v. Synapse)")
                                                doc.add_paragraph(f"Failed to upload detailed report for {db}-{table_name}_(Netezza v. Synapse)", style='List Bullet')
                                                print(e)
                                                doc.add_paragraph(f'{e}', style='Intense Quote')
                                        except Exception as e:
                                            print(f"Failed to perform detailed validation on {db}.{table_name}")
                                            doc.add_paragraph(f"Failed to perform detailed validation on {db}.{table_name}", style='List Bullet')
                                            print(e)
                                            doc.add_paragraph(f'{e}', style='Intense Quote')
                                            continue                                   
                                    except Exception as e:
                                        print(f'Bad measure query configured for {db}.{table_name}. Please correct the query. No measure level validation performed')
                                        doc.add_paragraph(f'Bad measure query configured for {db}.{table_name}. Please correct the query. No measure level validation performed', style = 'List Bullet')
                                        print(e)
                                        doc.add_paragraph(f'{e}', style='Intense Quote')



                        else:
                            print(f'Table size is less than or equal to {max_rows_permitted}. Performing detailed validation on full table {db}.{table_name}')
                            doc.add_paragraph(f'Table size is less than or equal to {max_rows_permitted}. Performing detailed validation on full table {db}.{table_name}', style = 'List Bullet')
                            
                            try:
                                #table_level_diff = get_pandas(netezza_df.subtract(df_synapse))
                                table_level_diff = get_pandas(trim_space(netezza_df).subtract(trim_space(df_synapse)))
                                table_level_diff_cnt = len(table_level_diff)
                                print(f'Successfully performed full table validation on {db}.{table_name}, comparing Netezza to Synapse table')
                                doc.add_paragraph(f'Successfully performed full table validation on {db}.{table_name}, comparing Netezza to Synapse table', style='List Bullet')

                                ## Define excel writer
                                with pd.ExcelWriter(f"{db}-{table_name}_(Netezza v. Synapse).xlsx", engine="openpyxl", mode = 'w') as writer:

                                    ## Write missing and mismatched information to sheets in excel workbook
                                    table_level_diff.head(max_excel_row).to_excel(writer, sheet_name = "difference_record", startrow = 0, startcol=0, index = False)
                                    

                                ## Upload workbook to storage
                                try:        
                                    workbook_path = path + f"{db}-{table_name}_(Netezza v. Synapse).xlsx" 
                                    shutil.copy2(f"{db}-{table_name}_(Netezza v. Synapse).xlsx",workbook_path)
                                    os.remove(f"{db}-{table_name}_(Netezza v. Synapse).xlsx")

                                    ## Append detail summary to lists
                                    netezza_synapse_database_name.append(db)
                                    netezza_synapse_table_name.append(table_name)
                                    netezza_synapse_partition_count_mismatched_cnt.append('')
                                    netezza_synapse_measure_level_mismatched_cnt_diff.append('')
                                    netezza_synapse_partition_level_mismatched_cnt_diff.append('')
                                    netezza_synapse_partition_level_missing_cnt.append('')
                                    netezza_synapse_table_level_mismatched_cnt_diff.append('')
                                    netezza_synapse_table_level_missing_cnt.append('')
                                    netezza_synapse_partition_level_diff_cnt.append('')
                                    netezza_synapse_table_level_diff_cnt.append(table_level_diff_cnt)
                                    netezza_synapse_detailed_report_path.append(workbook_path)   

                                    print(f"Successfully uploaded detailed report for {db}-{table_name}_(Netezza v. Synapse)")
                                    doc.add_paragraph(f"Successfully uploaded detailed report for {db}-{table_name}_(Netezza v. Synapse)", style='List Bullet')
                                    

                                except Exception as e:
                                    print(f"Failed to upload detailed report for {db}-{table_name}_(Netezza v. Synapse)")
                                    doc.add_paragraph(f"Failed to upload detailed report for {db}-{table_name}_(Netezza v. Synapse)", style='List Bullet')
                                    print(e)
                                    doc.add_paragraph(f'{e}', style='Intense Quote')

                            except Exception as e:
                                print(f'Failed to generate detailed report for {db}.{table_name}, when comparing Netezza to Synapse table')
                                doc.add_paragraph(f'Failed to generate detailed report for {db}.{table_name}, when comparing Netezza to Synapse table', style='List Bullet')
                                print(e)
                                doc.add_paragraph(f'{e}', style='Intense Quote')

            except Exception as e:
                print(f'ERROR OCCURRED: Failed to generate detailed report for {db}.{table_name}, when comparing Netezza to Synapse table')
                doc.add_paragraph(f'ERROR OCCURRED: Failed to generate detailed report for {db}.{table_name}, when comparing Netezza to Synapse table', style='List Bullet')
                print(e)
                doc.add_paragraph(f'{e}', style='Intense Quote')
                #continue                                                       

    doc.add_paragraph()

    ## Upload log file
    save_log(doc)

except Exception as e:
    doc.add_paragraph("Data Validation Run Failed", style="Intense Quote")
    doc.add_paragraph('')
    print(e)
    doc.add_paragraph(f'{e}', style='Intense Quote')

    ## Upload log file
    save_log(doc)

    ## Exit the notebook run
    dbutils.notebook.exit("Data Validation Run Failed") 
              

            
        


In [0]:
netezza_gold_database_name = []
netezza_gold_table_name = []
netezza_gold_partition_count_mismatched_cnt = []
netezza_gold_measure_level_mismatched_cnt_diff = []
netezza_gold_partition_level_mismatched_cnt_diff = []
netezza_gold_partition_level_missing_cnt = []
netezza_gold_table_level_mismatched_cnt_diff = []
netezza_gold_table_level_missing_cnt = []
netezza_gold_measure_level_diff_cnt = []
netezza_gold_partition_level_diff_cnt = []
netezza_gold_table_level_diff_cnt = []
netezza_gold_detailed_report_path = []

In [0]:
tab_gd_cnt = 0

In [0]:
if (PL_TL != '7GS') & (Load_Type == 'Historical'): 
    
    from pyspark.sql.functions import *
    from pyspark.sql.types import DecimalType, IntegerType, DateType, StructType, TimestampType
    from datetime import date, timedelta, datetime
    from delta.tables import *

    # Header for log document
    doc.add_heading('Log From Detailed Report Generation Comparing Netezza v. Gold', 2)
    doc.add_heading('KEY TYPE UNIQUE = Y',3)
    container = 'gold'

    try:

        for db in relevant_gold_db:

            if Env == 'QA':
                db_tmp = f"GDC_{db}"
            elif Env == 'PROD_DR':
                db_tmp = f"{db}_DR"
            else:
                db_tmp = db
            print('DATABASE: ', db)


            for table_dict in [v for d in validation_tables_info_2 for k,v in d.items() if k == db][0]:
                print(table_dict)
                table_name=list(table_dict.keys())[0]
                print('TABLE NAME: ',table_name)   

                try:      
                    partition_key=list(table_dict.values())[0][0]
                    print(type(partition_key))
                    primary_key=list(table_dict.values())[0]
                    print('PRIMARY KEY: ',primary_key)
                    Partition_column=spark.sql(f"select distinct PARTITION_COLUMN from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and PARTITION_COLUMN is not null")
                    KEY_TYPE=spark.sql(f"select distinct KEY_TYPE_UNIQUE from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and KEY_TYPE_UNIQUE is not null")
                    KEY_TYPES=KEY_TYPE.collect()[0][0]
                    print('KEY_TYPES: ',KEY_TYPES)

                

                    if KEY_TYPES == "Y" :
                        tab_gd_cnt += 1
                        print(tab_gd_cnt)

                        doc.add_heading(f"{db}.{table_name}", 4)
                        
                        if Partition_column.count() == 0:
                            Partition_col = None
                            print(f'No partition column configured for {db}.{table_name}')
                            doc.add_paragraph(f'No partition column configured for {db}.{table_name}', style='List Bullet')
                        else:
                            Partition_col =Partition_column.collect()[0][0]
                            print(f'Partition column configured for {db}.{table_name} is {Partition_col}')
                            doc.add_paragraph(f'Partition column configured for {db}.{table_name} is {Partition_col}', style='List Bullet')


                        ## The ADLS gold table
                        Data_Lake_df=spark.read.format("delta").option("header","true").load("/mnt/gold/"+db+"/ADMIN/"+table_name)
                        Data_Lake_df.createOrReplaceTempView("Data_Lake_df")

                        if ((Partition_col != None) and (Partition_col != 'NA') ):
                            ## Retrieve dataframe of the queries for row count per partition for Netezza tables, and Retrieve dataframe of query for generating a measure from Netezza and ADLS_Gold
                            print(f'Retrieving Partition, Measure and Field Validation Queries from Fwk_Gold_Tables for {db}.{table_name}')
                            doc.add_paragraph(f'Retrieving Partition, Measure and Field Validation Queries from Fwk_Gold_Tables for {db}.{table_name}', style='List Bullet')
                            
                            Query_Agg_Netezza=spark.sql(f"select distinct PARTITION_QUERY_NETEZZA from gold_df where TABLE_NAME = '{table_name}' and DATABASE ='{db}' and PARTITION_QUERY_NETEZZA is not null")
                            Query_Measure_Netezza=spark.sql(f" select distinct MEASURE_QUERY_NETEZZA from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and MEASURE_QUERY_NETEZZA is not null")
                            Query_Measure_Gold=spark.sql(f" select distinct ADLS_QUERY_FOR_MEASURE from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and ADLS_QUERY_FOR_MEASURE is not null")

                            Query_Field_Level_Netezza=spark.sql(f"select distinct FIELD_VALIDATION_QUERY_NETEZZA from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and FIELD_VALIDATION_QUERY_NETEZZA is not null")
                            Query_Field_Level_Gold=spark.sql(f"select distinct ADLS_QUERY_FOR_FIELD from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and ADLS_QUERY_FOR_FIELD is not null")  


                            
                            ## Retrieve dataframe of the field used for validation (YEAR, MONTH, DAY)
                            PARTITION_ON=spark.sql(f"select PARTITION_ON from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and PARTITION_ON is not null")
                            Partition_Type=PARTITION_ON.collect()[0][0].upper()
                            print('PARTITION TYPE:', Partition_Type)
                            


                            ## Skip table if no queries are configured
                            if (Query_Field_Level_Netezza.count() == 0) & (Query_Measure_Netezza.count() == 0) & (Query_Agg_Netezza.count() == 0):
                                print(f"No field validation query or measure query or partition query configured. No detailed report generated for {db}.{table_name}")
                                doc.add_paragraph(f"No field validation query or measure query or partition query configured. No detailed report generated for {db}.{table_name}", style = 'List Bullet')
                                continue
                            

                            ## PERFORM PARTITION COUNT VALIDATION

                            if (Query_Agg_Netezza.count() == 0):
                                print(f'No partition queries configured for {db}.{table_name} in Fwk_Gold_Tables for Netezza. No partition count validation performed.')
                                doc.add_paragraph(f'No partition queries configured for {db}.{table_name} in Fwk_Gold_Tables for Netezza. No partition count validation performed.', style='List Bullet')
                                partition_count_validation_status = 'False'
                            else:
                                query_PARTITION_CNT_NZ = Query_Agg_Netezza.collect()[0][0]
                                if (query_PARTITION_CNT_NZ == ''):
                                    partition_count_validation_status = 'False'
                                    print('No Partition Query is configured. No partition count validation performed')
                                    doc.add_paragraph(('No Partition Query is configured. No partition count validation performed'), style= 'List Bullet')
                                else:
                                    print(f'Attempting to perform partition count validation for {db}.{table_name}')
                                    doc.add_paragraph(f'Attempting to perform partition count validation for {db}.{table_name}', style='List Bullet')

                                    try:
                                        query_PARTITION_CNT_NZ = query_PARTITION_CNT_NZ.replace('db_tmp', db_tmp)
                                        netezza_partition_cnt_df = readfromNetizza(Env, query_PARTITION_CNT_NZ, db)
                                        if Partition_Type == 'YEAR' :
                                            Data_Lake_df_PARTITION_CNT=spark.sql(f"select DATE_FORMAT({Partition_col}, 'yyyy') as {Partition_col}, count(*) as CNT from Data_Lake_df group by DATE_FORMAT({Partition_col}, 'yyyy')")
                                        elif Partition_Type == 'MONTH' :
                                            Data_Lake_df_PARTITION_CNT=spark.sql(f"select DATE_FORMAT({Partition_col}, 'yyyy MMM') as {Partition_col}, count(*) as CNT from Data_Lake_df group by DATE_FORMAT({Partition_col}, 'yyyy MMM')")
                                        elif Partition_Type == 'DAY' :
                                            Data_Lake_df_PARTITION_CNT=spark.sql(f"select DATE_FORMAT({Partition_col}, 'yyyy MMM dd') as {Partition_col}, count(*) as CNT from Data_Lake_df group by DATE_FORMAT({Partition_col}, 'yyyy MMM dd')")

                                        try:
                                            partition_count_mismatched_cnt, partition_count_mismatched_df, partition_count_mismatched_detail_cnt, partition_count_mismatched_detail_df = find_Mismatched(netezza_partition_cnt_df,Data_Lake_df_PARTITION_CNT,Partition_col)
                                            print(f"Successfully performed partition count validation for {db}.{table_name}")
                                            doc.add_paragraph(f"Successfully performed partition count validation for {db}.{table_name}", style = 'List Bullet')
                                            partition_count_validation_status = 'True'
                                        except Exception as e:
                                            partition_count_validation_status = 'False'
                                            print(f'Failed to perform partition count validation for {db}.{table_name}')
                                            doc.add_paragraph(f'Failed to perform partition count validation for {db}.{table_name}', style='List Bullet')
                                            print(e)
                                            doc.add_paragraph(f'{e}', style='Intense Quote')
                                    except Exception as e:
                                        partition_count_validation_status = 'False'
                                        print(f'Bad partition query configured for {db}.{table_name}. Please correct the query. No partition count validation performed')
                                        doc.add_paragraph(f'Bad partition query configured for {db}.{table_name}. Please correct the query. No partition count validation performed', style = 'List Bullet')
                                        print(e)
                                        doc.add_paragraph(f'{e}', style='Intense Quote')



                            ## PERFORM MEASURE LEVEL VALIDATION

                            if (Query_Measure_Netezza.count() == 0) | (Query_Measure_Gold.count() == 0):
                                print(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.')
                                doc.add_paragraph(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.', style='List Bullet')
                                measure_validation_status = 'False'
                            else:
                                query_measure_netezza =Query_Measure_Netezza.collect()[0][0]
                                query_measure_gold = Query_Measure_Gold.collect()[0][0]
                                if (query_measure_netezza == '') | (query_measure_gold == ''):
                                    print(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.')
                                    doc.add_paragraph(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.', style='List Bullet')
                                    measure_validation_status = 'False' 
                                else:
                                    print(f'Attempting to perform measure level validation for {db}.{table_name}')
                                    doc.add_paragraph(f'Attempting to perform measure level validation for {db}.{table_name}', style='List Bullet')

                                    try:
                                        query_measure_netezza = query_measure_netezza.replace('db_tmp', db_tmp)
                                        netezza_df_measure=readfromNetizza(Env,query_measure_netezza,db)
                                        adls_gold_df_measure = spark.sql(f"{query_measure_gold}")

                                        try:
                                            measure_level_mismatched_cnt_diff, measure_level_mismatched_record_df, measure_level_mismatched_detail_cnt, measure_level_mismatched_detail_df = find_Mismatched(netezza_df_measure,adls_gold_df_measure,Partition_col)
                                            print(f"Successfully performed measure level validation for {db}.{table_name}")
                                            doc.add_paragraph(f"Successfully performed measure level validation for {db}.{table_name}", style='List Bullet')
                                            measure_validation_status = 'True'
                                        except Exception as e:
                                            measure_validation_status = 'False'
                                            print(f'Failed to perform measure level validation for {db}.{table_name}')
                                            doc.add_paragraph(f'Failed to perform measure level validation for {db}.{table_name}', style='List Bullet')
                                            print(e)
                                            doc.add_paragraph(f'{e}', style='Intense Quote')
                                    except Exception as e:
                                        measure_validation_status = 'False'
                                        print(f'Bad measure query configured for {db}.{table_name}. Please correct the query. No measure level validation performed')
                                        doc.add_paragraph(f'Bad measure query configured for {db}.{table_name}. Please correct the query. No measure level validation performed', style = 'List Bullet')
                                        print(e)
                                        doc.add_paragraph(f'{e}', style='Intense Quote')



                            ## PERFORM PARTITION LEVEL VALIDATION

                            if (Query_Field_Level_Netezza.count()==0) | (Query_Field_Level_Gold.count()==0):
                                print(f'No field validation queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No partition level validation performed.')
                                doc.add_paragraph(f'No field validation queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No partition level validation performed.', style='List Bullet')
                                partition_validation_status = 'False'
                            else:
                                query_PARTITION_NZ = Query_Field_Level_Netezza.collect()[0][0]
                                query_PARTITION_Gold = Query_Field_Level_Gold.collect()[0][0]
                                if (query_PARTITION_NZ == '') | (query_PARTITION_Gold == ''):
                                    print(f'No field validation queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No partition level validation performed.')
                                    doc.add_paragraph(f'No field validation queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No partition level validation performed.', style='List Bullet')
                                    partition_validation_status = 'False'  
                                else:
                                    try:
                                        query_PARTITION_NZ = query_PARTITION_NZ.replace('db_tmp', db_tmp)
                                        netezza_df_partition=readfromNetizza(Env,query_PARTITION_NZ,db)
                                        Data_Lake_df_1=spark.sql(query_PARTITION_Gold)  

                                        ## Validate that the partition size is within the max row permitted limit
                                        print('Attempting to perform partition level validation. Need to confirm that the partition size is within acceptable limit')
                                        doc.add_paragraph('Attempting to perform partition level validation. Need to confirm that the partition size is within acceptable limit', style = 'List Bullet')
                                        
                                        cnt_netezza_df_partition = netezza_df_partition.limit(max_rows_permitted + 1).count()
                                        cnt_Data_Lake_df_1 = Data_Lake_df_1.limit(max_rows_permitted + 1).count()

                                        if (cnt_netezza_df_partition == 0) | (cnt_Data_Lake_df_1 == 0):
                                            print(f'Partition for table {db}.{table_name} is empty either in Netezza or in Gold. No partition validation performed')
                                            doc.add_paragraph(f'Partition for table {db}.{table_name} is empty either in Netezza or in Gold. No partition validation performed', style = 'List Bullet')
                                            partition_validation_status = 'False'

                                        elif (cnt_netezza_df_partition > max_rows_permitted) | (cnt_Data_Lake_df_1 > max_rows_permitted):
                                            print(f'Partition for {db}.{table_name} is greater than {max_rows_permitted}. No partition validation performed. Please reconfigure the partition for {db}.{table_name}')
                                            doc.add_paragraph(f'Partition for {db}.{table_name} is greater than {max_rows_permitted}. No partition validation performed. Please reconfigure the partition for {db}.{table_name}', style = 'List Bullet')
                                            partition_validation_status = 'False'
                                        else:
                                            ## Perform validation on the partition
                                            print(f'Partition for {db}.{table_name} is less than or equal to {max_rows_permitted}.')
                                            doc.add_paragraph(f'Partition for {db}.{table_name} is less than or equal to {max_rows_permitted}.', style = 'List Bullet')
                                            try:
                                                #partition_level_mismatched_cnt_diff, partition_level_mismatched_record_df, partition_level_mismatched_record_detail_cnt, partition_level_mismatched_record_detail_df, partition_level_missing_cnt, partition_level_missing_output_df = find_difference(netezza_df_partition, Data_Lake_df_1, primary_key)
                                                partition_level_mismatched_cnt_diff, partition_level_mismatched_record_df, partition_level_mismatched_record_detail_cnt, partition_level_mismatched_record_detail_df, partition_level_missing_cnt, partition_level_missing_output_df = find_difference(trim_space(netezza_df_partition), trim_space(Data_Lake_df_1), primary_key)
                                                print(f"Successfully performed partition level validation for {db}.{table_name}")
                                                doc.add_paragraph(f"Successfully performed partition level validation for {db}.{table_name}", style='List Bullet')
                                                partition_validation_status = 'True'
                                            except Exception as e:
                                                partition_validation_status = 'False'
                                                print(f'Failed to perform partiton level validation for {db}.{table_name}_(Netezza v. Gold)')
                                                doc.add_paragraph(f'Failed to perform partiton level validation for {db}.{table_name}_(Netezza v. Gold)', style='List Bullet')
                                                print(e)
                                                doc.add_paragraph(f'{e}', style='Intense Quote')
                                    except Exception as e:
                                        partition_validation_status = 'False'
                                        print(f'Bad field validation query configured for {db}.{table_name}. Please correct the query. No partition level validation performed')
                                        doc.add_paragraph(f'Bad field validation query configured for {db}.{table_name}. Please correct the query. No partition level validation performed', style = 'List Bullet')
                                        print(e)
                                        doc.add_paragraph(f'{e}', style='Intense Quote')

                                                    
                            
                            ## Define excel writer
                            try:
                                with pd.ExcelWriter(f"{db}-{table_name}_(Netezza v. Gold).xlsx", engine="openpyxl", mode = 'w') as writer:

                                    ## Write missing and mismatched information to sheets in excel workbook
                                    if partition_validation_status == 'True':
                                        try:
                                            partition_level_missing_output_df.sort_values(by=primary_key).head(max_excel_row).to_excel(writer, sheet_name = f"missing_record_partition", startrow = 0, startcol=0, index = False)
                                        except:
                                            partition_level_missing_output_df.head(max_excel_row).to_excel(writer, sheet_name = f"missing_record_partition", startrow = 0, startcol=0, index = False)
                                        try:
                                            partition_level_mismatched_record_df.sort_values(by=primary_key).head(max_excel_row).to_excel(writer, sheet_name = f"mismatched_record_partition", startrow = 0, startcol=0, index = False)
                                            partition_level_mismatched_record_detail_df.sort_values(by=primary_key).head(max_excel_row).to_excel(writer, sheet_name = f"mismatched_detail_partition", startrow = 0, startcol=0, index = False)
                                        except:
                                            partition_level_mismatched_record_df.head(max_excel_row).to_excel(writer, sheet_name = f"mismatched_record_partition", startrow = 0, startcol=0, index = False)
                                            partition_level_mismatched_record_detail_df.head(max_excel_row).to_excel(writer, sheet_name = f"mismatched_detail_partition", startrow = 0, startcol=0, index = False)
                                    if measure_validation_status == 'True':
                                        try:
                                            measure_level_mismatched_record_df.sort_values(by=Partition_col).head(max_excel_row).to_excel(writer, sheet_name = "measure_mismatched_record", startrow = 0, startcol=0, index = False)
                                            measure_level_mismatched_detail_df.sort_values(by=Partition_col).head(max_excel_row).to_excel(writer, sheet_name = "measure_mismatched_detail", startrow = 0, startcol=0, index = False)
                                        except:
                                            measure_level_mismatched_record_df.head(max_excel_row).to_excel(writer, sheet_name = "measure_mismatched_record", startrow = 0, startcol=0, index = False)
                                            measure_level_mismatched_detail_df.head(max_excel_row).to_excel(writer, sheet_name = "measure_mismatched_detail", startrow = 0, startcol=0, index = False)
                                    if partition_count_validation_status == 'True':
                                        try:
                                            partition_count_mismatched_df.sort_values(by=Partition_col).head(max_excel_row).to_excel(writer, sheet_name = "partition_count_mismatched_record", startrow = 0, startcol=0, index = False)
                                            partition_count_mismatched_detail_df.sort_values(by=Partition_col).head(max_excel_row).to_excel(writer, sheet_name = "partition_count_mismatched_detail", startrow = 0, startcol=0, index = False)
                                        except:
                                            partition_count_mismatched_df.head(max_excel_row).to_excel(writer, sheet_name = "partition_count_mismatched_record", startrow = 0, startcol=0, index = False)
                                            partition_count_mismatched_detail_df.head(max_excel_row).to_excel(writer, sheet_name = "partition_count_mismatched_detail", startrow = 0, startcol=0, index = False)
                                print('Detailed report generated')
                                doc.add_paragraph('Detailed report generated', style = 'List Bullet')
                                detailed_report_status = 'True'
                            except Exception as e:
                                detailed_report_status = 'False'
                                print('No detailed report generated')
                                doc.add_paragraph('No detailed report generated', style = 'List Bullet')
                                print(e)
                                doc.add_paragraph(f'{e}', style= 'Intense Quote')
                                    

                            ## Upload workbook to storage
                            if detailed_report_status == 'True':
                                try:        
                                    workbook_path = path + f"{db}-{table_name}_(Netezza v. Gold).xlsx" 
                                    shutil.copy2(f"{db}-{table_name}_(Netezza v. Gold).xlsx",workbook_path)
                                    os.remove(f"{db}-{table_name}_(Netezza v. Gold).xlsx")

                                    ## Append detail summary to lists
                                    netezza_gold_database_name.append(db)
                                    netezza_gold_table_name.append(table_name)
                                    if partition_count_validation_status == 'True':
                                        netezza_gold_partition_count_mismatched_cnt.append(str(partition_count_mismatched_cnt))
                                    else:
                                        netezza_gold_partition_count_mismatched_cnt.append('')
                                    if measure_validation_status == 'True':
                                        netezza_gold_measure_level_mismatched_cnt_diff.append(str(measure_level_mismatched_cnt_diff))
                                    else:
                                        netezza_gold_measure_level_mismatched_cnt_diff.append('')
                                    if partition_validation_status == 'True':
                                        netezza_gold_partition_level_mismatched_cnt_diff.append(str(partition_level_mismatched_cnt_diff))
                                        netezza_gold_partition_level_missing_cnt.append(str(partition_level_missing_cnt))
                                    else:
                                        netezza_gold_partition_level_mismatched_cnt_diff.append('')
                                        netezza_gold_partition_level_missing_cnt.append('')
                                    netezza_gold_table_level_mismatched_cnt_diff.append('')
                                    netezza_gold_table_level_missing_cnt.append('')
                                    netezza_gold_partition_level_diff_cnt.append('')
                                    netezza_gold_table_level_diff_cnt.append('')
                                    netezza_gold_detailed_report_path.append(workbook_path)

                                    print(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Gold)")
                                    doc.add_paragraph(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Gold)", style='List Bullet')
                                    

                                except Exception as e:
                                    print(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Gold)")
                                    doc.add_paragraph(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Gold)", style='List Bullet')
                                    print(e)
                                    doc.add_paragraph(f'{e}', style='Intense Quote')
                            


                        else :
                            ## This condition is to perform detailed validation for small and medium size tables
                            print(f'Attempting to perform detailed validation on {db}.{table_name}. Need to confirm it is a small to medium size table')
                            

                            ## Retrieve measure level query
                            Query_Measure_Netezza=spark.sql(f" select distinct MEASURE_QUERY_NETEZZA from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and MEASURE_QUERY_NETEZZA is not null")
                            Query_Measure_Gold=spark.sql(f" select distinct ADLS_QUERY_FOR_MEASURE from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and ADLS_QUERY_FOR_MEASURE is not null")

                            print("else Full load Non Transactional ")
                            ## Get FULL netezza table for validation
                            Query_Netezza = f"""select * from {db_tmp}.{table_name}"""
                            netezza_df=readfromNetizza(Env,Query_Netezza,db)

                                

                            ## Confirm table is small or medium size (i.e. Not greater than 5 million rows)
                            

                            try:
                                ## Retrieve row count
                                cnt_netezza_df = int(report_dataframe_1.select(f'NETEZZA_TABLE_ROW_COUNT ({netezza_sub})').filter((col('NETEZZA_SCHEMA') == f'{db}') & (col('NETEZZA_TABLE') == f'{table_name}')).collect()[0][0]) #netezza_df.limit(max_rows_permitted + 1).count()
                                cnt_df_ADLS_gold = int(report_dataframe_1.select('GOLD_TABLE_ROW_COUNT').filter((col('SYNAPSE_SCHEMA') == f'{db}') & (col('SYNAPSE_TABLE') == f'{table_name}')).collect()[0][0])#df_synapse.limit(max_rows_permitted + 1).count()
                            except:
                                print(f'{db}.{table_name} not available in summary report')
                                cnt_netezza_df = netezza_df.limit(max_rows_permitted + 1).count()
                                cnt_df_ADLS_gold = Data_Lake_df.limit(max_rows_permitted + 1).count()

                            

                            if (cnt_netezza_df == 0) | (cnt_df_ADLS_gold == 0):
                                print(f'Table {db}.{table_name} is empty either in Netezza or in Gold. No detailed validation performed')
                                doc.add_paragraph(f'Table {db}.{table_name} is empty either in Netezza or in Gold. No detailed validation performed', style = 'List Bullet')

                            elif (cnt_netezza_df > max_rows_permitted) | (cnt_df_ADLS_gold > max_rows_permitted):
                                print(f'Table {db}.{table_name} is a big table, with no partition column configures. No table level validation performed')
                                doc.add_paragraph(f'Table {db}.{table_name} is a big table, with no partition column configures. No table level validation performed', style='List Bullet')

                                print('Will attempt to perform measure level validation')
                                doc.add_paragraph('Attempting measure level validation for a big table with no partition configured', style= 'List Bullet')

                                ## PERFORM MEASURE LEVEL VALIDATION
                                if (Query_Measure_Netezza.count() == 0) | (Query_Measure_Gold.count() == 0):
                                    print(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.')
                                    doc.add_paragraph(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.', style='List Bullet')
                                    measure_validation_status = 'False'
                                else:
                                    query_measure_netezza =Query_Measure_Netezza.collect()[0][0]
                                    query_measure_gold = Query_Measure_Gold.collect()[0][0]
                                    if (query_measure_netezza == '') | (query_measure_gold == ''):
                                        print(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.')
                                        doc.add_paragraph(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.', style='List Bullet')
                                        measure_validation_status = 'False' 
                                    else:
                                        print(f'Measure queries configured. Attempting to perform measure level validation for {db}.{table_name}')
                                        doc.add_paragraph(f'Measure queries configured. Attempting to perform measure level validation for {db}.{table_name}', style='List Bullet')

                                        try:
                                            query_measure_netezza = query_measure_netezza.replace('db_tmp', db_tmp)
                                            netezza_df_measure=readfromNetizza(Env,query_measure_netezza,db)
                                            adls_gold_df_measure = spark.sql(f"{query_measure_gold}")

                                            try:
                                                measure_level_mismatched_record_df = get_pandas(adls_gold_df_measure.subtract(netezza_df_measure))
                                                measure_level_mismatched_cnt_diff = len(measure_level_mismatched_record_df)
                                                print(f"Successfully performed measure mismatched validation on {db}.{table_name}. This is a big table does not have a partition column configured")
                                                doc.add_paragraph(f"Successfully performed measure mismatched validation on {db}.{table_name}. This is a big table does not have a partition column configured", style='List Bullet')
                                                measure_validation_status = 'True'


                                                ## Define excel writer
                                                with pd.ExcelWriter(f"{db}-{table_name}_(Netezza v. Gold).xlsx", engine="openpyxl", mode = 'w') as writer:

                                                    ## Write missing and mismatched information to sheets in excel workbook
                                                    measure_level_mismatched_record_df.head(max_excel_row).to_excel(writer, sheet_name = f"measure_mismatched_record", startrow = 0, startcol=0, index = False)

                                                ## Upload workbook to storage
                                                try:        
                                                    workbook_path = path + f"{db}-{table_name}_(Netezza v. Gold).xlsx" 
                                                    shutil.copy2(f"{db}-{table_name}_(Netezza v. Gold).xlsx",workbook_path)
                                                    os.remove(f"{db}-{table_name}_(Netezza v. Gold).xlsx")

                                                    ## Append detail summary to lists
                                                    netezza_gold_database_name.append(db)
                                                    netezza_gold_table_name.append(table_name)
                                                    netezza_gold_partition_count_mismatched_cnt.append('')
                                                    netezza_gold_measure_level_mismatched_cnt_diff.append(str(measure_level_mismatched_cnt_diff))
                                                    netezza_gold_partition_level_mismatched_cnt_diff.append('')
                                                    netezza_gold_partition_level_missing_cnt.append('')
                                                    netezza_gold_table_level_mismatched_cnt_diff.append('')
                                                    netezza_gold_table_level_missing_cnt.append('')
                                                    netezza_gold_partition_level_diff_cnt.append('')
                                                    netezza_gold_table_level_diff_cnt.append('')
                                                    netezza_gold_detailed_report_path.append(workbook_path)

                                                    print(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Gold)")
                                                    doc.add_paragraph(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Gold)", style='List Bullet')

                                                except Exception as e:
                                                        print(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Gold)")
                                                        doc.add_paragraph(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Gold)", style='List Bullet')
                                                        print(e)
                                                        doc.add_paragraph(f'{e}', style='Intense Quote')
                                                
                                            except Exception as e:
                                                print(f"Failed to perform detailed validation on {db}.{table_name}")
                                                doc.add_paragraph(f"Failed to perform detailed validation on {db}.{table_name}", style='List Bullet')
                                                print(e)
                                                doc.add_paragraph(f'{e}', style='Intense Quote')
                                                continue

                                        except Exception as e:
                                            print(f'Bad measure query configured for {db}.{table_name}. Please correct the query. No measure level validation performed')
                                            doc.add_paragraph(f'Bad measure query configured for {db}.{table_name}. Please correct the query. No measure level validation performed', style = 'List Bullet')
                                            print(e)
                                            doc.add_paragraph(f'{e}', style='Intense Quote')
                                        

                            else:
                                print(f'Table size is less than or equal to {max_rows_permitted}. Performing detailed validation on full table {db}.{table_name}')
                                doc.add_paragraph(f'Table size is less than or equal to {max_rows_permitted}. Performing detailed validation on full table {db}.{table_name}', style = 'List Bullet')
                                
                                try:
                                    #table_level_mismatched_cnt_diff, table_level_mismatched_record_df, table_level_mismatched_record_detail_cnt, table_level_mismatched_record_detail_df, table_level_missing_cnt, table_level_missing_output_df = find_difference(netezza_df, Data_Lake_df, primary_key)
                                    table_level_mismatched_cnt_diff, table_level_mismatched_record_df, table_level_mismatched_record_detail_cnt, table_level_mismatched_record_detail_df, table_level_missing_cnt, table_level_missing_output_df = find_difference(trim_space(netezza_df), trim_space(Data_Lake_df), primary_key)
                                    print(f'Successfully performed full table validation on {db}.{table_name}, comparing Netezza to Gold table')
                                    doc.add_paragraph(f'Successfully performed full table validation on {db}.{table_name}, comparing Netezza to Gold table', style='List Bullet')

                                    ## Define excel writer
                                    with pd.ExcelWriter(f"{db}-{table_name}_(Netezza v. Gold).xlsx", engine="openpyxl", mode = 'w') as writer:

                                        ## Write missing and mismatched information to sheets in excel workbook
                                        try:
                                            table_level_missing_output_df.sort_values(by=primary_key).head(max_excel_row).to_excel(writer, sheet_name = "missing_record", startrow = 0, startcol=0, index = False)
                                        except:
                                            table_level_missing_output_df.head(max_excel_row).to_excel(writer, sheet_name = "missing_record", startrow = 0, startcol=0, index = False)
                                        try:
                                            table_level_mismatched_record_df.sort_values(by=primary_key).head(max_excel_row).to_excel(writer, sheet_name = "mismatched_record", startrow = 0, startcol=0, index = False)
                                            table_level_mismatched_record_detail_df.sort_values(by=primary_key).head(max_excel_row).to_excel(writer, sheet_name = "mismatched_detail", startrow = 0, startcol=0, index = False)
                                        except:
                                            table_level_mismatched_record_df.head(max_excel_row).to_excel(writer, sheet_name = "mismatched_record", startrow = 0, startcol=0, index = False)
                                            table_level_mismatched_record_detail_df.head(max_excel_row).to_excel(writer, sheet_name = "mismatched_detail", startrow = 0, startcol=0, index = False)
                                        

                                    ## Upload workbook to storage
                                    try:        
                                        workbook_path = path + f"{db}-{table_name}_(Netezza v. Gold).xlsx" 
                                        shutil.copy2(f"{db}-{table_name}_(Netezza v. Gold).xlsx",workbook_path)
                                        os.remove(f"{db}-{table_name}_(Netezza v. Gold).xlsx")

                                        ## Append detail summary to lists
                                        netezza_gold_database_name.append(db)
                                        netezza_gold_table_name.append(table_name)
                                        netezza_gold_partition_count_mismatched_cnt.append('')
                                        netezza_gold_measure_level_mismatched_cnt_diff.append('')
                                        netezza_gold_partition_level_mismatched_cnt_diff.append('')
                                        netezza_gold_partition_level_missing_cnt.append('')
                                        netezza_gold_table_level_mismatched_cnt_diff.append(str(table_level_mismatched_cnt_diff))
                                        netezza_gold_table_level_missing_cnt.append(str(table_level_missing_cnt))
                                        netezza_gold_partition_level_diff_cnt.append('')
                                        netezza_gold_table_level_diff_cnt.append('')
                                        netezza_gold_detailed_report_path.append(workbook_path)

                                        print(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Gold)")
                                        doc.add_paragraph(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Gold)", style='List Bullet')
                                        

                                    except Exception as e:
                                        print(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Gold)")
                                        doc.add_paragraph(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Gold)", style='List Bullet')
                                        print(e)
                                        doc.add_paragraph(f'{e}', style='Intense Quote')


                                except Exception as e:
                                    print(f'Failed to generate detailed report for {db}.{table_name}, when comparing Netezza to Gold table')
                                    doc.add_paragraph(f'Failed to generate detailed report for {db}.{table_name}, when comparing Netezza to Gold table', style='List Bullet')
                                    print(e)
                                    doc.add_paragraph(f'{e}', style='Intense Quote')
                                    
    
                except Exception as e:
                    print(f'ERROR OCCURRED: Failed to generate detailed report for {db}.{table_name}, when comparing Netezza to Gold table')
                    doc.add_paragraph(f'ERROR OCCURRED: Failed to generate detailed report for {db}.{table_name}, when comparing Netezza to Gold table', style='List Bullet')
                    print(e)
                    doc.add_paragraph(f'{e}', style='Intense Quote')
                    #continue


        ## Upload log file
        save_log(doc)

    except Exception as e:
        doc.add_paragraph("Data Validation Run Failed", style="List Bullet")
        doc.add_paragraph('')
        print(e)
        doc.add_paragraph(f'{e}', style='Intense Quote')

        ## Upload log file
        save_log(doc)

        ## Exit the notebook run
        dbutils.notebook.exit("Data Validation Run Failed") 




    
            

In [0]:
if (PL_TL != '7GS') & (Load_Type == 'Historical'): 
    
    from pyspark.sql.functions import *
    from pyspark.sql.types import DecimalType, IntegerType, DateType, StructType, TimestampType
    from datetime import date, timedelta, datetime
    from delta.tables import *

    # Header for log document
    doc.add_heading('KEY TYPE UNIQUE = N',3)
    container = 'gold'

    try:

        for db in relevant_gold_db:

            if Env == 'QA':
                db_tmp = f"GDC_{db}"
            elif Env == 'PROD_DR':
                db_tmp = f"{db}_DR"
            else:
                db_tmp = db
            print('DATABASE: ', db)


            for table_dict in [v for d in validation_tables_info_2 for k,v in d.items() if k == db][0]:
                print(table_dict)
                table_name=list(table_dict.keys())[0]
                print('TABLE NAME: ',table_name)    

                try:    
                
                    Partition_column=spark.sql(f"select distinct PARTITION_COLUMN from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and PARTITION_COLUMN is not null")
                    KEY_TYPE=spark.sql(f"select distinct KEY_TYPE_UNIQUE from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and KEY_TYPE_UNIQUE is not null")
                    KEY_TYPES=KEY_TYPE.collect()[0][0]
                    print('KEY_TYPES: ',KEY_TYPES)

                        

                    if KEY_TYPES == "N" :
                        tab_gd_cnt += 1
                        print(tab_gd_cnt)

                        doc.add_heading(f"{db}.{table_name}", 4)
                        
                        if Partition_column.count() == 0:
                            Partition_col = None
                            print(f'No partition column configured for {db}.{table_name}')
                            doc.add_paragraph(f'No partition column configured for {db}.{table_name}', style='List Bullet')
                        else:
                            Partition_col =Partition_column.collect()[0][0]
                            print(f'Partition column configured for {db}.{table_name} is {Partition_col}')
                            doc.add_paragraph(f'Partition column configured for {db}.{table_name} is {Partition_col}', style='List Bullet')

                        ## The ADLS gold table
                        Data_Lake_df=spark.read.format("delta").option("header","true").load("/mnt/gold/"+db+"/ADMIN/"+table_name)
                        Data_Lake_df.createOrReplaceTempView("Data_Lake_df")

                        if ((Partition_col != None) and (Partition_col != 'NA') ):
                            ## Retrieve dataframe of the queries for row count per partition for Netezza tables, and Retrieve dataframe of query for generating a measure from Netezza and ADLS_Gold
                            print(f'Retrieving Partition, Measure and Field Validation Queries from Fwk_Gold_Tables for {db}.{table_name}')
                            doc.add_paragraph(f'Retrieving Partition, Measure and Field Validation Queries from Fwk_Gold_Tables for {db}.{table_name}', style='List Bullet')
                            
                            Query_Agg_Netezza=spark.sql(f"select distinct PARTITION_QUERY_NETEZZA from gold_df where TABLE_NAME = '{table_name}' and DATABASE ='{db}' and PARTITION_QUERY_NETEZZA is not null")
                            Query_Measure_Netezza=spark.sql(f" select distinct MEASURE_QUERY_NETEZZA from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and MEASURE_QUERY_NETEZZA is not null")
                            Query_Measure_Gold=spark.sql(f" select distinct ADLS_QUERY_FOR_MEASURE from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and ADLS_QUERY_FOR_MEASURE is not null")

                            Query_Field_Level_Netezza=spark.sql(f"select distinct FIELD_VALIDATION_QUERY_NETEZZA from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and FIELD_VALIDATION_QUERY_NETEZZA is not null")
                            Query_Field_Level_Gold=spark.sql(f"select distinct ADLS_QUERY_FOR_FIELD from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and ADLS_QUERY_FOR_FIELD is not null")  


                            ## Retrieve dataframe of the field used for validation (YEAR, MONTH, DAY)
                            PARTITION_ON=spark.sql(f"select PARTITION_ON from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and PARTITION_ON is not null")
                            Partition_Type=PARTITION_ON.collect()[0][0].upper()
                            print('PARTITION TYPE:', Partition_Type)

                            ## Skip table if no queries are configured
                            if (Query_Field_Level_Netezza.count() == 0) & (Query_Measure_Netezza.count() == 0) & (Query_Agg_Netezza.count() == 0):
                                print(f"No field validation query or measure query or partition query configured. No detailed report generated for {db}.{table_name}")
                                doc.add_paragraph(f"No field validation query or measure query or partition query configured. No detailed report generated for {db}.{table_name}", style = 'List Bullet')
                                continue
                            

                            ## Get the various queries, partition field and partition value
                            '''
                            if Query_Agg_Netezza.count()!=0:
                                query_table_load_netezza =Query_Agg_Netezza.collect()[0][0]
                                netezza_df=readfromNetizza(Env,query_table_load_netezza,db)
                            '''

                            ## PERFORM PARTITION COUNT VALIDATION
                            if (Query_Agg_Netezza.count() == 0):
                                print(f'No partition queries configured for {db}.{table_name} in Fwk_Gold_Tables for Netezza. No partition count validation performed.')
                                doc.add_paragraph(f'No partition queries configured for {db}.{table_name} in Fwk_Gold_Tables for Netezza. No partition count validation performed.', style='List Bullet')
                                partition_count_validation_status = 'False'
                            else:
                                query_PARTITION_CNT_NZ = Query_Agg_Netezza.collect()[0][0]
                                if (query_PARTITION_CNT_NZ == ''):
                                    partition_count_validation_status = 'False'
                                    print('No Partition Query is configured. No partition count validation performed')
                                    doc.add_paragraph(('No Partition Query is configured. No partition count validation performed'), style= 'List Bullet')
                                else:
                                    print(f'Attempting to perform partition count validation for {db}.{table_name}')
                                    doc.add_paragraph(f'Attempting to perform partition count validation for {db}.{table_name}', style='List Bullet')

                                    try:
                                        query_PARTITION_CNT_NZ = query_PARTITION_CNT_NZ.replace('db_tmp', db_tmp)
                                        netezza_partition_cnt_df = readfromNetizza(Env, query_PARTITION_CNT_NZ, db)
                                        if Partition_Type == 'YEAR' :
                                            Data_Lake_df_PARTITION_CNT=spark.sql(f"select DATE_FORMAT({Partition_col}, 'yyyy') as {Partition_col}, count(*) as CNT from Data_Lake_df group by DATE_FORMAT({Partition_col}, 'yyyy')")
                                        elif Partition_Type == 'MONTH' :
                                            Data_Lake_df_PARTITION_CNT=spark.sql(f"select DATE_FORMAT({Partition_col}, 'yyyy MMM') as {Partition_col}, count(*) as CNT from Data_Lake_df group by DATE_FORMAT({Partition_col}, 'yyyy MMM')")
                                        elif Partition_Type == 'DAY' :
                                            Data_Lake_df_PARTITION_CNT=spark.sql(f"select DATE_FORMAT({Partition_col}, 'yyyy MMM dd') as {Partition_col}, count(*) as CNT from Data_Lake_df group by DATE_FORMAT({Partition_col}, 'yyyy MMM dd')")

                                        try:
                                            partition_count_mismatched_cnt, partition_count_mismatched_df, partition_count_mismatched_detail_cnt, partition_count_mismatched_detail_df = find_Mismatched(netezza_partition_cnt_df,Data_Lake_df_PARTITION_CNT,Partition_col)
                                            print(f"Successfully performed partition count validation for {db}.{table_name}")
                                            doc.add_paragraph(f"Successfully performed partition count validation for {db}.{table_name}", style = 'List Bullet')
                                            partition_count_validation_status = 'True'
                                        except Exception as e:
                                            partition_count_validation_status = 'False'
                                            print(f'Failed to perform partition count validation for {db}.{table_name}')
                                            doc.add_paragraph(f'Failed to perform partition count validation for {db}.{table_name}', style='List Bullet')
                                            print(e)
                                            doc.add_paragraph(f'{e}', style='Intense Quote')
                                    except Exception as e:
                                        partition_count_validation_status = 'False'
                                        print(f'Bad partition query configured for {db}.{table_name}. Please correct the query. No partition count validation performed')
                                        doc.add_paragraph(f'Bad partition query configured for {db}.{table_name}. Please correct the query. No partition count validation performed', style = 'List Bullet')
                                        print(e)
                                        doc.add_paragraph(f'{e}', style='Intense Quote')

                            
                            ## PERFORM MEASURE LEVEL VALIDATION

                            if (Query_Measure_Netezza.count() == 0) | (Query_Measure_Gold.count() == 0):
                                print(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.')
                                doc.add_paragraph(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.', style='List Bullet')
                                measure_validation_status = 'False'
                            else:
                                query_measure_netezza =Query_Measure_Netezza.collect()[0][0]
                                query_measure_gold = Query_Measure_Gold.collect()[0][0]
                                if (query_measure_netezza == '') | (query_measure_gold == ''):
                                    print(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.')
                                    doc.add_paragraph(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.', style='List Bullet')
                                    measure_validation_status = 'False' 
                                else:
                                    print(f'Attempting to perform measure level validation for {db}.{table_name}')
                                    doc.add_paragraph(f'Attempting to perform measure level validation for {db}.{table_name}', style='List Bullet')

                                    try:
                                        query_measure_netezza = query_measure_netezza.replace('db_tmp', db_tmp)
                                        netezza_df_measure=readfromNetizza(Env,query_measure_netezza,db)
                                        adls_gold_df_measure = spark.sql(f"{query_measure_gold}")

                                        try:
                                            measure_level_mismatched_cnt_diff, measure_level_mismatched_record_df, measure_level_mismatched_detail_cnt, measure_level_mismatched_detail_df = find_Mismatched(netezza_df_measure,adls_gold_df_measure,Partition_col)
                                            print(f"Successfully performed measure level validation for {db}.{table_name}")
                                            doc.add_paragraph(f"Successfully performed measure level validation for {db}.{table_name}", style='List Bullet')
                                            measure_validation_status = 'True'
                                        except Exception as e:
                                            measure_validation_status = 'False'
                                            print(f'Failed to perform measure level validation for {db}.{table_name}')
                                            doc.add_paragraph(f'Failed to perform measure level validation for {db}.{table_name}', style='List Bullet')
                                            print(e)
                                            doc.add_paragraph(f'{e}', style='Intense Quote')
                                    except Exception as e:
                                        measure_validation_status = 'False'
                                        print(f'Bad measure query configured for {db}.{table_name}. Please correct the query. No measure level validation performed')
                                        doc.add_paragraph(f'Bad measure query configured for {db}.{table_name}. Please correct the query. No measure level validation performed', style = 'List Bullet')
                                        print(e)
                                        doc.add_paragraph(f'{e}', style='Intense Quote')

                            

                            ## PERFORM PARTITION LEVEL VALIDATION

                            if (Query_Field_Level_Netezza.count()==0) | (Query_Field_Level_Gold.count()==0):
                                print(f'No field validation queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No partition level validation performed.')
                                doc.add_paragraph(f'No field validation queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No partition level validation performed.', style='List Bullet')
                                partition_validation_status = 'False'
                            else:
                                query_PARTITION_NZ = Query_Field_Level_Netezza.collect()[0][0]
                                query_PARTITION_Gold = Query_Field_Level_Gold.collect()[0][0]
                                if (query_PARTITION_NZ == '') | (query_PARTITION_Gold == ''):
                                    print(f'No field validation queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No partition level validation performed.')
                                    doc.add_paragraph(f'No field validation queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No partition level validation performed.', style='List Bullet')
                                    partition_validation_status = 'False'  
                                else:
                                    try:
                                        query_PARTITION_NZ = query_PARTITION_NZ.replace('db_tmp', db_tmp)
                                        netezza_df_partition=readfromNetizza(Env,query_PARTITION_NZ,db)
                                        Data_Lake_df_1=spark.sql(query_PARTITION_Gold)  

                                        ## Validate that the partition size is within the max row permitted limit
                                        print('Attempting to perform partition level validation. Need to confirm that the partition size is within acceptable limit')
                                        doc.add_paragraph('Attempting to perform partition level validation. Need to confirm that the partition size is within acceptable limit', style = 'List Bullet')
                                        
                                        cnt_netezza_df_partition = netezza_df_partition.limit(max_rows_permitted + 1).count()
                                        cnt_Data_Lake_df_1 = Data_Lake_df_1.limit(max_rows_permitted + 1).count()

                                        if (cnt_netezza_df_partition == 0) | (cnt_Data_Lake_df_1 == 0):
                                            print(f'Partition for table {db}.{table_name} is empty either in Netezza or in Gold. No partition validation performed')
                                            doc.add_paragraph(f'Partition for table {db}.{table_name} is empty either in Netezza or in Gold. No partition validation performed', style = 'List Bullet')
                                            partition_validation_status = 'False'

                                        elif (cnt_netezza_df_partition > max_rows_permitted) | (cnt_Data_Lake_df_1 > max_rows_permitted):
                                            print(f'Partition for {db}.{table_name} is greater than {max_rows_permitted}. No partition validation performed. Please reconfigure the partition for {db}.{table_name}')
                                            doc.add_paragraph(f'Partition for {db}.{table_name} is greater than {max_rows_permitted}. No partition validation performed. Please reconfigure the partition for {db}.{table_name}', style = 'List Bullet')
                                            partition_validation_status = 'False'
                                        else:
                                            ## Perform validation on the partition
                                            print(f'Partition for {db}.{table_name} is less than or equal to {max_rows_permitted}.')
                                            doc.add_paragraph(f'Partition for {db}.{table_name} is less than or equal to {max_rows_permitted}.', style = 'List Bullet')
                                            try:
                                                #partition_level_diff = get_pandas(netezza_df_partition.subtract(Data_Lake_df_1))
                                                partition_level_diff = get_pandas(trim_space(netezza_df_partition).subtract(trim_space(Data_Lake_df_1)))
                                                partition_level_diff_cnt = len(partition_level_diff)
                                                print(f"Successfully performed partition level validation for {db}.{table_name}")
                                                doc.add_paragraph(f"Successfully performed partition level validation for {db}.{table_name}", style= 'List Bullet')
                                                partition_validation_status = 'True'

                                            except Exception as e:
                                                partition_validation_status = 'False'
                                                print(f'Failed to perform partiton level validation for {db}.{table_name}_(Netezza v. Gold)')
                                                doc.add_paragraph(f'Failed to perform partiton level validation for {db}.{table_name}_(Netezza v. Gold)', style='List Bullet')
                                                print(e)
                                                doc.add_paragraph(f'{e}', style='Intense Quote')
                                    except Exception as e:
                                        partition_validation_status = 'False'
                                        print(f'Bad field validation query configured for {db}.{table_name}. Please correct the query. No partition level validation performed')
                                        doc.add_paragraph(f'Bad field validation query configured for {db}.{table_name}. Please correct the query. No partition level validation performed', style = 'List Bullet')
                                        print(e)
                                        doc.add_paragraph(f'{e}', style='Intense Quote')
        

                            ## Define excel writer
                            try:
                                with pd.ExcelWriter(f"{db}-{table_name}_(Netezza v. Gold).xlsx", engine="openpyxl", mode = 'w') as writer:

                                    ## Write missing and mismatched information to sheets in excel workbook
                                    if partition_validation_status == 'True':
                                        partition_level_diff.head(max_excel_row).to_excel(writer, sheet_name = f"difference_record_partition", startrow = 0, startcol=0, index = False)
                                    if measure_validation_status == 'True':
                                        try:
                                            measure_level_mismatched_record_df.sort_values(by=Partition_col).head(max_excel_row).to_excel(writer, sheet_name = "measure_mismatched_record", startrow = 0, startcol=0, index = False)
                                            measure_level_mismatched_detail_df.sort_values(by=Partition_col).head(max_excel_row).to_excel(writer, sheet_name = "measure_mismatched_detail", startrow = 0, startcol=0, index = False)
                                        except:
                                            measure_level_mismatched_record_df.head(max_excel_row).to_excel(writer, sheet_name = "measure_mismatched_record", startrow = 0, startcol=0, index = False)
                                            measure_level_mismatched_detail_df.head(max_excel_row).to_excel(writer, sheet_name = "measure_mismatched_detail", startrow = 0, startcol=0, index = False)
                                    if partition_count_validation_status == 'True':
                                        try:
                                            partition_count_mismatched_df.sort_values(by=Partition_col).head(max_excel_row).to_excel(writer, sheet_name = "partition_count_mismatched_record", startrow = 0, startcol=0, index = False)
                                            partition_count_mismatched_detail_df.sort_values(by=Partition_col).head(max_excel_row).to_excel(writer, sheet_name = "partition_count_mismatched_detail", startrow = 0, startcol=0, index = False)
                                        except:
                                            partition_count_mismatched_df.head(max_excel_row).to_excel(writer, sheet_name = "partition_count_mismatched_record", startrow = 0, startcol=0, index = False)
                                            partition_count_mismatched_detail_df.head(max_excel_row).to_excel(writer, sheet_name = "partition_count_mismatched_detail", startrow = 0, startcol=0, index = False)
                                print('Detailed report generated')
                                doc.add_paragraph('Detailed report generated', style = 'List Bullet')
                                detailed_report_status = 'True'
                            except Exception as e:
                                detailed_report_status = 'False'
                                print('No detailed report generated')
                                doc.add_paragraph('No detailed report generated', style = 'List Bullet')
                                print(e)
                                doc.add_paragraph(f'{e}', style= 'Intense Quote')
                                    

                            ## Upload workbook to storage
                            if detailed_report_status == 'True':
                                try:        
                                    workbook_path = path + f"{db}-{table_name}_(Netezza v. Gold).xlsx" 
                                    shutil.copy2(f"{db}-{table_name}_(Netezza v. Gold).xlsx",workbook_path)
                                    os.remove(f"{db}-{table_name}_(Netezza v. Gold).xlsx")

                                    ## Append detail summary to lists
                                    netezza_gold_database_name.append(db)
                                    netezza_gold_table_name.append(table_name)
                                    if partition_count_validation_status == 'True':
                                        netezza_gold_partition_count_mismatched_cnt.append(str(partition_count_mismatched_cnt))
                                    else:
                                        netezza_gold_partition_count_mismatched_cnt.append('')
                                    if measure_validation_status == 'True':
                                        netezza_gold_measure_level_mismatched_cnt_diff.append(str(measure_level_mismatched_cnt_diff))
                                    else:
                                        netezza_gold_measure_level_mismatched_cnt_diff.append('')
                                    netezza_gold_partition_level_mismatched_cnt_diff.append('')
                                    netezza_gold_partition_level_missing_cnt.append('')
                                    netezza_gold_table_level_mismatched_cnt_diff.append('')
                                    netezza_gold_table_level_missing_cnt.append('')
                                    if partition_validation_status == 'True':
                                        netezza_gold_partition_level_diff_cnt.append(str(partition_level_diff_cnt))
                                    else:
                                        netezza_gold_partition_level_diff_cnt.append('')
                                    netezza_gold_table_level_diff_cnt.append('')
                                    netezza_gold_detailed_report_path.append(workbook_path)

                                    print(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Gold)")
                                    doc.add_paragraph(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Gold)", style='List Bullet')
                                

                                except Exception as e:
                                    print(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Gold)")
                                    doc.add_paragraph(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Gold)", style='List Bullet')
                                    print(e)
                                    doc.add_paragraph(f'{e}', style='Intense Quote')


                        else :
                            ## This condition is to perform detailed validation for small and medium size tables
                            print(f'Attempting to perform detailed validation on {db}.{table_name}. Need to confirm it is a small to medium size table')
                            

                             
                            ## Retrieve measure level query
                            Query_Measure_Netezza=spark.sql(f" select distinct MEASURE_QUERY_NETEZZA from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and MEASURE_QUERY_NETEZZA is not null")
                            ##TODO- Include Query_Measure_Gold if it exists. Confirm the Query_Measure_Gold
                            Query_Measure_Gold=spark.sql(f" select distinct ADLS_QUERY_FOR_MEASURE from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and ADLS_QUERY_FOR_MEASURE is not null")

                            print("else Full load Non Transactional ")
                            ## Get FULL netezza table for validation
                            Query_Netezza = f"""select * from {db_tmp}.{table_name}"""
                            netezza_df=readfromNetizza(Env,Query_Netezza,db)

                                

                            ## Confirm table is small or medium size (i.e. Not greater than 5 million rows)

                            try:
                                ## Retrieve row count
                                cnt_netezza_df = int(report_dataframe_1.select(f'NETEZZA_TABLE_ROW_COUNT ({netezza_sub})').filter((col('NETEZZA_SCHEMA') == f'{db}') & (col('NETEZZA_TABLE') == f'{table_name}')).collect()[0][0]) #netezza_df.limit(max_rows_permitted + 1).count()
                                cnt_df_ADLS_gold = int(report_dataframe_1.select('GOLD_TABLE_ROW_COUNT').filter((col('SYNAPSE_SCHEMA') == f'{db}') & (col('SYNAPSE_TABLE') == f'{table_name}')).collect()[0][0])#df_synapse.limit(max_rows_permitted + 1).count()
                            except:
                                print(f'{db}.{table_name} not available in summary report')
                                cnt_netezza_df = netezza_df.limit(max_rows_permitted + 1).count()
                                cnt_df_ADLS_gold = Data_Lake_df.limit(max_rows_permitted + 1).count()

                            if (cnt_netezza_df == 0) | (cnt_df_ADLS_gold == 0):
                                print(f'Table {db}.{table_name} is empty either in Netezza or in Gold. No detailed validation performed')
                                doc.add_paragraph(f'Table {db}.{table_name} is empty either in Netezza or in Gold. No detailed validation performed', style = 'List Bullet')

                            elif (cnt_netezza_df > max_rows_permitted) | (cnt_df_ADLS_gold > max_rows_permitted):
                                print(f'Table {db}.{table_name} is a big table, with no partition column configured. No table level validation performed')
                                doc.add_paragraph(f'Table {db}.{table_name} is a big table, with no partition column configured. No table_level vallidation performed', style='List Bullet')

                                print('Will attempt to perform measure level validation')
                                doc.add_paragraph('Attempting measure level validation for a big table with no partition configured', style= 'List Bullet')

                                ## PERFORM MEASURE LEVEL VALIDATION
                                if (Query_Measure_Netezza.count() == 0) | (Query_Measure_Gold.count() == 0):
                                    print(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.')
                                    doc.add_paragraph(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.', style='List Bullet')
                                    measure_validation_status = 'False'
                                else:
                                    query_measure_netezza =Query_Measure_Netezza.collect()[0][0]
                                    query_measure_gold = Query_Measure_Gold.collect()[0][0]
                                    if (query_measure_netezza == '') | (query_measure_gold == ''):
                                        print(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.')
                                        doc.add_paragraph(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.', style='List Bullet')
                                        measure_validation_status = 'False' 
                                    else:
                                        print(f'Measure queries configured. Attempting to perform measure level validation for {db}.{table_name}')
                                        doc.add_paragraph(f'Measure queries configured. Attempting to perform measure level validation for {db}.{table_name}', style='List Bullet')

                                        try:
                                            query_measure_netezza = query_measure_netezza.replace('db_tmp', db_tmp)
                                            netezza_df_measure=readfromNetizza(Env,query_measure_netezza,db)
                                            adls_gold_df_measure = spark.sql(f"{query_measure_gold}")

                                            try:
                                                measure_level_mismatched_record_df = get_pandas(adls_gold_df_measure.subtract(netezza_df_measure))
                                                measure_level_mismatched_cnt_diff = len(measure_level_mismatched_record_df)
                                                print(f"Successfully performed measure mismatched validation on {db}.{table_name}. This is a big table does not have a partition column configured")
                                                doc.add_paragraph(f"Successfully performed measure mismatched validation on {db}.{table_name}. This is a big table does not have a partition column configured", style='List Bullet')
                                                measure_validation_status = 'True'

                                                ## Define excel writer
                                                with pd.ExcelWriter(f"{db}-{table_name}_(Netezza v. Gold).xlsx", engine="openpyxl", mode = 'w') as writer:
                                                    measure_level_mismatched_record_df.head(max_excel_row).to_excel(writer, sheet_name = f"measure_mismatched_record", startrow = 0, startcol=0, index = False)

                                                ## Upload workbook to storage
                                                try:        
                                                    workbook_path = path + f"{db}-{table_name}_(Netezza v. Gold).xlsx" 
                                                    shutil.copy2(f"{db}-{table_name}_(Netezza v. Gold).xlsx",workbook_path)
                                                    os.remove(f"{db}-{table_name}_(Netezza v. Gold).xlsx")

                                                    ## Append detail summary to lists
                                                    netezza_gold_database_name.append(db)
                                                    netezza_gold_table_name.append(table_name)
                                                    netezza_gold_partition_count_mismatched_cnt.append('')
                                                    netezza_gold_measure_level_mismatched_cnt_diff.append(str(measure_level_mismatched_cnt_diff))
                                                    netezza_gold_partition_level_mismatched_cnt_diff.append('')
                                                    netezza_gold_partition_level_missing_cnt.append('')
                                                    netezza_gold_table_level_mismatched_cnt_diff.append('')
                                                    netezza_gold_table_level_missing_cnt.append('')
                                                    netezza_gold_partition_level_diff_cnt.append('')
                                                    netezza_gold_table_level_diff_cnt.append('')
                                                    netezza_gold_detailed_report_path.append(workbook_path)

                                                    print(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Gold)")
                                                    doc.add_paragraph(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Gold)", style='List Bullet')

                                                except Exception as e:
                                                        print(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Gold)")
                                                        doc.add_paragraph(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Gold)", style='List Bullet')
                                                        print(e)
                                                        doc.add_paragraph(f'{e}', style='Intense Quote')
                                                
                                            except Exception as e:
                                                print(f"Failed to perform detailed validation on {db}.{table_name}")
                                                doc.add_paragraph(f"Failed to perform detailed validation on {db}.{table_name}", style='List Bullet')
                                                print(e)
                                                doc.add_paragraph(f'{e}', style='Intense Quote')
                                                continue

                                        except Exception as e:
                                            print(f'Bad measure query configured for {db}.{table_name}. Please correct the query. No measure level validation performed')
                                            doc.add_paragraph(f'Bad measure query configured for {db}.{table_name}. Please correct the query. No measure level validation performed', style = 'List Bullet')
                                            print(e)
                                            doc.add_paragraph(f'{e}', style='Intense Quote')
                                        
                                        
                            else:
                                print(f'Table size is less than or equal to {max_rows_permitted}. Performing detailed validation on full table {db}.{table_name}')
                                doc.add_paragraph(f'Table size is less than or equal to {max_rows_permitted}. Performing detailed validation on full table {db}.{table_name}', style = 'List Bullet')

                                try:
                                    #table_level_diff = get_pandas(netezza_df.subtract(Data_Lake_df))
                                    table_level_diff = get_pandas(trim_space(netezza_df).subtract(trim_space(Data_Lake_df)))
                                    table_level_diff_cnt = len(table_level_diff)
                                    print(f'Successfully performed full table validation on {db}.{table_name}, comparing Netezza to Gold table')
                                    doc.add_paragraph(f'Successfully performed full table validation on {db}.{table_name}, comparing Netezza to Gold table', style='List Bullet')

                                    ## Define excel writer
                                    with pd.ExcelWriter(f"{db}-{table_name}_(Netezza v. Gold).xlsx", engine="openpyxl", mode = 'w') as writer:

                                        ## Write missing and mismatched information to sheets in excel workbook
                                        table_level_diff.head(max_excel_row).to_excel(writer, sheet_name = "difference_record", startrow = 0, startcol=0, index = False)
                                        
                                        

                                    ## Upload workbook to storage
                                    try:        
                                        workbook_path = path + f"{db}-{table_name}_(Netezza v. Gold).xlsx" 
                                        shutil.copy2(f"{db}-{table_name}_(Netezza v. Gold).xlsx",workbook_path)
                                        os.remove(f"{db}-{table_name}_(Netezza v. Gold).xlsx")

                                        ## Append detail summary to lists
                                        netezza_gold_database_name.append(db)
                                        netezza_gold_table_name.append(table_name)
                                        netezza_gold_partition_count_mismatched_cnt.append('')
                                        netezza_gold_measure_level_mismatched_cnt_diff.append('')
                                        netezza_gold_partition_level_mismatched_cnt_diff.append('')
                                        netezza_gold_partition_level_missing_cnt.append('')
                                        netezza_gold_table_level_mismatched_cnt_diff.append('')
                                        netezza_gold_table_level_missing_cnt.append('')
                                        netezza_gold_partition_level_diff_cnt.append('')
                                        netezza_gold_table_level_diff_cnt.append(table_level_diff_cnt)
                                        netezza_gold_detailed_report_path.append(workbook_path)  

                                        print(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Gold)")
                                        doc.add_paragraph(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Gold)", style='List Bullet')
                                        

                                    except Exception as e:
                                        print(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Gold)")
                                        doc.add_paragraph(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Gold)", style='List Bullet')
                                        print(e)
                                        doc.add_paragraph(f'{e}', style='Intense Quote')

                                except Exception as e:
                                    print(f'Failed to generate detailed report for {db}.{table_name}, when comparing Netezza to Gold table')
                                    doc.add_paragraph(f'Failed to generate detailed report for {db}.{table_name}, when comparing Netezza to Gold table', style='List Bullet')
                                    print(e)
                                    doc.add_paragraph(f'{e}', style='Intense Quote')

                except Exception as e:
                    print(f'ERROR OCCURRED: Failed to generate detailed report for {db}.{table_name}, when comparing Netezza to Gold table')
                    doc.add_paragraph(f'ERROR OCCURRED: Failed to generate detailed report for {db}.{table_name}, when comparing Netezza to Gold table', style='List Bullet')
                    print(e)
                    doc.add_paragraph(f'{e}', style='Intense Quote')
                    #continue                   
                                    
        doc.add_paragraph('')

        ## Upload log file
        save_log(doc)

    except Exception as e:
        doc.add_paragraph("Data Validation Run Failed", style="List Bullet")
        doc.add_paragraph('')
        print(e)
        doc.add_paragraph(f'{e}', style='Intense Quote')

        ## Upload log file
        save_log(doc)

        ## Exit the notebook run
        dbutils.notebook.exit("Data Validation Run Failed") 


    
            

In [0]:
netezza_silver_database_name = []
netezza_silver_table_name = []
netezza_silver_partition_count_mismatched_cnt = []
netezza_silver_measure_level_mismatched_cnt_diff = []
netezza_silver_partition_level_mismatched_cnt_diff = []
netezza_silver_partition_level_missing_cnt = []
netezza_silver_table_level_mismatched_cnt_diff = []
netezza_silver_table_level_missing_cnt = []
netezza_silver_measure_level_diff_cnt = []
netezza_silver_partition_level_diff_cnt = []
netezza_silver_table_level_diff_cnt = []
netezza_silver_detailed_report_path = []

In [0]:
tab_si_cnt = 0

In [0]:
if (PL_TL != '7GS') & (Load_Type == 'Historical'):     
    
    from pyspark.sql.functions import *
    from pyspark.sql.types import DecimalType, IntegerType, DateType, StructType, TimestampType
    from datetime import date, timedelta, datetime
    from delta.tables import *

    # Header for log document
    doc.add_heading('Log From Detailed Report Generation Comparing Netezza v. Silver', 2)
    doc.add_heading('KEY TYPE UNIQUE = Y',3)
    container = 'silver'

    try:

        for db in relevant_silver_db:

            if Env == 'QA':
                db_tmp = f"GDC_{db}"
            elif Env == 'PROD_DR':
                db_tmp = f"{db}_DR"
            else:
                db_tmp = db
            print('DATABASE: ', db)


            for table_dict in [v for d in validation_tables_info_2 for k,v in d.items() if k == db][0]:
                print(table_dict)
                table_name=list(table_dict.keys())[0]
                print('TABLE NAME: ',table_name)    

                try:     
                    partition_key=list(table_dict.values())[0][0]
                    print(type(partition_key))
                    primary_key=list(table_dict.values())[0]
                    print('PRIMARY KEY: ',primary_key)
                    Partition_column=spark.sql(f"select distinct PARTITION_COLUMN from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and PARTITION_COLUMN is not null")
                    KEY_TYPE=spark.sql(f"select distinct KEY_TYPE_UNIQUE from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and KEY_TYPE_UNIQUE is not null")
                    KEY_TYPES=KEY_TYPE.collect()[0][0]
                    print('KEY_TYPES: ',KEY_TYPES)

                

                    if KEY_TYPES == "Y" :
                        tab_si_cnt += 1
                        print(tab_si_cnt)

                        doc.add_heading(f"{db}.{table_name}", 4)
                        
                        if Partition_column.count() == 0:
                            Partition_col = None
                            print(f'No partition column configured for {db}.{table_name}')
                            doc.add_paragraph(f'No partition column configured for {db}.{table_name}', style='List Bullet')
                        else:
                            Partition_col =Partition_column.collect()[0][0]
                            print(f'Partition column configured for {db}.{table_name} is {Partition_col}')
                            doc.add_paragraph(f'Partition column configured for {db}.{table_name} is {Partition_col}', style='List Bullet')
                        
                        ## The ADLS silver table
                        Data_Lake_df=spark.read.format("delta").option("header","true").load("/mnt/silver/"+db+"/ADMIN/"+table_name)
                        Data_Lake_df.createOrReplaceTempView("Data_Lake_df")

                        if ((Partition_col != None) and (Partition_col != 'NA') ):
                            ## Retrieve dataframe of the queries for row count per partition for Netezza tables, and Retrieve dataframe of query for generating a measure from Netezza and ADLS_Gold
                            print(f'Retrieving Partition, Measure and Field Validation Queries from Fwk_Gold_Tables for {db}.{table_name}')
                            doc.add_paragraph(f'Retrieving Partition, Measure and Field Validation Queries from Fwk_Gold_Tables for {db}.{table_name}', style='List Bullet')
                            
                            Query_Agg_Netezza=spark.sql(f"select distinct PARTITION_QUERY_NETEZZA from gold_df where TABLE_NAME = '{table_name}' and DATABASE ='{db}' and PARTITION_QUERY_NETEZZA is not null")
                            Query_Measure_Netezza=spark.sql(f" select distinct MEASURE_QUERY_NETEZZA from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and MEASURE_QUERY_NETEZZA is not null")
                            Query_Measure_Silver=spark.sql(f" select distinct ADLS_QUERY_FOR_MEASURE from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and ADLS_QUERY_FOR_MEASURE is not null")

                            Query_Field_Level_Netezza=spark.sql(f"select distinct FIELD_VALIDATION_QUERY_NETEZZA from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and FIELD_VALIDATION_QUERY_NETEZZA is not null")
                            Query_Field_Level_Silver=spark.sql(f"select distinct ADLS_QUERY_FOR_FIELD from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and ADLS_QUERY_FOR_FIELD is not null")  


                            ## Retrieve dataframe of the field used for validation (YEAR, MONTH, DAY)
                            PARTITION_ON=spark.sql(f"select PARTITION_ON from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and PARTITION_ON is not null")
                            Partition_Type=PARTITION_ON.collect()[0][0].upper()
                            print('PARTITION TYPE:', Partition_Type)

                            ## Skip table if no queries are configured
                            if (Query_Field_Level_Netezza.count() == 0) & (Query_Measure_Netezza.count() == 0) & (Query_Agg_Netezza.count() == 0):
                                print(f"No field validation query or measure query or partition query configured. No detailed report generated for {db}.{table_name}")
                                doc.add_paragraph(f"No field validation query or measure query or partition query configured. No detailed report generated for {db}.{table_name}", style = 'List Bullet')
                                continue
                            

                            ## Get the various queries, partition field and partition value
                            '''
                            if Query_Agg_Netezza.count()!=0:
                                query_table_load_netezza =Query_Agg_Netezza.collect()[0][0]
                                netezza_partition_cnt_df=readfromNetizza(Env,query_table_load_netezza,db)
                            ''' 
                                

                            ## PERFORM PARTITION COUNT VALIDATION
                            if (Query_Agg_Netezza.count() == 0):
                                print(f'No partition queries configured for {db}.{table_name} in Fwk_Gold_Tables for Netezza. No partition count validation performed.')
                                doc.add_paragraph(f'No partition queries configured for {db}.{table_name} in Fwk_Gold_Tables for Netezza. No partition count validation performed.', style='List Bullet')
                                partition_count_validation_status = 'False'
                            else:
                                query_PARTITION_CNT_NZ = Query_Agg_Netezza.collect()[0][0]
                                if (query_PARTITION_CNT_NZ == ''):
                                    partition_count_validation_status = 'False'
                                    print('No Partition Query is configured. No partition count validation performed')
                                    doc.add_paragraph(('No Partition Query is configured. No partition count validation performed'), style= 'List Bullet')
                                else:
                                    print(f'Attempting to perform partition count validation for {db}.{table_name}')
                                    doc.add_paragraph(f'Attempting to perform partition count validation for {db}.{table_name}', style='List Bullet')

                                    try:
                                        query_PARTITION_CNT_NZ = query_PARTITION_CNT_NZ.replace('db_tmp', db_tmp)
                                        netezza_partition_cnt_df = readfromNetizza(Env, query_PARTITION_CNT_NZ, db)
                                        if Partition_Type == 'YEAR' :
                                            Data_Lake_df_PARTITION_CNT=spark.sql(f"select DATE_FORMAT({Partition_col}, 'yyyy') as {Partition_col}, count(*) as CNT from Data_Lake_df group by DATE_FORMAT({Partition_col}, 'yyyy')")
                                        elif Partition_Type == 'MONTH' :
                                            Data_Lake_df_PARTITION_CNT=spark.sql(f"select DATE_FORMAT({Partition_col}, 'yyyy MMM') as {Partition_col}, count(*) as CNT from Data_Lake_df group by DATE_FORMAT({Partition_col}, 'yyyy MMM')")
                                        elif Partition_Type == 'DAY' :
                                            Data_Lake_df_PARTITION_CNT=spark.sql(f"select DATE_FORMAT({Partition_col}, 'yyyy MMM dd') as {Partition_col}, count(*) as CNT from Data_Lake_df group by DATE_FORMAT({Partition_col}, 'yyyy MMM dd')")

                                        try:
                                            partition_count_mismatched_cnt, partition_count_mismatched_df, partition_count_mismatched_detail_cnt, partition_count_mismatched_detail_df = find_Mismatched(netezza_partition_cnt_df,Data_Lake_df_PARTITION_CNT,Partition_col)
                                            print(f"Successfully performed partition count validation for {db}.{table_name}")
                                            doc.add_paragraph(f"Successfully performed partition count validation for {db}.{table_name}", style = 'List Bullet')
                                            partition_count_validation_status = 'True'
                                        except Exception as e:
                                            partition_count_validation_status = 'False'
                                            print(f'Failed to perform partition count validation for {db}.{table_name}')
                                            doc.add_paragraph(f'Failed to perform partition count validation for {db}.{table_name}', style='List Bullet')
                                            print(e)
                                            doc.add_paragraph(f'{e}', style='Intense Quote')
                                    except Exception as e:
                                        partition_count_validation_status = 'False'
                                        print(f'Bad partition query configured for {db}.{table_name}. Please correct the query. No partition count validation performed')
                                        doc.add_paragraph(f'Bad partition query configured for {db}.{table_name}. Please correct the query. No partition count validation performed', style = 'List Bullet')
                                        print(e)
                                        doc.add_paragraph(f'{e}', style='Intense Quote')

                        

                            ## PERFORM MEASURE LEVEL VALIDATION
                            if (Query_Measure_Netezza.count() == 0) | (Query_Measure_Silver.count() == 0):
                                print(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.')
                                doc.add_paragraph(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.', style='List Bullet')
                                measure_validation_status = 'False'
                            else:
                                query_measure_netezza =Query_Measure_Netezza.collect()[0][0]
                                query_measure_silver = Query_Measure_Silver.collect()[0][0]
                                if (query_measure_netezza == '') | (query_measure_silver == ''):
                                    print(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.')
                                    doc.add_paragraph(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.', style='List Bullet')
                                    measure_validation_status = 'False' 
                                else:
                                    print(f'Attempting to perform measure level validation for {db}.{table_name}')
                                    doc.add_paragraph(f'Attempting to perform measure level validation for {db}.{table_name}', style='List Bullet')

                                    try:
                                        query_measure_netezza = query_measure_netezza.replace('db_tmp', db_tmp)
                                        netezza_df_measure=readfromNetizza(Env,query_measure_netezza,db)
                                        adls_silver_df_measure = spark.sql(f"{query_measure_silver}")

                                        try:
                                            measure_level_mismatched_cnt_diff, measure_level_mismatched_record_df, measure_level_mismatched_detail_cnt, measure_level_mismatched_detail_df = find_Mismatched(netezza_df_measure,adls_silver_df_measure,Partition_col)
                                            print(f"Successfully performed measure level validation for {db}.{table_name}")
                                            doc.add_paragraph(f"Successfully performed measure level validation for {db}.{table_name}", style='List Bullet')
                                            measure_validation_status = 'True'

                                        except Exception as e:
                                            measure_validation_status = 'False'
                                            print(f'Failed to perform measure level validation for {db}.{table_name}')
                                            doc.add_paragraph(f'Failed to perform measure level validation for {db}.{table_name}', style='List Bullet')
                                            print(e)
                                            doc.add_paragraph(f'{e}', style='Intense Quote')
                                    except Exception as e:
                                        measure_validation_status = 'False'
                                        print(f'Bad measure query configured for {db}.{table_name}. Please correct the query. No measure level validation performed')
                                        doc.add_paragraph(f'Bad measure query configured for {db}.{table_name}. Please correct the query. No measure level validation performed', style = 'List Bullet')
                                        print(e)
                                        doc.add_paragraph(f'{e}', style='Intense Quote')
                                                            

                            ## PERFORM PARTITION LEVEL VALIDATION

                            if (Query_Field_Level_Netezza.count()==0) | (Query_Field_Level_Silver.count()==0):
                                print(f'No field validation queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No partition level validation performed.')
                                doc.add_paragraph(f'No field validation queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No partition level validation performed.', style='List Bullet')
                                partition_validation_status = 'False'
                            else:
                                query_PARTITION_NZ = Query_Field_Level_Netezza.collect()[0][0]
                                query_PARTITION_Silver = Query_Field_Level_Silver.collect()[0][0]
                                if (query_PARTITION_NZ == '') | (query_PARTITION_Silver == ''):
                                    print(f'No field validation queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No partition level validation performed.')
                                    doc.add_paragraph(f'No field validation queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No partition level validation performed.', style='List Bullet')
                                    partition_validation_status = 'False'  
                                else:
                                    try:
                                        query_PARTITION_NZ = query_PARTITION_NZ.replace('db_tmp', db_tmp)
                                        netezza_df_partition=readfromNetizza(Env,query_PARTITION_NZ,db)
                                        Data_Lake_df_1=spark.sql(query_PARTITION_Silver)  

                                        ## Validate that the partition size is within the max row permitted limit
                                        print('Attempting to perform partition level validation. Need to confirm that the partition size is within acceptable limit')
                                        doc.add_paragraph('Attempting to perform partition level validation. Need to confirm that the partition size is within acceptable limit', style = 'List Bullet')
                                        
                                        cnt_netezza_df_partition = netezza_df_partition.limit(max_rows_permitted + 1).count()
                                        cnt_Data_Lake_df_1 = Data_Lake_df_1.limit(max_rows_permitted + 1).count()

                                        if (cnt_netezza_df_partition == 0) | (cnt_Data_Lake_df_1 == 0):
                                            print(f'Partition for table {db}.{table_name} is empty either in Netezza or in Gold. No partition validation performed')
                                            doc.add_paragraph(f'Partition for table {db}.{table_name} is empty either in Netezza or in Gold. No partition validation performed', style = 'List Bullet')
                                            partition_validation_status = 'False'

                                        elif (cnt_netezza_df_partition > max_rows_permitted) | (cnt_Data_Lake_df_1 > max_rows_permitted):
                                            print(f'Partition for {db}.{table_name} is greater than {max_rows_permitted}. No partition validation performed. Please reconfigure the partition for {db}.{table_name}')
                                            doc.add_paragraph(f'Partition for {db}.{table_name} is greater than {max_rows_permitted}. No partition validation performed. Please reconfigure the partition for {db}.{table_name}', style = 'List Bullet')
                                            partition_validation_status = 'False'
                                        else:
                                            ## Perform validation on the partition
                                            print(f'Partition for {db}.{table_name} is less than or equal to {max_rows_permitted}.')
                                            doc.add_paragraph(f'Partition for {db}.{table_name} is less than or equal to {max_rows_permitted}.', style = 'List Bullet')
                                            try:
                                                #partition_level_mismatched_cnt_diff, partition_level_mismatched_record_df, partition_level_mismatched_record_detail_cnt, partition_level_mismatched_record_detail_df, partition_level_missing_cnt, partition_level_missing_output_df = find_difference(netezza_df_partition, Data_Lake_df_1, primary_key)
                                                partition_level_mismatched_cnt_diff, partition_level_mismatched_record_df, partition_level_mismatched_record_detail_cnt, partition_level_mismatched_record_detail_df, partition_level_missing_cnt, partition_level_missing_output_df = find_difference(trim_space(netezza_df_partition), trim_space(Data_Lake_df_1), primary_key)
                                                partition_level_mismatched_record_df.spark.cache()
                                                partition_level_mismatched_record_detail_df.spark.cache()
                                                partition_level_missing_output_df.spark.cache()
                                                

                                                print(f"Successfully performed partition level validation for {db}.{table_name}")
                                                doc.add_paragraph(f"Successfully performed partition level validation for {db}.{table_name}", style='List Bullet')
                                                partition_validation_status = 'True'
                                            except Exception as e:
                                                partition_validation_status = 'False'
                                                print(f'Failed to generate detailed report for {db}.{table_name}_(Netezza v. Silver)')
                                                doc.add_paragraph(f'Failed to generate detailed report for {db}.{table_name}_(Netezza v. Silver)', style='List Bullet')
                                                print(e)
                                                doc.add_paragraph(f'{e}', style='Intense Quote') 
                                    except Exception as e:
                                        partition_validation_status = 'False'
                                        print(f'Bad field validation query configured for {db}.{table_name}. Please correct the query. No partition level validation performed')
                                        doc.add_paragraph(f'Bad field validation query configured for {db}.{table_name}. Please correct the query. No partition level validation performed', style = 'List Bullet')
                                        print(e)
                                        doc.add_paragraph(f'{e}', style='Intense Quote')
    

                            ## Define excel writer
                            try:
                                with pd.ExcelWriter(f"{db}-{table_name}_(Netezza v. Silver).xlsx", engine="openpyxl", mode = 'w') as writer:

                                    ## Write missing and mismatched information to sheets in excel workbook
                                    if partition_validation_status == 'True':
                                        try:
                                            partition_level_missing_output_df.sort_values(by=primary_key).head(max_excel_row).to_excel(writer, sheet_name = f"missing_record_partition", startrow = 0, startcol=0, index = False)
                                        except:
                                            partition_level_missing_output_df.head(max_excel_row).to_excel(writer, sheet_name = f"missing_record_partition", startrow = 0, startcol=0, index = False)
                                        try:
                                            partition_level_mismatched_record_df.sort_values(by=primary_key).head(max_excel_row).to_excel(writer, sheet_name = f"mismatched_record_partition", startrow = 0, startcol=0, index = False)
                                            partition_level_mismatched_record_detail_df.sort_values(by=primary_key).head(max_excel_row).to_excel(writer, sheet_name = f"mismatched_detail_partition", startrow = 0, startcol=0, index = False)
                                        except:
                                            partition_level_mismatched_record_df.head(max_excel_row).to_excel(writer, sheet_name = f"mismatched_record_partition", startrow = 0, startcol=0, index = False)
                                            partition_level_mismatched_record_detail_df.head(max_excel_row).to_excel(writer, sheet_name = f"mismatched_detail_partition", startrow = 0, startcol=0, index = False)
                                    if measure_validation_status == 'True':
                                        try:
                                            measure_level_mismatched_record_df.sort_values(by=Partition_col).head(max_excel_row).to_excel(writer, sheet_name = "measure_mismatched_record", startrow = 0, startcol=0, index = False)
                                            measure_level_mismatched_detail_df.sort_values(by=Partition_col).head(max_excel_row).to_excel(writer, sheet_name = "measure_mismatched_detail", startrow = 0, startcol=0, index = False)
                                        except:
                                            measure_level_mismatched_record_df.head(max_excel_row).to_excel(writer, sheet_name = "measure_mismatched_record", startrow = 0, startcol=0, index = False)
                                            measure_level_mismatched_detail_df.head(max_excel_row).to_excel(writer, sheet_name = "measure_mismatched_detail", startrow = 0, startcol=0, index = False)
                                    if partition_count_validation_status == 'True':
                                        try:
                                            partition_count_mismatched_df.sort_values(by=Partition_col).head(max_excel_row).to_excel(writer, sheet_name = "partition_count_mismatched_record", startrow = 0, startcol=0, index = False)
                                            partition_count_mismatched_detail_df.sort_values(by=Partition_col).head(max_excel_row).to_excel(writer, sheet_name = "partition_count_mismatched_detail", startrow = 0, startcol=0, index = False)
                                        except:
                                            partition_count_mismatched_df.head(max_excel_row).to_excel(writer, sheet_name = "partition_count_mismatched_record", startrow = 0, startcol=0, index = False)
                                            partition_count_mismatched_detail_df.head(max_excel_row).to_excel(writer, sheet_name = "partition_count_mismatched_detail", startrow = 0, startcol=0, index = False)
                                print('Detailed report generated')
                                doc.add_paragraph('Detailed report generated', style = 'List Bullet')
                                detailed_report_status = 'True'
                            except Exception as e:
                                detailed_report_status = 'False'
                                print('No detailed report generated')
                                doc.add_paragraph('No detailed report generated', style = 'List Bullet')
                                print(e)
                                doc.add_paragraph(f'{e}', style= 'Intense Quote')

                            ## Upload workbook to storage
                            if detailed_report_status == 'True':
                                try:        
                                    workbook_path = path + f"{db}-{table_name}_(Netezza v. Silver).xlsx" 
                                    shutil.copy2(f"{db}-{table_name}_(Netezza v. Silver).xlsx",workbook_path)
                                    os.remove(f"{db}-{table_name}_(Netezza v. Silver).xlsx")

                                    ## Append detail summary to lists
                                    netezza_silver_database_name.append(db)
                                    netezza_silver_table_name.append(table_name)
                                    if partition_count_validation_status == 'True':
                                        netezza_silver_partition_count_mismatched_cnt.append(str(partition_count_mismatched_cnt))
                                    else:
                                        netezza_silver_partition_count_mismatched_cnt.append('')
                                    if measure_validation_status == 'True':
                                        netezza_silver_measure_level_mismatched_cnt_diff.append(str(measure_level_mismatched_cnt_diff))
                                    else:
                                        netezza_silver_measure_level_mismatched_cnt_diff.append('')
                                    if partition_validation_status == 'True':
                                        netezza_silver_partition_level_mismatched_cnt_diff.append(str(partition_level_mismatched_cnt_diff))
                                        netezza_silver_partition_level_missing_cnt.append(str(partition_level_missing_cnt))
                                    else:
                                        netezza_silver_partition_level_mismatched_cnt_diff.append('')
                                        netezza_silver_partition_level_missing_cnt.append('')
                                    netezza_silver_table_level_mismatched_cnt_diff.append('')
                                    netezza_silver_table_level_missing_cnt.append('')
                                    netezza_silver_partition_level_diff_cnt.append('')
                                    netezza_silver_table_level_diff_cnt.append('')
                                    netezza_silver_detailed_report_path.append(workbook_path)

                                    print(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Silver)")
                                    doc.add_paragraph(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Silver)", style='List Bullet')
                                    
                                except Exception as e:
                                    print(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Silver)")
                                    doc.add_paragraph(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Silver)", style='List Bullet')
                                    print(e)
                                    doc.add_paragraph(f'{e}', style='Intense Quote')


                        else :
                            ## This condition is to perform detailed validation for small and medium size tables
                            print(f'Attempting to perform detailed validation on {db}.{table_name}. Need to confirm it is a small to medium size table')
                            

                            
                            ## Retrieve measure level query
                            Query_Measure_Netezza=spark.sql(f" select distinct MEASURE_QUERY_NETEZZA from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and MEASURE_QUERY_NETEZZA is not null")
                            ##TODO- Include Query_Measure_Silver if it exists. Confirm the Query_Measure_Silver
                            Query_Measure_Silver=spark.sql(f" select distinct ADLS_QUERY_FOR_MEASURE from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and ADLS_QUERY_FOR_MEASURE is not null")

                            print("else Full load Non Transactional ")
                            ## Get FULL netezza table for validation
                            Query_Netezza = f"""select * from {db_tmp}.{table_name}"""
                            netezza_df=readfromNetizza(Env,Query_Netezza,db)

                                

                            ## Confirm table is small or medium size (i.e. Not greater than 5 million rows)
                            #max_rows_permitted = 5000000

                            try:
                                ## Retrieve row count
                                cnt_netezza_df = int(report_dataframe_2.select(f'NETEZZA_TABLE_ROW_COUNT ({netezza_sub})').filter((col('NETEZZA_SCHEMA') == f'{db}') & (col('NETEZZA_TABLE') == f'{table_name}')).collect()[0][0]) #netezza_df.limit(max_rows_permitted + 1).count()
                                cnt_df_ADLS_silver = int(report_dataframe_2.select('SILVER_TABLE_ROW_COUNT').filter((col('SILVER_SCHEMA') == f'{db}') & (col('SILVER_TABLE') == f'{table_name}')).collect()[0][0])#df_synapse.limit(max_rows_permitted + 1).count()
                            except:
                                print(f'{db}.{table_name} not available in summary report')
                                cnt_netezza_df = netezza_df.limit(max_rows_permitted + 1).count()
                                cnt_df_ADLS_silver = Data_Lake_df.limit(max_rows_permitted + 1).count()

                            if (cnt_netezza_df == 0) | (cnt_df_ADLS_silver == 0):
                                print(f'Table {db}.{table_name} is empty either in Netezza or in Silver. No detailed validation performed')
                                doc.add_paragraph(f'Table {db}.{table_name} is empty either in Netezza or in Silver. No detailed validation performed', style = 'List Bullet')

                            elif (cnt_netezza_df > max_rows_permitted) | (cnt_df_ADLS_silver > max_rows_permitted):
                                print(f'Table {db}.{table_name} is a big table, with no partition column configured. No table level validation performed')
                                doc.add_paragraph(f'Table {db}.{table_name} is a big table, with no partition column configures. No table level validation performed', style='List Bullet')

                                print('Will attempt to perform measure level validation')
                                doc.add_paragraph('Attempting measure level validation for a big table with no partition configured', style= 'List Bullet')

                                ## PERFORM MEASURE LEVEL VALIDATION
                                if (Query_Measure_Netezza.count() == 0) | (Query_Measure_Silver.count() == 0):
                                    print(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.')
                                    doc.add_paragraph(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.', style='List Bullet')
                                    measure_validation_status = 'False'
                                else:
                                    query_measure_netezza =Query_Measure_Netezza.collect()[0][0]
                                    query_measure_silver = Query_Measure_Silver.collect()[0][0]
                                    if (query_measure_netezza == '') | (query_measure_silver == ''):
                                        print(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.')
                                        doc.add_paragraph(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.', style='List Bullet')
                                        measure_validation_status = 'False' 
                                    else:
                                        print(f'Measure queries configured. Attempting to perform measure level validation for {db}.{table_name}')
                                        doc.add_paragraph(f'Measure queries configured. Attempting to perform measure level validation for {db}.{table_name}', style='List Bullet')

                                        try:
                                            query_measure_netezza = query_measure_netezza.replace('db_tmp', db_tmp)
                                            netezza_df_measure=readfromNetizza(Env,query_measure_netezza,db)
                                            adls_silver_df_measure = spark.sql(f"{query_measure_silver}")

                                            try:
                                                measure_level_mismatched_record_df = get_pandas(adls_silver_df_measure.subtract(netezza_df_measure))
                                                measure_level_mismatched_cnt_diff = len(measure_level_mismatched_record_df)
                                                print(f"Successfully performed measure mismatched validation on {db}.{table_name}. This is a big table does not have a partition column configured")
                                                doc.add_paragraph(f"Successfully performed measure mismatched validation on {db}.{table_name}. This is a big table does not have a partition column configured", style='List Bullet')
                                                measure_validation_status = 'True'
                                
                                                ## Define excel writer
                                                with pd.ExcelWriter(f"{db}-{table_name}_(Netezza v. Silver).xlsx", engine="openpyxl", mode = 'w') as writer:

                                                    ## Write missing and mismatched information to sheets in excel workbook
                                                    measure_level_mismatched_record_df.head(max_excel_row).to_excel(writer, sheet_name = "measure_mismatched_record", startrow = 0, startcol=0, index = False)

                                                ## Upload workbook to storage
                                                try:        
                                                    workbook_path = path + f"{db}-{table_name}_(Netezza v. Silver).xlsx" 
                                                    shutil.copy2(f"{db}-{table_name}_(Netezza v. Silver).xlsx",workbook_path)
                                                    os.remove(f"{db}-{table_name}_(Netezza v. Silver).xlsx")

                                                    ## Append detail summary to lists
                                                    netezza_silver_database_name.append(db)
                                                    netezza_silver_table_name.append(table_name)
                                                    netezza_silver_partition_count_mismatched_cnt.append('')
                                                    netezza_silver_measure_level_mismatched_cnt_diff.append(str(measure_level_mismatched_cnt_diff))
                                                    netezza_silver_partition_level_mismatched_cnt_diff.append('')
                                                    netezza_silver_partition_level_missing_cnt.append('')
                                                    netezza_silver_table_level_mismatched_cnt_diff.append('')
                                                    netezza_silver_table_level_missing_cnt.append('')
                                                    netezza_silver_partition_level_diff_cnt.append('')
                                                    netezza_silver_table_level_diff_cnt.append('')
                                                    netezza_silver_detailed_report_path.append(workbook_path)

                                                    print(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Silver)")
                                                    doc.add_paragraph(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Silver)", style='List Bullet')

                                                except Exception as e:
                                                    print(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Silver)")
                                                    doc.add_paragraph(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Silver)", style='List Bullet')
                                                    print(e)
                                                    doc.add_paragraph(f'{e}', style='Intense Quote')
                                            
                                            except Exception as e:
                                                print(f"Failed to perform detailed validation on {db}.{table_name}")
                                                doc.add_paragraph(f"Failed to perform detailed validation on {db}.{table_name}", style='List Bullet')
                                                print(e)
                                                doc.add_paragraph(f'{e}', style='Intense Quote')
                                                continue

                                        except Exception as e:
                                            print(f'Bad measure query configured for {db}.{table_name}. Please correct the query. No measure level validation performed')
                                            doc.add_paragraph(f'Bad measure query configured for {db}.{table_name}. Please correct the query. No measure level validation performed', style = 'List Bullet')
                                            doc.add_paragraph(f'Netezza measure query: {query_measure_netezza}', style = 'Intense Quote')
                                            doc.add_paragraph(f'Silver measure query: {query_measure_silver}', style = 'Intense Quote')
                                            print(e)
                                            doc.add_paragraph(f'{e}', style='Intense Quote')
            
                            else:
                                print(f'Table size is less than or equal to {max_rows_permitted}. Performing detailed validation on full table {db}.{table_name}')
                                doc.add_paragraph(f'Table size is less than or equal to {max_rows_permitted}. Performing detailed validation on full table {db}.{table_name}', style = 'List Bullet')

                                try:
                                    #table_level_mismatched_cnt_diff, table_level_mismatched_record_df, table_level_mismatched_record_detail_cnt, table_level_mismatched_record_detail_df, table_level_missing_cnt, table_level_missing_output_df = find_difference(netezza_df, Data_Lake_df, primary_key)
                                    table_level_mismatched_cnt_diff, table_level_mismatched_record_df, table_level_mismatched_record_detail_cnt, table_level_mismatched_record_detail_df, table_level_missing_cnt, table_level_missing_output_df = find_difference(trim_space(netezza_df), trim_space(Data_Lake_df), primary_key)
                                    print(f'Successfully performed full table validation on {db}.{table_name}, comparing Netezza to Silver table')
                                    doc.add_paragraph(f'Successfully performed full table validation on {db}.{table_name}, comparing Netezza to Silver table', style='List Bullet')

                                    ## Define excel writer
                                    with pd.ExcelWriter(f"{db}-{table_name}_(Netezza v. Silver).xlsx", engine="openpyxl", mode = 'w') as writer:

                                        ## Write missing and mismatched information to sheets in excel workbook
                                        try:
                                            table_level_missing_output_df.sort_values(by=primary_key).head(max_excel_row).to_excel(writer, sheet_name = "missing_record", startrow = 0, startcol=0, index = False)
                                        except:
                                            table_level_missing_output_df.head(max_excel_row).to_excel(writer, sheet_name = "missing_record", startrow = 0, startcol=0, index = False)
                                        try:
                                            table_level_mismatched_record_df.sort_values(by=primary_key).head(max_excel_row).to_excel(writer, sheet_name = "mismatched_record", startrow = 0, startcol=0, index = False)
                                            table_level_mismatched_record_detail_df.sort_values(by=primary_key).head(max_excel_row).to_excel(writer, sheet_name = "mismatched_detail", startrow = 0, startcol=0, index = False)
                                        except:
                                            table_level_mismatched_record_df.head(max_excel_row).to_excel(writer, sheet_name = "mismatched_record", startrow = 0, startcol=0, index = False)
                                            table_level_mismatched_record_detail_df.head(max_excel_row).to_excel(writer, sheet_name = "mismatched_detail", startrow = 0, startcol=0, index = False)
                                        

                                    ## Upload workbook to storage
                                    try:        
                                        workbook_path = path + f"{db}-{table_name}_(Netezza v. Silver).xlsx" 
                                        shutil.copy2(f"{db}-{table_name}_(Netezza v. Silver).xlsx",workbook_path)
                                        os.remove(f"{db}-{table_name}_(Netezza v. Silver).xlsx")

                                        ## Append detail summary to lists
                                        netezza_silver_database_name.append(db)
                                        netezza_silver_table_name.append(table_name)
                                        netezza_silver_partition_count_mismatched_cnt.append('')
                                        netezza_silver_measure_level_mismatched_cnt_diff.append('')
                                        netezza_silver_partition_level_mismatched_cnt_diff.append('')
                                        netezza_silver_partition_level_missing_cnt.append('')
                                        netezza_silver_table_level_mismatched_cnt_diff.append(str(table_level_mismatched_cnt_diff))
                                        netezza_silver_table_level_missing_cnt.append(str(table_level_missing_cnt))
                                        netezza_silver_partition_level_diff_cnt.append('')
                                        netezza_silver_table_level_diff_cnt.append('')
                                        netezza_silver_detailed_report_path.append(workbook_path)

                                        print(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Silver)")
                                        doc.add_paragraph(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Silver)", style='List Bullet')
                                        

                                    except Exception as e:
                                        print(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Silver)")
                                        doc.add_paragraph(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Silver)", style='List Bullet')
                                        print(e)
                                        doc.add_paragraph(f'{e}', style='Intense Quote')

                                except Exception as e:
                                    print(f'Failed to generate detailed report for {db}.{table_name}, when comparing Netezza to Silver table')
                                    doc.add_paragraph(f'Failed to generate detailed report for {db}.{table_name}, when comparing Netezza to Silver table', style='List Bullet')
                                    print(e)
                                    doc.add_paragraph(f'{e}', style='Intense Quote')

                except Exception as e:
                    print(f'ERROR OCCURRED: Failed to generate detailed report for {db}.{table_name}, when comparing Netezza to Silver table')
                    doc.add_paragraph(f'ERROR OCCURRED: Failed to generate detailed report for {db}.{table_name}, when comparing Netezza to Silver table', style='List Bullet')
                    print(e)
                    doc.add_paragraph(f'{e}', style='Intense Quote')
                    #continue
                                        
        ## Upload log file
        save_log(doc)

    except Exception as e:
        doc.add_paragraph(f'{e}', style='Intense Quote')
        doc.add_paragraph("Data Validation Run Failed", style="List Bullet")
        print(e)
        doc.add_paragraph(f'{e}', style='Intense Quote')
        doc.add_paragraph('')

        ## Upload log file
        save_log(doc)

        ## Exit the notebook run
        dbutils.notebook.exit("Data Validation Run Failed") 



In [0]:
if (PL_TL != '7GS') & (Load_Type == 'Historical'): 
    
    from pyspark.sql.functions import *
    from pyspark.sql.types import DecimalType, IntegerType, DateType, StructType, TimestampType
    from datetime import date, timedelta, datetime
    from delta.tables import *



    # Header for log document
    doc.add_heading('KEY TYPE UNIQUE = N',3)
    container = 'silver'

    try:

        for db in relevant_silver_db:

            if Env == 'QA':
                db_tmp = f"GDC_{db}"
            elif Env == 'PROD_DR':
                db_tmp = f"{db}_DR"
            else:
                db_tmp = db
            print('DATABASE: ', db)


            for table_dict in [v for d in validation_tables_info_2 for k,v in d.items() if k == db][0]:
                print(table_dict)
                table_name=list(table_dict.keys())[0]
                print('TABLE NAME: ',table_name)     

                try:   
                
                    Partition_column=spark.sql(f"select distinct PARTITION_COLUMN from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and PARTITION_COLUMN is not null")
                    KEY_TYPE=spark.sql(f"select distinct KEY_TYPE_UNIQUE from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and KEY_TYPE_UNIQUE is not null")
                    KEY_TYPES=KEY_TYPE.collect()[0][0]
                    print('KEY_TYPES: ',KEY_TYPES)

                

                    if KEY_TYPES == "N" :
                        tab_si_cnt += 1
                        print(tab_si_cnt)

                        doc.add_heading(f"{db}.{table_name}", 4)
                        
                        if Partition_column.count() == 0:
                            Partition_col = None
                            print(f'No partition column configured for {db}.{table_name}')
                            doc.add_paragraph(f'No partition column configured for {db}.{table_name}', style='List Bullet')
                        else:
                            Partition_col =Partition_column.collect()[0][0]
                            print(f'Partition column configured for {db}.{table_name} is {Partition_col}')
                            doc.add_paragraph(f'Partition column configured for {db}.{table_name} is {Partition_col}', style='List Bullet')

                        ## The ADLS silver table
                        Data_Lake_df=spark.read.format("delta").option("header","true").load("/mnt/silver/"+db+"/ADMIN/"+table_name)
                        Data_Lake_df.createOrReplaceTempView("Data_Lake_df")

                        if ((Partition_col != None) and (Partition_col != 'NA') ):
                            ## Retrieve dataframe of the queries for row count per partition for Netezza tables, and Retrieve dataframe of query for generating a measure from Netezza and ADLS_Gold
                            print(f'Retrieving Partition, Measure and Field Validation Queries from Fwk_Gold_Tables for {db}.{table_name}')
                            doc.add_paragraph(f'Retrieving Partition, Measure and Field Validation Queries from Fwk_Gold_Tables for {db}.{table_name}', style='List Bullet')
                            
                            
                            Query_Agg_Netezza=spark.sql(f"select distinct PARTITION_QUERY_NETEZZA from gold_df where TABLE_NAME = '{table_name}' and DATABASE ='{db}' and PARTITION_QUERY_NETEZZA is not null")
                            Query_Measure_Netezza=spark.sql(f" select distinct MEASURE_QUERY_NETEZZA from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and MEASURE_QUERY_NETEZZA is not null")
                            ##TODO- Include Query_Measure_Silver if it exists. Confirm the Query_Measure_Silver
                            Query_Measure_Silver=spark.sql(f" select distinct ADLS_QUERY_FOR_MEASURE from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and ADLS_QUERY_FOR_MEASURE is not null")

                            Query_Field_Level_Netezza=spark.sql(f"select distinct FIELD_VALIDATION_QUERY_NETEZZA from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and FIELD_VALIDATION_QUERY_NETEZZA is not null")
                            Query_Field_Level_Silver=spark.sql(f"select distinct ADLS_QUERY_FOR_FIELD from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and ADLS_QUERY_FOR_FIELD is not null")  


                            ## Retrieve dataframe of the field used for validation (YEAR, MONTH, DAY)
                            PARTITION_ON=spark.sql(f"select PARTITION_ON from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and PARTITION_ON is not null")
                            Partition_Type=PARTITION_ON.collect()[0][0].upper()
                            print('PARTITION TYPE:', Partition_Type)

                            ## Skip table if no queries are configured
                            if (Query_Field_Level_Netezza.count() == 0) & (Query_Measure_Netezza.count() == 0):
                                print(f"No field validation query or measure query configured. No detailed report generated for {db}.{table_name}")
                                doc.add_paragraph(f"No field validation query or measure query configured. No detailed report generated for {db}.{table_name}", style = 'List Bullet')
                                continue
                            

                            ## Get the various queries, partition field and partition value
                            '''
                            if Query_Agg_Netezza.count()!=0:
                                query_table_load_netezza =Query_Agg_Netezza.collect()[0][0]
                                netezza_df=readfromNetizza(Env,query_table_load_netezza,db)
                            '''

                            ## PERFORM PARTITION COUNT VALIDATION
                            if (Query_Agg_Netezza.count() == 0):
                                print(f'No partition queries configured for {db}.{table_name} in Fwk_Gold_Tables for Netezza. No partition count validation performed.')
                                doc.add_paragraph(f'No partition queries configured for {db}.{table_name} in Fwk_Gold_Tables for Netezza. No partition count validation performed.', style='List Bullet')
                                partition_count_validation_status = 'False'
                            else:
                                query_PARTITION_CNT_NZ = Query_Agg_Netezza.collect()[0][0]
                                if (query_PARTITION_CNT_NZ == ''):
                                    partition_count_validation_status = 'False'
                                    print('No Partition Query is configured. No partition count validation performed')
                                    doc.add_paragraph(('No Partition Query is configured. No partition count validation performed'), style= 'List Bullet')
                                else:
                                    print(f'Attempting to perform partition count validation for {db}.{table_name}')
                                    doc.add_paragraph(f'Attempting to perform partition count validation for {db}.{table_name}', style='List Bullet')

                                    try:
                                        query_PARTITION_CNT_NZ = query_PARTITION_CNT_NZ.replace('db_tmp', db_tmp)
                                        netezza_partition_cnt_df = readfromNetizza(Env, query_PARTITION_CNT_NZ, db)
                                        if Partition_Type == 'YEAR' :
                                            Data_Lake_df_PARTITION_CNT=spark.sql(f"select DATE_FORMAT({Partition_col}, 'yyyy') as {Partition_col}, count(*) as CNT from Data_Lake_df group by DATE_FORMAT({Partition_col}, 'yyyy')")
                                        elif Partition_Type == 'MONTH' :
                                            Data_Lake_df_PARTITION_CNT=spark.sql(f"select DATE_FORMAT({Partition_col}, 'yyyy MMM') as {Partition_col}, count(*) as CNT from Data_Lake_df group by DATE_FORMAT({Partition_col}, 'yyyy MMM')")
                                        elif Partition_Type == 'DAY' :
                                            Data_Lake_df_PARTITION_CNT=spark.sql(f"select DATE_FORMAT({Partition_col}, 'yyyy MMM dd') as {Partition_col}, count(*) as CNT from Data_Lake_df group by DATE_FORMAT({Partition_col}, 'yyyy MMM dd')")

                                        try:
                                            partition_count_mismatched_cnt, partition_count_mismatched_df, partition_count_mismatched_detail_cnt, partition_count_mismatched_detail_df = find_Mismatched(netezza_partition_cnt_df,Data_Lake_df_PARTITION_CNT,Partition_col)
                                            print(f"Successfully performed partition count validation for {db}.{table_name}")
                                            doc.add_paragraph(f"Successfully performed partition count validation for {db}.{table_name}", style = 'List Bullet')
                                            partition_count_validation_status = 'True'
                                        except Exception as e:
                                            partition_count_validation_status = 'False'
                                            print(f'Failed to perform partition count validation for {db}.{table_name}')
                                            doc.add_paragraph(f'Failed to perform partition count validation for {db}.{table_name}', style='List Bullet')
                                            print(e)
                                            doc.add_paragraph(f'{e}', style='Intense Quote')
                                    except Exception as e:
                                        partition_count_validation_status = 'False'
                                        print(f'Bad partition query configured for {db}.{table_name}. Please correct the query. No partition count validation performed')
                                        doc.add_paragraph(f'Bad partition query configured for {db}.{table_name}. Please correct the query. No partition count validation performed', style = 'List Bullet')
                                        print(e)
                                        doc.add_paragraph(f'{e}', style='Intense Quote')

                            

                            ## PERFORM MEASURE LEVEL VALIDATION

                            if (Query_Measure_Netezza.count() == 0) | (Query_Measure_Silver.count() == 0):
                                print(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.')
                                doc.add_paragraph(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.', style='List Bullet')
                                measure_validation_status = 'False'
                            else:
                                query_measure_netezza =Query_Measure_Netezza.collect()[0][0]
                                query_measure_silver = Query_Measure_Silver.collect()[0][0]
                                if (query_measure_netezza == '') | (query_measure_silver == ''):
                                    print(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.')
                                    doc.add_paragraph(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.', style='List Bullet')
                                    measure_validation_status = 'False' 
                                else:
                                    print(f'Attempting to perform measure level validation for {db}.{table_name}')
                                    doc.add_paragraph(f'Attempting to perform measure level validation for {db}.{table_name}', style='List Bullet')

                                    try:
                                        query_measure_netezza = query_measure_netezza.replace('db_tmp', db_tmp)
                                        netezza_df_measure=readfromNetizza(Env,query_measure_netezza,db)
                                        adls_silver_df_measure = spark.sql(f"{query_measure_silver}")

                                        try:
                                            measure_level_mismatched_cnt_diff, measure_level_mismatched_record_df, measure_level_mismatched_detail_cnt, measure_level_mismatched_detail_df = find_Mismatched(netezza_df_measure,adls_silver_df_measure,Partition_col)
                                            print(f"Successfully performed measure level validation for {db}.{table_name}")
                                            doc.add_paragraph(f"Successfully performed measure level validation for {db}.{table_name}", style='List Bullet')
                                            measure_validation_status = 'True'

                                        except Exception as e:
                                            measure_validation_status = 'False'
                                            print(f'Failed to perform measure level validation for {db}.{table_name}')
                                            doc.add_paragraph(f'Failed to perform measure level validation for {db}.{table_name}', style='List Bullet')
                                            print(e)
                                            doc.add_paragraph(f'{e}', style='Intense Quote')
                                    except Exception as e:
                                        measure_validation_status = 'False'
                                        print(f'Bad measure query configured for {db}.{table_name}. Please correct the query. No measure level validation performed')
                                        doc.add_paragraph(f'Bad measure query configured for {db}.{table_name}. Please correct the query. No measure level validation performed', style = 'List Bullet')
                                        print(e)
                                        doc.add_paragraph(f'{e}', style='Intense Quote')


                            ## PERFORM PARTITION LEVEL VALIDATION

                            if (Query_Field_Level_Netezza.count()==0) | (Query_Field_Level_Silver.count()==0):
                                print(f'No field validation queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No partition level validation performed.')
                                doc.add_paragraph(f'No field validation queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No partition level validation performed.', style='List Bullet')
                                partition_validation_status = 'False'
                            else:
                                query_PARTITION_NZ = Query_Field_Level_Netezza.collect()[0][0]
                                query_PARTITION_Silver = Query_Field_Level_Silver.collect()[0][0]
                                if (query_PARTITION_NZ == '') | (query_PARTITION_Silver == ''):
                                    print(f'No field validation queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No partition level validation performed.')
                                    doc.add_paragraph(f'No field validation queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No partition level validation performed.', style='List Bullet')
                                    partition_validation_status = 'False'  
                                else:
                                    try:
                                        query_PARTITION_NZ = query_PARTITION_NZ.replace('db_tmp', db_tmp)
                                        netezza_df_partition=readfromNetizza(Env,query_PARTITION_NZ,db)
                                        Data_Lake_df_1=spark.sql(query_PARTITION_Silver)  

                                        ## Validate that the partition size is within the max row permitted limit
                                        print('Attempting to perform partition level validation. Need to confirm that the partition size is within acceptable limit')
                                        doc.add_paragraph('Attempting to perform partition level validation. Need to confirm that the partition size is within acceptable limit', style = 'List Bullet')
                                        
                                        cnt_netezza_df_partition = netezza_df_partition.limit(max_rows_permitted + 1).count()
                                        cnt_Data_Lake_df_1 = Data_Lake_df_1.limit(max_rows_permitted + 1).count()

                                        if (cnt_netezza_df_partition == 0) | (cnt_Data_Lake_df_1 == 0):
                                            print(f'Partition for table {db}.{table_name} is empty either in Netezza or in Gold. No partition validation performed')
                                            doc.add_paragraph(f'Partition for table {db}.{table_name} is empty either in Netezza or in Gold. No partition validation performed', style = 'List Bullet')
                                            partition_validation_status = 'False'

                                        elif (cnt_netezza_df_partition > max_rows_permitted) | (cnt_Data_Lake_df_1 > max_rows_permitted):
                                            print(f'Partition for {db}.{table_name} is greater than {max_rows_permitted}. No partition validation performed. Please reconfigure the partition for {db}.{table_name}')
                                            doc.add_paragraph(f'Partition for {db}.{table_name} is greater than {max_rows_permitted}. No partition validation performed. Please reconfigure the partition for {db}.{table_name}', style = 'List Bullet')
                                            partition_validation_status = 'False'
                                        else:
                                            ## Perform validation on the partition
                                            print(f'Partition for {db}.{table_name} is less than or equal to {max_rows_permitted}.')
                                            doc.add_paragraph(f'Partition for {db}.{table_name} is less than or equal to {max_rows_permitted}.', style = 'List Bullet')
                                            try:
                                                #partition_level_diff = get_pandas(netezza_df_partition.subtract(Data_Lake_df_1))
                                                partition_level_diff = get_pandas(trim_space(netezza_df_partition).subtract(trim_space(Data_Lake_df_1)))
                                                partition_level_diff_cnt = len(partition_level_diff)
                                                print(f"Successfully performed partition level validation for {db}.{table_name}")
                                                doc.add_paragraph(f"Successfully performed partition level validation for {db}.{table_name}", style='List Bullet')
                                                partiton_validation_status = 'True'
                                            except Exception as e:
                                                partiton_validation_status = 'False'
                                                print(f'Failed to perform partiton level validation for {db}.{table_name}_(Netezza v. Silver)')
                                                doc.add_paragraph(f'Failed to perform partiton level validation for {db}.{table_name}_(Netezza v. Silver)', style='List Bullet')
                                                print(e)
                                                doc.add_paragraph(f'{e}', style='Intense Quote') 
                                    except Exception as e:
                                        partition_validation_status = 'False'
                                        print(f'Bad field validation query configured for {db}.{table_name}. Please correct the query. No partition level validation performed')
                                        doc.add_paragraph(f'Bad field validation query configured for {db}.{table_name}. Please correct the query. No partition level validation performed', style = 'List Bullet')
                                        print(e)
                                        doc.add_paragraph(f'{e}', style='Intense Quote')


                            ## Define excel writer
                            try:             
                                with pd.ExcelWriter(f"{db}-{table_name}_(Netezza v. Silver).xlsx", engine="openpyxl", mode = 'w') as writer:

                                    ## Write missing and mismatched information to sheets in excel workbook
                                    if partiton_validation_status == 'True':
                                        partition_level_diff.head(max_excel_row).to_excel(writer, sheet_name = f"partition_difference_record", startrow = 0, startcol=0, index = False)
                                    if measure_validation_status == 'True':
                                        try:
                                            measure_level_mismatched_record_df.sort_values(by=Partition_col).head(max_excel_row).to_excel(writer, sheet_name = "measure_mismatched_record", startrow = 0, startcol=0, index = False)
                                            measure_level_mismatched_detail_df.sort_values(by=Partition_col).head(max_excel_row).to_excel(writer, sheet_name = "measure_mismatched_detail", startrow = 0, startcol=0, index = False)
                                        except:
                                            measure_level_mismatched_record_df.head(max_excel_row).to_excel(writer, sheet_name = "measure_mismatched_record", startrow = 0, startcol=0, index = False)
                                            measure_level_mismatched_detail_df.head(max_excel_row).to_excel(writer, sheet_name = "measure_mismatched_detail", startrow = 0, startcol=0, index = False)
                                    if partition_count_validation_status == 'True':
                                        try:
                                            partition_count_mismatched_df.sort_values(by=Partition_col).head(max_excel_row).to_excel(writer, sheet_name = "partition_count_mismatched_record", startrow = 0, startcol=0, index = False)
                                            partition_count_mismatched_detail_df.sort_values(by=Partition_col).head(max_excel_row).to_excel(writer, sheet_name = "partition_count_mismatched_detail", startrow = 0, startcol=0, index = False)
                                        except:
                                            partition_count_mismatched_df.head(max_excel_row).to_excel(writer, sheet_name = "partition_count_mismatched_record", startrow = 0, startcol=0, index = False)
                                            partition_count_mismatched_detail_df.head(max_excel_row).to_excel(writer, sheet_name = "partition_count_mismatched_detail", startrow = 0, startcol=0, index = False)
                                print('Detailed report generated')
                                doc.add_paragraph('Detailed report generated', style = 'List Bullet')
                                detailed_report_status = 'True'
                            except Exception as e:
                                detailed_report_status = 'False'
                                print('No detailed report generated')
                                doc.add_paragraph('No detailed report generated', style = 'List Bullet')
                                print(e)
                                doc.add_paragraph(f'{e}', style= 'Intense Quote')
                                    

                            ## Upload workbook to storage
                            if detailed_report_status == 'True':
                                try:        
                                    workbook_path = path + f"{db}-{table_name}_(Netezza v. Silver).xlsx" 
                                    shutil.copy2(f"{db}-{table_name}_(Netezza v. Silver).xlsx",workbook_path)
                                    os.remove(f"{db}-{table_name}_(Netezza v. Silver).xlsx")

                                    ## Append detail summary to lists
                                    netezza_silver_database_name.append(db)
                                    netezza_silver_table_name.append(table_name)
                                    if partition_count_validation_status == 'True':
                                        netezza_silver_partition_count_mismatched_cnt.append(str(partition_count_mismatched_cnt))
                                    else:
                                        netezza_silver_partition_count_mismatched_cnt.append('')
                                    if measure_validation_status == 'True':
                                        netezza_silver_measure_level_mismatched_cnt_diff.append(str(measure_level_mismatched_cnt_diff))
                                    else:
                                        netezza_silver_measure_level_mismatched_cnt_diff.append('')
                                    netezza_silver_partition_level_mismatched_cnt_diff.append('')
                                    netezza_silver_partition_level_missing_cnt.append('')
                                    netezza_silver_table_level_mismatched_cnt_diff.append('')
                                    netezza_silver_table_level_missing_cnt.append('')
                                    if partiton_validation_status == 'True':
                                        netezza_silver_partition_level_diff_cnt.append(str(partition_level_diff_cnt))
                                    else:
                                        netezza_silver_partition_level_diff_cnt.append('')
                                    netezza_silver_table_level_diff_cnt.append('')
                                    netezza_silver_detailed_report_path.append(workbook_path)

                                    print(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Silver)")
                                    doc.add_paragraph(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Silver)", style='List Bullet')
                                    

                                except Exception as e:
                                    print(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Silver)")
                                    doc.add_paragraph(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Silver)", style='List Bullet')
                                    print(e)
                                    doc.add_paragraph(f'{e}', style='Intense Quote')


                            


                        else :
                            ## This condition is to perform detailed validation for small and medium size tables
                            print(f'Attempting to perform detailed validation on {db}.{table_name}. Need to confirm it is a small to medium size table')
                            


                            ## Retrieve measure level query
                            Query_Measure_Netezza=spark.sql(f" select distinct MEASURE_QUERY_NETEZZA from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and MEASURE_QUERY_NETEZZA is not null")
                            Query_Measure_Silver=spark.sql(f" select distinct ADLS_QUERY_FOR_MEASURE from gold_df where TABLE_NAME='{table_name}' and DATABASE ='{db}' and ADLS_QUERY_FOR_MEASURE is not null")

                            print("else Full load Non Transactional ")
                            ## Get FULL netezza table for validation
                            Query_Netezza = f"""select * from {db_tmp}.{table_name}"""
                            netezza_df=readfromNetizza(Env,Query_Netezza,db)
                            netezza_df.cache()

                                

                            ## Confirm table is small or medium size (i.e. Not greater than 5 million rows)
                            #max_rows_permitted = 5000000

                            try:
                                ## Retrieve row count
                                cnt_netezza_df = int(report_dataframe_2.select(f'NETEZZA_TABLE_ROW_COUNT ({netezza_sub})').filter((col('NETEZZA_SCHEMA') == f'{db}') & (col('NETEZZA_TABLE') == f'{table_name}')).collect()[0][0]) #netezza_df.limit(max_rows_permitted + 1).count()
                                cnt_df_ADLS_silver = int(report_dataframe_2.select('SILVER_TABLE_ROW_COUNT').filter((col('SILVER_SCHEMA') == f'{db}') & (col('SILVER_TABLE') == f'{table_name}')).collect()[0][0])#df_synapse.limit(max_rows_permitted + 1).count()
                            except:
                                print(f'{db}.{table_name} not available in summary report')
                                cnt_netezza_df = netezza_df.limit(max_rows_permitted + 1).count()
                                cnt_df_ADLS_silver = Data_Lake_df.limit(max_rows_permitted + 1).count()

                            if (cnt_netezza_df == 0) | (cnt_df_ADLS_silver == 0):
                                print(f'Table {db}.{table_name} is empty either in Netezza or in Silver. No detailed validation performed')
                                doc.add_paragraph(f'Table {db}.{table_name} is empty either in Netezza or in Silver. No detailed validation performed', style = 'List Bullet')

                            elif (cnt_netezza_df > max_rows_permitted) | (cnt_df_ADLS_silver > max_rows_permitted):
                                print(f'Table {db}.{table_name} is a big table, with no partition column configured. No table level validation performed')
                                doc.add_paragraph(f'Table {db}.{table_name} is a big table, with no partition column configured. No table level validation performed', style='List Bullet')

                                print('Will attempt to perform measure level validation')
                                doc.add_paragraph('Attempting measure level validation for a big table with no partition configured', style= 'List Bullet')

                                ## PERFORM MEASURE LEVEL VALIDATION
                                if (Query_Measure_Netezza.count() == 0) | (Query_Measure_Silver.count() == 0):
                                    print(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.')
                                    doc.add_paragraph(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.', style='List Bullet')
                                    measure_validation_status = 'False'
                                else:
                                    query_measure_netezza =Query_Measure_Netezza.collect()[0][0]
                                    query_measure_silver = Query_Measure_Silver.collect()[0][0]
                                    if (query_measure_netezza == '') | (query_measure_silver == ''):
                                        print(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.')
                                        doc.add_paragraph(f'No measure queries configured for {db}.{table_name} in Fwk_Gold_Tables, either for Netezza or ADLS. No measure level validation performed.', style='List Bullet')
                                        measure_validation_status = 'False' 
                                    else:
                                        print(f'Measure queries configured. Attempting to perform measure level validation for {db}.{table_name}')
                                        doc.add_paragraph(f'Measure queries configured. Attempting to perform measure level validation for {db}.{table_name}', style='List Bullet')

                                        try:
                                            query_measure_netezza = query_measure_netezza.replace('db_tmp', db_tmp)
                                            netezza_df_measure=readfromNetizza(Env,query_measure_netezza,db)
                                            adls_silver_df_measure = spark.sql(f"{query_measure_silver}")

                                            try:
                                                measure_level_mismatched_record_df = get_pandas(adls_silver_df_measure.subtract(netezza_df_measure))
                                                measure_level_mismatched_cnt_diff = len(measure_level_mismatched_record_df)
                                                print(f"Successfully performed measure mismatched validation on {db}.{table_name}. This is a big table does not have a partition column configured")
                                                doc.add_paragraph(f"Successfully performed measure mismatched validation on {db}.{table_name}. This is a big table does not have a partition column configured", style='List Bullet')
                                                measure_validation_status = 'True'


                                                ## Define excel writer
                                                with pd.ExcelWriter(f"{db}-{table_name}_(Netezza v. Silver).xlsx", engine="openpyxl", mode = 'w') as writer:
                                                    measure_level_mismatched_record_df.head(max_excel_row).to_excel(writer, sheet_name = f"measure_mismatched_record", startrow = 0, startcol=0, index = False)

                                                ## Upload workbook to storage
                                                try:        
                                                    workbook_path = path + f"{db}-{table_name}_(Netezza v. Silver).xlsx" 
                                                    shutil.copy2(f"{db}-{table_name}_(Netezza v. Silver).xlsx",workbook_path)
                                                    os.remove(f"{db}-{table_name}_(Netezza v. Silver).xlsx")

                                                    ## Append detail summary to lists
                                                    netezza_silver_database_name.append(db)
                                                    netezza_silver_table_name.append(table_name)
                                                    netezza_silver_partition_count_mismatched_cnt.append('')
                                                    netezza_silver_measure_level_mismatched_cnt_diff.append(str(measure_level_mismatched_cnt_diff))
                                                    netezza_silver_partition_level_mismatched_cnt_diff.append('')
                                                    netezza_silver_partition_level_missing_cnt.append('')
                                                    netezza_silver_table_level_mismatched_cnt_diff.append('')
                                                    netezza_silver_table_level_missing_cnt.append('')
                                                    netezza_silver_partition_level_diff_cnt.append('')
                                                    netezza_silver_table_level_diff_cnt.append('')
                                                    netezza_silver_detailed_report_path.append(workbook_path)

                                                    print(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Silver)")
                                                    doc.add_paragraph(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Silver)", style='List Bullet')

                                                except Exception as e:
                                                    print(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Silver)")
                                                    doc.add_paragraph(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Silver)", style='List Bullet')
                                                    print(e)
                                                    doc.add_paragraph(f'{e}', style='Intense Quote')
                                            
                                            except Exception as e:
                                                print(f"Failed to perform detailed validation on {db}.{table_name}")
                                                doc.add_paragraph(f"Failed to perform detailed validation on {db}.{table_name}", style='List Bullet')
                                                print(e)
                                                doc.add_paragraph(f'{e}', style='Intense Quote')
                                                continue

                                        except Exception as e:
                                            print(f'Bad measure query configured for {db}.{table_name}. Please correct the query. No measure level validation performed')
                                            doc.add_paragraph(f'Bad measure query configured for {db}.{table_name}. Please correct the query. No measure level validation performed', style = 'List Bullet')
                                            print(e)
                                            doc.add_paragraph(f'{e}', style='Intense Quote')
                                        

                            else:
                                print(f'Table size is less than or equal to {max_rows_permitted}. Performing detailed validation on full table {db}.{table_name}')
                                doc.add_paragraph(f'Table size is less than or equal to {max_rows_permitted}. Performing detailed validation on full table {db}.{table_name}', style = 'List Bullet')
                                
                                try:
                                    #table_level_diff = get_pandas(netezza_df.subtract(Data_Lake_df))
                                    table_level_diff = get_pandas(trim_space(netezza_df).subtract(trim_space(Data_Lake_df)))
                                    table_level_diff_cnt = len(table_level_diff)
                                    print(f'Successfully performed full table validation on {db}.{table_name}, comparing Netezza to Silver table')
                                    doc.add_paragraph(f'Successfully performed full table validation on {db}.{table_name}, comparing Netezza to Silver table', style='List Bullet')


                                    ## Define excel writer
                                    with pd.ExcelWriter(f"{db}-{table_name}_(Netezza v. Silver).xlsx", engine="openpyxl", mode = 'w') as writer:

                                        ## Write missing and mismatched information to sheets in excel workbook
                                        table_level_diff.head(max_excel_row).to_excel(writer, sheet_name = "difference_record", startrow = 0, startcol=0, index = False)
                                        

                                    ## Upload workbook to storage
                                    try:        
                                        workbook_path = path + f"{db}-{table_name}_(Netezza v. Silver).xlsx" 
                                        shutil.copy2(f"{db}-{table_name}_(Netezza v. Silver).xlsx",workbook_path)
                                        os.remove(f"{db}-{table_name}_(Netezza v. Silver).xlsx")

                                        ## Append detail summary to lists
                                        netezza_silver_database_name.append(db)
                                        netezza_silver_table_name.append(table_name)
                                        netezza_silver_partition_count_mismatched_cnt.append('')
                                        netezza_silver_measure_level_mismatched_cnt_diff.append('')
                                        netezza_silver_partition_level_mismatched_cnt_diff.append('')
                                        netezza_silver_partition_level_missing_cnt.append('')
                                        netezza_silver_table_level_mismatched_cnt_diff.append('')
                                        netezza_silver_table_level_missing_cnt.append('')
                                        netezza_silver_partition_level_diff_cnt.append('')
                                        netezza_silver_table_level_diff_cnt.append(table_level_diff_cnt)
                                        netezza_silver_detailed_report_path.append(workbook_path)

                                        print(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Silver)")
                                        doc.add_paragraph(f"Successfully uploaded detailed report for {db}.{table_name}_(Netezza v. Silver)", style='List Bullet')
                                        

                                    except Exception as e:
                                        print(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Silver)")
                                        doc.add_paragraph(f"Failed to upload detailed report for {db}.{table_name}_(Netezza v. Silver)", style='List Bullet')
                                        print(e)
                                        doc.add_paragraph(f'{e}', style='Intense Quote')


                                except Exception as e:
                                    print(f'Failed to generate detailed report for {db}.{table_name}, when comparing Netezza to Silver table')
                                    doc.add_paragraph(f'Failed to generate detailed report for {db}.{table_name}, when comparing Netezza to Silver table', style='List Bullet')
                                    print(e)
                                    doc.add_paragraph(f'{e}', style='Intense Quote')
                                    
                except Exception as e:
                    print(f'ERROR OCCURRED: Failed to generate detailed report for {db}.{table_name}, when comparing Netezza to Silver table')
                    doc.add_paragraph(f'ERROR OCCURRED: Failed to generate detailed report for {db}.{table_name}, when comparing Netezza to Silver table', style='List Bullet')
                    print(e)
                    doc.add_paragraph(f'{e}', style='Intense Quote')
                    #continue

        doc.add_paragraph('')

        ## Upload log file
        save_log(doc)

    except Exception as e:
        doc.add_paragraph("Data Validation Run Failed", style="List Bullet")
        print(e)
        doc.add_paragraph(f'{e}', style='Intense Quote')
        doc.add_paragraph('')
        
        ## Upload log file
        save_log(doc)

        ## Exit the notebook run
        dbutils.notebook.exit("Data Validation Run Failed") 
    
            

In [0]:
doc.add_heading('Update summary report', level=2)

try:

    ## Detail summary for Netezza v. Synapse

    dt_netezza_synapse = pd.DataFrame({'NETEZZA_SCHEMA':netezza_synapse_database_name, 'NETEZZA_TABLE':netezza_synapse_table_name,  'PARTITION_COUNT_MISMATCHED_RECORDS(NETEZZA v. SYNAPSE)_CNT': netezza_synapse_partition_count_mismatched_cnt, 'MEASURE_LEVEL_MISMATCHED_RECORDS(NETEZZA v. SYNAPSE)_CNT': netezza_synapse_measure_level_mismatched_cnt_diff, 'PARTITION_LEVEL_MISMATCHED_RECORDS(NETEZZA v. SYNAPSE)_CNT':netezza_synapse_partition_level_mismatched_cnt_diff, 'PARTITION_LEVEL_MISSING_RECORDS(NETEZZA v. SYNAPSE)_CNT':netezza_synapse_partition_level_missing_cnt, 'TABLE_LEVEL_MISMATCHED_RECORDS(NETEZZA v. SYNAPSE)_CNT': netezza_synapse_table_level_mismatched_cnt_diff, 'TABLE_LEVEL_MISSING_RECORDS(NETEZZA v. SYNAPSE)_CNT':netezza_synapse_table_level_missing_cnt, 'PARTITION_ROW_DIFFERENCE_CNT(NETEZZA v. SYNAPSE)(NON-UNIQUE KEY-TYPE)':netezza_synapse_partition_level_diff_cnt, 'TABLE_ROW_DIFFERENCE_CNT(NETEZZA v. SYNAPSE)(NON-UNIQUE KEY-TYPE)': netezza_synapse_table_level_diff_cnt, 'DETAILED_REPORT_PATH(NETEZZA v. SYNAPSE)':netezza_synapse_detailed_report_path})

    ## Detail summary dataframe for Netezza v. Gold

    dt_netezza_gold = pd.DataFrame({'NETEZZA_SCHEMA':netezza_gold_database_name, 'NETEZZA_TABLE':netezza_gold_table_name,  'PARTITION_COUNT_MISMATCHED_RECORDS(NETEZZA v. GOLD)_CNT': netezza_gold_partition_count_mismatched_cnt, 'MEASURE_LEVEL_MISMATCHED_RECORDS(NETEZZA v. GOLD)_CNT': netezza_gold_measure_level_mismatched_cnt_diff, 'PARTITION_LEVEL_MISMATCHED_RECORDS(NETEZZA v. GOLD)_CNT':netezza_gold_partition_level_mismatched_cnt_diff, 'PARTITION_LEVEL_MISSING_RECORDS(NETEZZA v. GOLD)_CNT':netezza_gold_partition_level_missing_cnt, 'TABLE_LEVEL_MISMATCHED_RECORDS(NETEZZA v. GOLD)_CNT': netezza_gold_table_level_mismatched_cnt_diff, 'TABLE_LEVEL_MISSING_RECORDS(NETEZZA v. GOLD)_CNT':netezza_gold_table_level_missing_cnt, 'PARTITION_ROW_DIFFERENCE_CNT(NETEZZA v. GOLD)(NON-UNIQUE KEY-TYPE)':netezza_gold_partition_level_diff_cnt, 'TABLE_ROW_DIFFERENCE_CNT(NETEZZA v. GOLD)(NON-UNIQUE KEY-TYPE)': netezza_gold_table_level_diff_cnt, 'DETAILED_REPORT_PATH(NETEZZA v. GOLD)':netezza_gold_detailed_report_path})

    ## Detail summary dataframe for Netezza v. Silver
    
    dt_netezza_silver = pd.DataFrame({'NETEZZA_SCHEMA':netezza_silver_database_name, 'NETEZZA_TABLE':netezza_silver_table_name, 'PARTITION_COUNT_MISMATCHED_RECORDS(NETEZZA v. SILVER)_CNT': netezza_silver_partition_count_mismatched_cnt, 'MEASURE_LEVEL_MISMATCHED_RECORDS(NETEZZA v. SILVER)_CNT': netezza_silver_measure_level_mismatched_cnt_diff, 'PARTITION_LEVEL_MISMATCHED_RECORDS(NETEZZA v. SILVER)_CNT':netezza_silver_partition_level_mismatched_cnt_diff, 'PARTITION_LEVEL_MISSING_RECORDS(NETEZZA v. SILVER)_CNT':netezza_silver_partition_level_missing_cnt, 'TABLE_LEVEL_MISMATCHED_RECORDS(NETEZZA v. SILVER)_CNT': netezza_silver_table_level_mismatched_cnt_diff, 'TABLE_LEVEL_MISSING_RECORDS(NETEZZA v. SILVER)_CNT':netezza_silver_table_level_missing_cnt,  'PARTITION_ROW_DIFFERENCE_CNT(NETEZZA v. SILVER)(NON-UNIQUE KEY-TYPE)':netezza_silver_partition_level_diff_cnt, 'TABLE_ROW_DIFFERENCE_CNT(NETEZZA v. SILVER)(NON-UNIQUE KEY-TYPE)': netezza_silver_table_level_diff_cnt, 'DETAILED_REPORT_PATH(NETEZZA v. SILVER)':netezza_silver_detailed_report_path})

    if (PL_TL == '7GS') | (Load_Type != 'Historical'): 
        summary_report_cols_1 = ['NETEZZA_SCHEMA','NETEZZA_TABLE', 'SYNAPSE_SCHEMA', 'SYNAPSE_TABLE','MISMATCHED_COLUMNS (NETEZZA - SYNAPSE)', 'MISMATCHED_COLUMNS (SYNAPSE - NETEZZA)',f'NETEZZA_TABLE_ROW_COUNT ({netezza_sub})', 'SYNAPSE_TABLE_ROW_COUNT', 'ROW_COUNT_DIFFERENCE(NETEZZA - SYNAPSE)','PARTITION_COUNT_MISMATCHED_RECORDS(NETEZZA v. SYNAPSE)_CNT',  'MEASURE_LEVEL_MISMATCHED_RECORDS(NETEZZA v. SYNAPSE)_CNT',  'PARTITION_LEVEL_MISMATCHED_RECORDS(NETEZZA v. SYNAPSE)_CNT', 'PARTITION_LEVEL_MISSING_RECORDS(NETEZZA v. SYNAPSE)_CNT', 'PARTITION_ROW_DIFFERENCE_CNT(NETEZZA v. SYNAPSE)(NON-UNIQUE KEY-TYPE)', 'TABLE_LEVEL_MISMATCHED_RECORDS(NETEZZA v. SYNAPSE)_CNT', 'TABLE_LEVEL_MISSING_RECORDS(NETEZZA v. SYNAPSE)_CNT',  'TABLE_ROW_DIFFERENCE_CNT(NETEZZA v. SYNAPSE)(NON-UNIQUE KEY-TYPE)', 'DETAILED_REPORT_PATH(NETEZZA v. SYNAPSE)','DETAILED_REPORT_PATH(NETEZZA v. GOLD)','Rpt_Exec_Ts']
    else:
        summary_report_cols_1 = ['NETEZZA_SCHEMA','NETEZZA_TABLE', 'SYNAPSE_SCHEMA', 'SYNAPSE_TABLE','MISMATCHED_COLUMNS (NETEZZA - SYNAPSE)', 'MISMATCHED_COLUMNS (SYNAPSE - NETEZZA)',f'NETEZZA_TABLE_ROW_COUNT ({netezza_sub})', 'SYNAPSE_TABLE_ROW_COUNT','GOLD_TABLE_ROW_COUNT', 'ROW_COUNT_DIFFERENCE(NETEZZA - SYNAPSE)', 'ROW_COUNT_DIFFERENCE(NETEZZA - GOLD)','PARTITION_COUNT_MISMATCHED_RECORDS(NETEZZA v. SYNAPSE)_CNT', 'PARTITION_COUNT_MISMATCHED_RECORDS(NETEZZA v. GOLD)_CNT','MEASURE_LEVEL_MISMATCHED_RECORDS(NETEZZA v. SYNAPSE)_CNT', 'MEASURE_LEVEL_MISMATCHED_RECORDS(NETEZZA v. GOLD)_CNT', 'PARTITION_LEVEL_MISMATCHED_RECORDS(NETEZZA v. SYNAPSE)_CNT', 'PARTITION_LEVEL_MISMATCHED_RECORDS(NETEZZA v. GOLD)_CNT', 'PARTITION_LEVEL_MISSING_RECORDS(NETEZZA v. SYNAPSE)_CNT','PARTITION_LEVEL_MISSING_RECORDS(NETEZZA v. GOLD)_CNT', 'PARTITION_ROW_DIFFERENCE_CNT(NETEZZA v. SYNAPSE)(NON-UNIQUE KEY-TYPE)', 'PARTITION_ROW_DIFFERENCE_CNT(NETEZZA v. GOLD)(NON-UNIQUE KEY-TYPE)', 'TABLE_LEVEL_MISMATCHED_RECORDS(NETEZZA v. SYNAPSE)_CNT', 'TABLE_LEVEL_MISMATCHED_RECORDS(NETEZZA v. GOLD)_CNT', 'TABLE_LEVEL_MISSING_RECORDS(NETEZZA v. SYNAPSE)_CNT', 'TABLE_LEVEL_MISSING_RECORDS(NETEZZA v. GOLD)_CNT',  'TABLE_ROW_DIFFERENCE_CNT(NETEZZA v. SYNAPSE)(NON-UNIQUE KEY-TYPE)', 'TABLE_ROW_DIFFERENCE_CNT(NETEZZA v. GOLD)(NON-UNIQUE KEY-TYPE)', 'DETAILED_REPORT_PATH(NETEZZA v. SYNAPSE)','DETAILED_REPORT_PATH(NETEZZA v. GOLD)','Rpt_Exec_Ts']

    #if len(relevant_gold_db) > 0:
    detailed_summary_1 = dt_netezza_synapse.merge(dt_netezza_gold, on = ['NETEZZA_SCHEMA', 'NETEZZA_TABLE'], how = 'outer')
        #detailed_summary_1 = detailed_summary_1[[summary_report_cols_1]]
    
    if (Load_Type == 'Historical'): 
        summary_report_cols_2 = ['NETEZZA_SCHEMA','NETEZZA_TABLE', 'SILVER_SCHEMA','SILVER_TABLE', 'MISMATCHED_COLUMNS (NETEZZA - SILVER)', 'MISMATCHED_COLUMNS (SILVER - NETEZZA)', f'NETEZZA_TABLE_ROW_COUNT ({netezza_sub})', 'SILVER_TABLE_ROW_COUNT', 'ROW_COUNT_DIFFERENCE(NETEZZA - SILVER)','PARTITION_COUNT_MISMATCHED_RECORDS(NETEZZA v. SILVER)_CNT','MEASURE_LEVEL_MISMATCHED_RECORDS(NETEZZA v. SILVER)_CNT','PARTITION_LEVEL_MISMATCHED_RECORDS(NETEZZA v. SILVER)_CNT','PARTITION_LEVEL_MISSING_RECORDS(NETEZZA v. SILVER)_CNT','PARTITION_ROW_DIFFERENCE_CNT(NETEZZA v. SILVER)(NON-UNIQUE KEY-TYPE)','TABLE_LEVEL_MISMATCHED_RECORDS(NETEZZA v. SILVER)_CNT','TABLE_LEVEL_MISSING_RECORDS(NETEZZA v. SILVER)_CNT','TABLE_ROW_DIFFERENCE_CNT(NETEZZA v. SILVER)(NON-UNIQUE KEY-TYPE)','DETAILED_REPORT_PATH(NETEZZA v. SILVER)', 'Rpt_Exec_Ts']
    
    #if len(relevant_silver_db) > 0:
    detailed_summary_2 = dt_netezza_silver   #.loc[:,[summary_report_cols_2]] 
     


    try:
        with pd.ExcelWriter("SUMMARY_REPORT.xlsx", engine="openpyxl", mode = 'w') as writer1:
            if len(relevant_gold_db) > 0:
                ## Read summary report
                summary_report_updated_1 = report_dataframe_pd_1.merge(detailed_summary_1, on = ['NETEZZA_SCHEMA', 'NETEZZA_TABLE'], how = 'outer')
                summary_report_updated_1 = summary_report_updated_1.loc[:,summary_report_cols_1]
                ## Write summary report for Netezza, Synapse and Gold to sheet in excel workbook
                if (PL_TL == '7GS') | (Load_Type != 'Historical'): 
                    sheet_name =  "NETEZZA v. SYNAPSE"
                else:
                    sheet_name =  "NETEZZA v. SYNAPSE v. GOLD"
                summary_report_updated_1.to_excel(writer1, sheet_name = sheet_name, startrow = 0, startcol=0, index = False)
            if (len(relevant_silver_db) > 0) & (Load_Type == 'Historical'):
                ## Read summary report
                summary_report_updated_2 = report_dataframe_pd_2.merge(detailed_summary_2, on = ['NETEZZA_SCHEMA', 'NETEZZA_TABLE'], how = 'outer')
                summary_report_updated_2 = summary_report_updated_2.loc[:,summary_report_cols_2]
                ## Write summary report for Netezza and Silver to sheet in excel workbook
                sheet_name =  "NETEZZA v. SILVER"
                summary_report_updated_2.to_excel(writer1, sheet_name = sheet_name, startrow = 0, startcol=0, index = False)
            
        workbook_path = path + 'SUMMARY_REPORT.xlsx'            
        shutil.copy2('SUMMARY_REPORT.xlsx',workbook_path)
        print('Successfully updated summary report')
        doc.add_paragraph('Successfully updated summary report', style='List Bullet')
        os.remove('SUMMARY_REPORT.xlsx')

    except Exception as e:
        print('Failed to update summary report')
        doc.add_paragraph('Failed to update summary report', style='List Bullet')
        print(e)
        doc.add_paragraph(f'{e}', style='Intense Quote')

    ## Upload log file
    save_log(doc)
    

except Exception as e:
    doc.add_paragraph("Data Validation Run Failed", style="List Bullet")
    print(e)
    doc.add_paragraph(f'{e}', style='Intense Quote')
    doc.add_paragraph('')
    ## Upload log file
    save_log(doc)

    ## Exit the notebook run
    #dbutils.notebook.exit("Data Validation Run Failed") 


Failed to update summary report
At least one sheet must be visible


In [0]:
from datetime import datetime
now = datetime.now()
now.strftime("%H:%M:%S")

'09:08:19'

In [0]:
if RptType == 'Detailed Report':
    dbutils.notebook.exit(RptType) 